Carrega um arquivo anotado com o PALAVRAS e armazena as estruturas de anotação em uma estrutura do Python

In [358]:
from bs4 import BeautifulSoup

In [387]:
# Monta dicionarios das tags

WORD_CLASS_TAGSET = {    
    "N": "Nouns",
    "PROP": "Proper nouns (names)",
    "SPEC": "Independent specifier pronouns (defined as non-inflecting pronouns, that can't be used as prenominals): e.g. indefinite pronouns, nominal quantifiers, nominal relatives",
    "DET": "Determiner pronouns (defined as inflecting pronouns, that can be used as prenominals): e.g. articles, attributive quantifiers",
    "PERS": "Personal pronouns (defined as person-inflecting pronouns)",
    "ADJ": "Adjectives (including ordinals, excluding participles 255 which are tagged V PCP)",
    "ADV": "Adverbs (both 'primary' adverbs and derived adverbs ending in '-mente')",
    "V": "Verbs (full verbs, auxiliaries)",
    "NUM": "Numerals (only cardinals)",
    "PRP": "Preposition",
    "KS": "Subordinating conjunctions",
    "KC": "Co-ordinating conjunctions",
    "IN": "Interjections",
    "EC": "Hyphen-separated prefix (\"elemento composto\", category being phased out)" 
}

WORD_SUBCLASS_TAGSET = {
    "<art>":"definite article (DET)",
    "<artd>":"definite article (DET)",
    "<arti>":"indefinite article (DET)",
    "<quant>":"quantifier (DET: <quant1>, <quant2>, <quant3>, SPEC: <quant0>)",
    "<quant0>":"quantifier (DET: <quant1>, <quant2>, <quant3>, SPEC: <quant0>)",
    "<quant1>":"quantifier (DET: <quant1>, <quant2>, <quant3>, SPEC: <quant0>)",
    "<quant2>":"quantifier (DET: <quant1>, <quant2>, <quant3>, SPEC: <quant0>)",
    "<quant3>":"quantifier (DET: <quant1>, <quant2>, <quant3>, SPEC: <quant0>)",
    "<dem>":"demonstrative pronoun (DET: <dem>, SPEC: <dem0>)",
    "<poss>":"possessive pronoun (DET)",
    "<refl>":"reflexive (\"se\" PERS ACC/DAT, \"si\" PERS PIV)",
    "<si>": "reflexive use of 3. person possessive",
    "<reci>": "reciprocal use of reflexive pronoun (= \"um ao outro\")",
    "<coll>": "collective reflexive (\"reunir-se\", \"associar-se\")", 
    "<diff>":"differentiator (DET) (e.g. \"outro\", \"mesmo\")",
    "<ident>": "identator (DET) (e.g. \"o próprio usuário\", \"a si mesmo\")",
    "<rel>":"relative pronoun (DET, SPEC)",
    "<interr>":"interrogative pronoun (DET, SPEC)",
    "<post-det>":"typically located as post-determiner (DET @N<)",
    "<post-attr>":"typically post-positioned adjective (ADJ @N<)",
    "<ante-attr>":"typically pre-positioned adjective (ADJ @>N)",
    "<pre-attr>":"obligatorily pre-positioned adjective (ADJ @>N, e.g. \"meio\")",
    "<adv>":"can be used adverbially (ADJ @ADVL)",
    "<post-adv>":"adverb occurring in post-nominal position (@N<, @A<, e.g. \"demais\", \"lá\")",
    "<KOMP> <igual>":"\"equalling\" comparative (ADJ, ADV) (e.g. \"tanto\", \"tão\")",
    "<KOMP> <corr>":"correlating comparative (ADJ, ADV) (e.g. \"mais velho\", \"melhor\")",
    "<komp> <igual>":"\"equalling\" particle referring to comparative (e.g. \"como\", \"quanto\")",
    "<komp> <corr>":"\"correlating\" particle referring to comparative (e.g. \"do=que\")",
    "<quant>":"intensity adverb (e.g. \"muito\")",
    "<setop>":"operational adverb (e.g. \"não\", \"nunca\", \"já\", \"mais\" in \"não mais\")",
    "<dei>":"discourse deictics (e.g. \"aqui\", \"ontem\")",
    "<ks>":"conjunctional adverbs (e.g. \"pois\")",
    "<kc>": "conjunctional adverb (pois, entretanto)",
    "<det>": "determiner usage/inflection of adverb (\"ela estava toda nua.\")",
    "<foc>": "focus marker adverb (also forms of \"ser\")", 
    "<prp>":"prepositional adverbs (e.g. \"conforme\", \"segundo\", \"como\")",
    "<SUP>": "superlative",
    "<setop>": "operational adverb (eg. \"não\", \"nunca\", \"já\", \"mais\" in \"não mais\")",    
    "<dei>": "discourse deictics (e.g. \"aqui\", \"ontem\")", 
    "<card>":"cardinal (NUM)",
    "<NUM-ord>":"ordinal (ADJ)",
    "<NUM-fract>":"fraction-numeral (N)",
    "<prop>": "noun, adjective etc. used as name (upper case initial in mid-sentence)",
    "<n>": "adjective or participle used as a \"noun\", typically as head of a nominal phrase",
    "<fmc>": "finite main clause heading verb",
    '<co-acc>': 'co-ordinator tags indicating what is co-ordinated: @ACC',
    '<co-advl>': 'co-ordinator tags indicating what is co-ordinated:  @ADVL',
    '<co-app>': 'co-ordinator tags indicating what is co-ordinated:  @APP',
    '<co-dat>': 'co-ordinator tags indicating what is co-ordinated:  @DAT',
    '<co-fmc>': 'co-ordinator tags indicating what is co-ordinated:  main clauses',
    '<co-ger>': 'co-ordinator tags indicating what is co-ordinated:  GER',
    '<co-inf>': 'co-ordinator tags indicating what is co-ordinated:  INF',
    '<co-oc>': 'co-ordinator tags indicating what is co-ordinated:  @OC',
    '<co-pcv>': 'co-ordinator tags indicating what is co-ordinated:  PCP-@IMV',
    '<co-postad>': 'co-ordinator tags indicating what is co-ordinated:  @A<',
    '<co-postnom>': 'co-ordinator tags indicating what is co-ordinated:  @N<', 
    '<co-pred>': 'co-ordinator tags indicating what is co-ordinated:  @PRED',
    '<co-prenom>': 'co-ordinator tags indicating what is co-ordinated:  @>N',
    '<co-prparg>': 'co-ordinator tags indicating what is co-ordinated:  @P<',
    '<co-sc>': 'co-ordinator tags indicating what is co-ordinated:  @SC',
    '<co-subj>': 'co-ordinator tags indicating what is co-ordinated:  @SUBJ',
    '<co-vfin>': 'co-ordinator tags indicating what is co-ordinated:  VFIN (ordered list matching the <co-...> tags)'    
}

TEXT_METAINFO_TAGSET = {
    '<cif>': "cipher (<card> NUM, <NUM-ord> ADJ)",
    '<sam->': "first part of morphologically fused word pair (\"de\" in \"dele\")",
    '<-sam>': "last part of morphologically fused word pair (\"ele\" in \"dele\")",
    '<*>': "1. letter capitalised",
    '<*1>': "left quote attached",
    '<*2>': "right quote attached",
    '<hyfen>': "hyphenated word",
    '<ABBR>': "abbreviation"
}


INFLEXION_TAGSET_ORIGINAL = {    
    "Gender": { 'M': "male", 'F': "female", 'M/F': "[for: N\', PROP\', SPEC\', DET, PERS, ADJ, V PCP, NUM]"},
    "Number": {'S': "Singular", 'P': "Plural", 
               'S/P': "[for: N, PROP', SPEC', DET, PERS, ADJ, V PCP, V VFIN, INF, NUM]"},
    "Case": {'NOM': "nominative", 'ACC': "accusative", 'DAT': "dative", 'PIV': "prepositive", 
             'ACC/DAT': "[for: PERS]", 'NOM/PIV': "[for: PERS]"},
    "Person": {'1': "first person", '2': "second person", '3': "third person", '1S': "[for: PERS, V VFIN, V INF]", 
               '1P':"[for: PERS, V VFIN, V INF]", '2S':"[for: PERS, V VFIN, V INF]", 
               '2P':"[for: PERS, V VFIN, V INF]", '3S':"[for: PERS, V VFIN, V INF]", 
               '3P':"[for: PERS, V VFIN, V INF]", '1/3S':"[for: PERS, V VFIN, V INF]",
               '0/1/3S':"[for: PERS, V VFIN, V INF]"
              },
    "Tense": {'PR': "present tense", 'IMPF': "imperfeito", 'PS': "perfeito simples", 'MQP': "mais-que-perfeito",
             'FUT': "futuro", 'COND': "conditional [for: V VFIN]"},
    "Mood": {'IND': "indicative", 'SUBJ': "subjunctive", "IMP": "imperative [for: V VFIN]"},
    "Finiteness": {'VFIN': "finite verb", 'INF': "infinitive", 'PCP': "participle", 'GER': "gerund [for: V]"}
}

INFLEXION_TAGSET = {
    'M': {'description': 'male', 'category': 'Gender'},
     'F': {'description': 'female', 'category': 'Gender'},
     'M/F': {'description': "[for: N', PROP', SPEC', DET, PERS, ADJ, V PCP, NUM]", 'category': 'Gender'},
     'S': {'description': 'Singular', 'category': 'Number'},
     'P': {'description': 'Plural', 'category': 'Number'},
     'S/P': {'description': "[for: N, PROP', SPEC', DET, PERS, ADJ, V PCP, V VFIN, INF, NUM]", 'category': 'Number'},
     'NOM': {'description': 'nominative', 'category': 'Case'},
     'ACC': {'description': 'accusative', 'category': 'Case'},
     'DAT': {'description': 'dative', 'category': 'Case'},
     'PIV': {'description': 'prepositive', 'category': 'Case'},
     'ACC/DAT': {'description': '[for: PERS]', 'category': 'Case'},
     'NOM/PIV': {'description': '[for: PERS]', 'category': 'Case'},
     '1': {'description': 'first person', 'category': 'Person'},
     '2': {'description': 'second person', 'category': 'Person'},
     '3': {'description': 'third person', 'category': 'Person'},
     '1S': {'description': '[for: PERS, V VFIN, V INF]', 'category': 'Person'},
     '1P': {'description': '[for: PERS, V VFIN, V INF]', 'category': 'Person'},
     '2S': {'description': '[for: PERS, V VFIN, V INF]', 'category': 'Person'},
     '2P': {'description': '[for: PERS, V VFIN, V INF]', 'category': 'Person'},
     '3S': {'description': '[for: PERS, V VFIN, V INF]', 'category': 'Person'},
     '3P': {'description': '[for: PERS, V VFIN, V INF]', 'category': 'Person'},
     '1/3S': {'description': '[for: PERS, V VFIN, V INF]', 'category': 'Person'},
     '0/1/3S': {'description': '[for: PERS, V VFIN, V INF]', 'category': 'Person'},
     'PR': {'description': 'present tense', 'category': 'Tense'},
     'IMPF': {'description': 'imperfeito', 'category': 'Tense'},
     'PS': {'description': 'perfeito simples', 'category': 'Tense'},
     'MQP': {'description': 'mais-que-perfeito', 'category': 'Tense'},
     'FUT': {'description': 'futuro', 'category': 'Tense'},
     'COND': {'description': 'conditional [for: V VFIN]', 'category': 'Tense'},
     'IND': {'description': 'indicative', 'category': 'Mood'},
     'SUBJ': {'description': 'subjunctive', 'category': 'Mood'},
     'IMP': {'description': 'imperative [for: V VFIN]', 'category': 'Mood'},
     'VFIN': {'description': 'finite verb', 'category': 'Finiteness'},
     'INF': {'description': 'infinitive', 'category': 'Finiteness'},
     'PCP': {'description': 'participle', 'category': 'Finiteness'},
     'GER': {'description': 'gerund [for: V]', 'category': 'Finiteness'}
}

SYNTATIC_TAGSET = {
    '@SUBJ>':"subject",
    '@<SUBJ':"subject",
    '@ACC>':"accusative (direct) object",
    '@<ACC':"accusative (direct) object",
    '@DAT>':"dative object (only pronominal)",
    '@<DAT':"dative object (only pronominal)",
    '@PIV>':"prepositional object",
    '@<PIV':"prepositional object",
    '@ADV>':"adverbial object (place, time, duration, quantity, cp. <va>, <vta>)",
    '@<ADV':"adverbial object (place, time, duration, quantity, cp. <va>, <vta>)",
    '@SC>':"subject predicative complement",
    '@<SC':"subject predicative complement",
    '@OC>':"object predicative complement",
    '@<OC':"object predicative complement",
    '@ADVL>':"adjunct adverbial",
    '@<ADVL':"adjunct adverbial",
    '@ADVL':"stray adverbial (in non-sentence expression)",
    '@NPHR':"stray noun phrase (in non-sentence expression without a top-node verb)",
    '@VOK':"'vocative' (e.g. 'free' addressing proper noun in direct speech)",
    '@>N':"prenominal adject (attaches to the nearest NP-head to the right, that is not an adnominal itself)",
    '@N<':"postnominal adject (attaches to the nearest NP-head to the left, that is not an adnominal itself)",
    '@>A':"adverbial pre-adject (intensifier adject) (attaches to the nearest ADJ/PCP/ADV or to a attributively used N <attr> to the right)",
    '@A<':"adverbial post-adject (rare as modifier: “caro demais”, more common as argument of adjective: “rico em”, or participle)",
    '@A<PIV/ADVL/SC':"arguments or adjuncts of an attributively used PCP",
    '@APP':"identifying apposition (always after NP + comma)",
    '@PRED>':"'forward' free predicative adjunct (refers to the following @SUBJ, even when this is incorporated in the VP)",
    '@<PRED':"‘backward’ free predicative adjunct or predicative adject (refers - as adject - to the nearest NP-head to the left, or - as adjunct - to the nearest main verb and its subject to the left)",
    '@N<PRED':"predicate in small clause introduced by 'com/sem' (rare, e.g. 'com a mão na bolsa', 'sem o pai ajudando, não conseguiu'); also used - in constituent grammar transformation - to indicate adject predicatives (i.e. group level @<PRED)",
    '@P<':"argument of preposition",
    '@>P':"modifier of prepositional phrase (\"até em casa\", \"muito de propósito\")",
    '@FAUX':"finite auxiliary (cp. @#ICL-AUX<)",
    '@FMV':"finite main verb",
    '@IAUX':"infinite auxiliary (cp. @#ICL-AUX<)",
    '@IMV':"infinite main verb",
    '@PRT-AUX<':"verb chain particle (preposition or \"que\" after auxiliary)",
    '@CO':"co-ordinating conjunction",
    '@SUB':"subordinating conjunction",
    '@KOMP<':"argument of comparative (e.g. \"do que\" referring to \"melhor\")",
    '@COM':"direct comparator without preceding comparative (e.g. \"word like\")",
    '@PRD':"role predicator (e.g. \"work as\", \"function as\")",
    '@#FS-':"finite subclause (combines with clausal role and intraclausal word tag, e.g. @#FS-<ACC @SUB for \"não acredito que seja verdade\")",
    '@#ICL-':"infinite subclause (combines with clausal role and intraclausal word tag, e.g. @#ICL-SUBJ> @IMV in \"consertar um relógio não é fácil\")",
    '@#ICL-AUX<':"argument verb in verb chain, refers to preceding auxiliary",
    '@#AS-':"'absolute' (i.e. verbless) subclause (combines with clausal role and intraclausal word tag, e.g. @#AS-<ADVL @ADVL> in \"ajudou onde possível\")",
    '@AS<':"argument of complementiser in absolute subclause",
    '@S<':"statement predicative (sentence apposition) (refers back to the whole preceding statement: \"não venceu o que muito o contrariou\")",
    '@FOC':"focus marker (\"gosta é de carne\")",
    '@<ACC-PASS':"passive use of pronoun se",
    '@<ADVS':"adverbial argument",
    '@<ADVO':"adverbial argument",
    '@<FOC':"focus marker (or right focus bracket)",
    '@<PASS':"agent of passive",
    '@>S':"modifier of clause (intensifier, operator or focus adverb)",
    '@A<ADV':"adverbial argument of attributive participle",
    '@A<ADVL':"adverbial adjunct of attributive participle",
    '@A<PASS':"agent of passive after attributive participle",
    '@A<PIV':"prepositional object of attributive participle",
    '@A<SC':"subject complement of attributive participle",
    '@ACC>-PASS':"passive use of pronoun se",
    '@ACC>>':"double-fronted accusative (direct) object before matrix verb",
    '@ADVS>':"adverbial argument",
    '@ADVO>':"adverbial argument",
    '@ADVL>A':"adjunct adverbial before attributive participle",
    '@ADVL>AS<':"adjunct adverbial in averbal clause",
    '@FOC>':"focus marker (or left focus bracket)",
    '@NUM<':"second part of numeral chain",
    '@PREF':"prefix (category being phased out)",
    '@SUBJ>>':"double-fronted subject, with interfering matrix og quoting verb",
    '@TOP':"topic constituent",
    '@AS-<ADVL': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@AS-<SC': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@AS-A<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@AS-ADVL': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@AS-ADVL>': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@AS-KOMP<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@AS-N<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-<<ACC': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-<ACC': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-<ADVS': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-<ADVL': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-<OC': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-<SC': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-<SUBJ': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-A<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-ACC>': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-ACC>>': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-ADVL': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-ADVL>': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-AS<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-KOMP<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-N<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-P<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-S<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@FS-SUBJ>': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-<ACC': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-<ADVL': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-<OC': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-<PRED': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-<SC': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-<SUBJ': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-ACC>': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-ADVL': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-ADVL>': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-APP': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-AS<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-AUX<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-IMV': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-N<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-N<PRED': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-NPHR': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-P<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-PRED>': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
     '@ICL-SUBJ>': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)'
}

VERBAL_VALENCY_TAGSET = {     
    'aux': "Auxiliary. Reference to @FAUX (new: <aux>) finite auxiliary (cp. @#ICL-AUX<)",
    '<mv>': "Main Verb. Reference to @FMV (new: <mv>) finite main verb",
    '<vt>': "monotransitive verb with accusative (direct) object",
    '<vi>': "intransitive verb (ideally, inergative)",
    '<ve>': "ergative (inaccusative) verbs. <ve> was introduced for corpus data reasons, to capture verbs with likely post-positioned “internal” (patient) subject, absolute participle constructions etc. The <ve> group may grow further on the expense of the <vi> group, and therefore, <vi> and <ve> can not (yet) be regarded as disjunct concepts (of inergative and inaccusative, respectively).",
    '<vtd>': "ditransitive verb with accusative and dative objects",
    '<PRP^vp>': "monotransitive verb with prepositional object (headed by PRP)",
    '<PRP^vtp>': "ditransitive verb with accusative and prepositional objects",
    '<vK>': "copula verb with subject predicative complement",
    '<vtK>': "copula verb with object predicative complement",
    '<va>': "transitive verb with adverbial argument relating to the subject: <va+LOC>, <va+DIR>,",
    '<va+LOC>': "transitive verb with NP as quantitative adverbial object (e.g. \"pesar\")",
    '<va+DIR>': "transitive verb with NP as quantitative adverbial object (e.g. \"pesar\")",
    '<vta+LOC>': "transitive verb with NP as quantitative adverbial object (e.g. \"pesar\")",
    '<vta+DIR>': "transitive verb with NP as quantitative adverbial object (e.g. \"pesar\")",
    '<vt+QUANT>': "transitive verb with NP as quantitative adverbial object (e.g. \"pesar\")",    
    '<vta>': "transitive verb with adverbial argument relating to the object: <vta+LOC>, <vta+DIR>",
    '<vt+QUANT>': "transitive verb with NP as quantitative adverbial object (e.g. \"pesar\")",
    '<vt+TID>': "transitive verb with NP as temporal adverbial object (e.g. \"durar\")", 
    '<vt+TEMP>': "transitive verb with NP as temporal adverbial object (e.g. \"durar\")",
    '<vU>': "\"impersonal\" verbs (normally in the 3S-person, e.g. \"chove\")",
    '<x>': "governs infinitive (as auxiliaries tagged @(F)AUX - @#ICL-AUX<)",
    '<x+PCP>': "governs participle (all are auxiliaries, tagged @(F)AUX - @#ICL-AUX<)",
    '<x+GER>': "governs gerund (all are auxiliaries, tagged @(F)AUX - @#ICL-AUX<)",
    '<PRP^xp>': "governs preposition mediated infinitive (as auxiliaries tagged as @(F)AUX - @PRT-AUX< - @#ICL-AUX<)",
    '<xt>': "governs infinitive clause with subject in the accusative case (e.g. ACI- and causative constructions, tagged as @(F)MV - @SUBJ> - @#ICL-ACC)",
    '<PRP^xtp>': "governs accusative object and prepositional object containing an infinitive clause with its (unexpressed) subject being identical to the preceding accusative object, tagged as @(F)MV - @<ACC - @<PIV - @#ICL-P<)",
    '<vr>': "reflexive verbs (also <vrp>, <xr>, <xrp>)",
    '<vq>': "\"cognitive\" verb governing a \'que\'-sentence",
    '<PRP^vpq>': "\"cognitive\" verb governing a prepositional phrase with a 'que'-sentence",
    '<qv>': "\"impersonal\" verb with 'que'-subclause as subject predicative (\"parece que\")",
    '<+interr>': "\"discourse\" verb or nominal governing an interrogative subclause"
}

NOMINAL_VALENCY_TAGSET = {
    '<+n>': "noun governing a name (PROP) (e.g. \"o senhor X\")",
    '<+num>': "noun governing a number (e.g. \"cap. 7\", \"no dia 5 de dezembro\")",
    '<num+>': "\"unit\" noun (e.g. \"20 metros\")",
    '<attr>': "attributive noun (e.g. \"um presidente comunista\")",
    '<mass>': "mass noun (e.g. \"leite\", \"água\")",
    '<+INF>': "nominal governing infinitive (N, ADJ)",
    '<+PRP>': "nominal governing prepositional phrase headed by PRP, e.g. <+sobre>",
    '<PRP+>': "(typically) argument of preposition PRP",
    '<+que>': "nominal governing a 'que'-subclause (N, ADJ)",
    '<+PRP+que>': "nominal governing a 'que'-subclause (N, ADJ)"
}

SEMANTIC_TAGSET = {
    '<H>': {'umbrella': True,
          'description': '+HUM-noun, human being -> cf. more specific categories below Human, umbrella tag Human prototypes',
          'examples': ['achacador',
           "'kidnapper'",
           'acionário',
           "'shareholder'",
           'inimigo',
           "'enemy'"],
          'category': 'ANIMATE HUMAN'},
     '<HM>': {'umbrella': False,
              'description': 'mystical or religious entity, constellations in anjo ‘angel’, duende ‘goblin’, astrology',
              'examples': ["anjo 'angel'",
               "duende 'goblin'",
               "hidra 'Hydra'",
               "tauro 'Taurus'"],
              'category': 'ANIMATE HUMAN'},
     '<N>': {'umbrella': False,
              'description': '',
              'examples': ["cigano 'gypsy'", "escandinavo 'Scandinavian'"],
              'category': 'ANIMATE HUMAN'},
     '<prof>': {'umbrella': False,
              'description': 'professional',
              'examples': ["-or: escritor 'author'", "filósofo 'philosopher'"],
              'category': 'ANIMATE HUMAN'},
     '<fam>': {'umbrella': False,
              'description': 'family member',
              'examples': ["pai 'father'", "mãe 'mother'"],
              'category': 'ANIMATE HUMAN'},
     '<title>': {'umbrella': False,
              'description': '+HUM, often governing name (a) regular title (b) others, apart from <prof> or <fam>',
              'examples': ["(a) rei 'king', presidente 'president', senhor 'mister'",
               "(b) moça 'girl', colega 'collegue'"],
              'category': 'ANIMATE HUMAN'},
     '<n+>': {'umbrella': False,
              'description': '-HUM, potencially governing name',
              'examples': ["restaurante 'restaurant'", "plano 'plan'", "rua 'street'"],
              'category': 'ANIMATE HUMAN'},
     '<attr>': {'umbrella': False,
              'description': 'attributive +HUM noun, often used as @N< (human) attributive adjective (not fully implemented, cp. <Hattr>, e.g. "um presidente COMUNISTA") OTHER SEMANTICALLY INSPIRED CATEGORIES',
              'examples': ["comunista /'communist/'"],
              'category': 'ANIMATE HUMAN'},
     '<HH>': {'umbrella': False,
              'description': 'group of H Group of humans (organisations, teams, companies, e.g. editora) Human prototypes',
              'examples': ["companhia 'company'", "equipe 'team'"],
              'category': 'ANIMATE HUMAN'},
     '<parti>': {'umbrella': False,
                  'description': '(political) party',
                  'examples': ["PT 'Labour'"],
                  'category': 'ANIMATE HUMAN'},
     '<inst>': {'umbrella': False,
              'description': 'institution [also animate human] (EMPRESA) organized site (e.g. restaurant, cp. <Linst>) Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)',
              'examples': ["igreja /'church/'",
               "polícia /'police/'",
               "-ria: padaria /'bakery/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<h>': {'umbrella': False,
              'description': '+HUM-adjective',
              'examples': ["jubiloso 'jubilant'", "louco 'crazy'"],
              'category': 'ANIMATE HUMAN'},
     '<n>': {'umbrella': False,
              'description': 'nationalaty-adective',
              'examples': ["dinamarquês /'Danish/'"],
              'category': 'ANIMATE HUMAN'},
     '<A>': {'umbrella': True,
              'description': '+ANIM,-HUM Animal, umbrella tag (clone, fêmea, fóssil, parasito, predador) Animal prototypes',
              'examples': [],
              'category': 'ANIMATE NON-HUMAN, MOVING'},
     '<AM>': {'umbrella': False,
              'description': 'mythological animal',
              'examples': ["pégaso 'Pegasus'", "licorne 'unicorn'"],
              'category': 'ANIMATE NON-HUMAN, MOVING'},
     '<AB>': {'umbrella': False,
              'description': 'bacteria, cells',
              'examples': ["macrófago 'macrophage'"],
              'category': 'ANIMATE NON-HUMAN, MOVING'},
     '<zo>': {'umbrella': False,
              'description': 'animal, including mammals',
              'examples': ["aligátor 'alligator'",
               "tênia 'tape-worm'",
               "babuíno 'baboon'"],
              'category': 'ANIMATE NON-HUMAN, MOVING'},
     '<D>': {'umbrella': False,
              'description': 'mammal, especially domestic',
              'examples': ["cavalo 'horse'", "vaca 'cow'"],
              'category': 'ANIMATE NON-HUMAN, MOVING'},
     '<orn>': {'umbrella': False,
              'description': 'bird',
              'examples': ["bem-te-vi 'Pitangus-bird'", "canário 'canary'"],
              'category': 'ANIMATE NON-HUMAN, MOVING'},
     '<ent>': {'umbrella': False,
              'description': 'insect',
              'examples': ["saúva 'ant'", "formiga 'ant'"],
              'category': 'ANIMATE NON-HUMAN, MOVING'},
     '<ich>': {'umbrella': False,
              'description': 'fish',
              'examples': ["perca 'perch'", "lobo-marinho 'sea-lion'"],
              'category': 'ANIMATE NON-HUMAN, MOVING'},
     '<AA>': {'umbrella': False,
              'description': 'group of animals (experimental: 4 AAorn, 5 DD) Group of animals (cardume, enxame, passarada, ninhada) Animal prototypes',
              'examples': ["manada 'herd'",
               "matilha 'pack'",
               "cria 'brood'",
               "vacada 'cow-herd'"],
              'category': 'ANIMATE NON-HUMAN, MOVING'},
     '<bo>': {'umbrella': False,
              'description': 'plant (general)',
              'examples': ["madressilva /'honeysuckle/'"],
              'category': 'ANIMATE NON-HUMAN, NON-MOVING'},
     '<B>': {'umbrella': True,
              'description': 'tree or bush (under, near, in) Plant, umbrella tag Plant prototypes',
              'examples': ['tree or bush (under, near, in)'],
              'category': 'ANIMATE NON-HUMAN, NON-MOVING'},
     '<BB>': {'umbrella': False,
              'description': 'group of plants (cp. topologica), place as defined by what grows on it Group of plants, plantation (field, forest etc.: mata, nabal) Plant prototypes',
              'examples': ["faial 'beech forest'", "floresta 'forest'", "roça 'field'"],
              'category': 'ANIMATE NON-HUMAN, NON-MOVING'},
     '<top>': {'umbrella': False,
              'description': 'toponym or natural topological (GEOGRAFICO) geographical location (cp. <Ltop>) Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)',
              'examples': ["Brasília, monte /'mountain/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<agua>': {'umbrella': False,
              'description': 'body of water (where one swims)',
              'examples': ["rio 'river'", "mar 'sea'", "laguna 'lagoon'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<sky>': {'umbrella': False,
              'description': 'sky, space, air space (where one flies)',
              'examples': ["céu 'sky'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<vej>': {'umbrella': False,
              'description': 'path, road (where one walks or drives)',
              'examples': ["rua 'street'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<topabs>': {'umbrella': False,
                  'description': 'abstract topological',
                  'examples': ["fim /'end/'", "curvatura /'curvature/'"],
                  'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<spids>': {'umbrella': False,
                  'description': 'point, tip',
                  'examples': ["farpa /'barb/'",
                   "pico /'sharp point, peak/'",
                   "cume /'peak/'"],
                  'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<area>': {'umbrella': False,
              'description': 'area, region',
              'examples': ["área /'area/'", "terreno /'area/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<hul>': {'umbrella': False,
              'description': 'hole or cavity, notch or groove',
              'examples': ["poro /'pore/'", "valeta /'gutter/'", "rasgão /'tear, gash/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<ejo>': {'umbrella': False,
              'description': 'functional place',
              'examples': ["quarto /'room/'", "banheiro /'bathroom/'", '-douro'],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<hus>': {'umbrella': False,
              'description': 'building [also cc]',
              'examples': ["casa /'house/'", "casebre /'hut/'", "torre /'tower/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<by>': {'umbrella': False,
              'description': 'group of houses, town, administrative unit country, state',
              'examples': ["vila /'village/'", "cidade /'town/'", "estado /'state/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<ta>': {'umbrella': False,
              'description': 'arquitectural feature',
              'examples': ["trave /'beam/'", "janela /'window/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<tm>': {'umbrella': False,
              'description': 'piece of furniture [also cc]',
              'examples': ['cadeira ‘chair’, mesa ‘table’'],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<fælde>': {'umbrella': False,
              'description': 'trap, snare',
              'examples': ["nassa /'wicker basket/'", "ratoeira /'mouse trap/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<kovr>': {'umbrella': False,
              'description': 'blanket, carpet, curtain, cover, lid (what things can be under)',
              'examples': ["tampa /'lid/'", "manta^259 /'blanket/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<ujo>': {'umbrella': False,
              'description': 'container',
              'examples': ["copo /'cup/'", "garrafa /'bottle/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<rør>': {'umbrella': False,
              'description': 'tube',
              'examples': ["tubo /'tube/'", "oleoduto /'pipeline/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<bild>': {'umbrella': False,
              'description': 'picture',
              'examples': ["pintura /'picture/'", "grafiti /'graffiti/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<r>': {'umbrella': False,
              'description': "things you can read and touch. All r (touchable readables) are also rr (readables), but not vice versa, as with poema 'poem'.",
              'examples': ["livro /'book/'", "jornal /'newspaper/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<bar>': {'umbrella': False,
              'description': 'fence or hedge, dike, dam (s.th. you pass over) barrier noun (dique, limite, muralha) Thing prototypes',
              'examples': ["fronteira /'border/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<dir>': {'umbrella': False,
              'description': 'direction direction noun (estibordo, contrasenso, norte) Collective prototypes',
              'examples': ["lés-sueste 'eastsoutheast'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<stil>': {'umbrella': False,
              'description': 'position (you hold)',
              'examples': ["presidência /'presidency/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<sit>': {'umbrella': False,
              'description': 'situation, state of affairs (in+) psychological situation or physical state of affairs (reclusão, arruaça, ilegalidade, more complex and more "locative" than <state> and <state-h> State-of-affairs prototypes',
              'examples': ["caos /'chaos/'", "circunstâncias /'circumstances/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<vejr>': {'umbrella': False,
              'description': 'weather',
              'examples': ["gelada /'hoar frost/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<vind>': {'umbrella': False,
              'description': 'wind',
              'examples': ["furacão /'hurricane/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<regn>': {'umbrella': False,
              'description': 'rain',
              'examples': ["chuvisco /'drizzle/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<an>': {'umbrella': True,
              'description': 'anatomical site (HUM) Anatomical noun, umbrella tag (carótida, clítoris, dorso) Anatomical prototypes',
              'examples': ["dorso /'back/'", "entranhas /'guts/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<anmov>': {'umbrella': False,
              'description': 'movable anatomical part-> +PL (hand, finger, head) Movable anatomy (arm, leg, braço, bíceps, cotovelo) Anatomical prototypes',
              'examples': ["sobrancelha /'eyebrow/'", "tentáculo /'tentacle/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<anorg>': {'umbrella': False,
              'description': 'anatomical organ-> +PL (in+) Organ (heart, liver, hipófise, coração, testículo) Anatomical prototypes',
              'examples': ["coração /'heart/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<anost>': {'umbrella': False,
              'description': 'bone Bone (calcâneo, fíbula, vértebra) Anatomical prototypes',
              'examples': ["fêmur /'thighbone/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<anfeat>': {'umbrella': False,
              'description': '(uncountable) anatomical "wearable" (grimace, tan, hair)-> <anfeatc> (countable anatomical feature=',
              'examples': ["queixo-duplo /'double chin/'",
               "riso=amarelo /'forced smile/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<anzo>': {'umbrella': False,
              'description': 'animal anatomy Animal anatomy (rúmen, carapaça, chifres, tromba) Anatomical prototypes',
              'examples': ["colmilho /'tusk/'", "focinha /'snout/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<anorn>': {'umbrella': False,
                  'description': 'bird anatomy Bird anatomy (bico, pluma) Anatomical prototypes',
                  'examples': ["pluma /'feather/'", "rostro /'beak/'"],
                  'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<anich>': {'umbrella': False,
                  'description': ' Fish anatomy (few: bránquias, siba) Anatomical prototypes',
                  'examples': [''],
                  'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<anent>': {'umbrella': False,
                  'description': 'insect anatomy Insect anatomy (few: tentáculo, olho composto) Anatomical prototypes',
                  'examples': ["rostro /'proboscis/'"],
                  'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<anbo>': {'umbrella': False,
              'description': 'plant anatomy Plant anatomy (bulbo, caule, folha) Anatomical prototypes',
              'examples': ["drupa /'drupe/'", "estame /'stamen/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<star>': {'umbrella': False,
              'description': 'star (cf. topologica)',
              'examples': ["estrela /'star/'", "planeta /'planet/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<surf>': {'umbrella': False,
              'description': 'surface (2-dimensional, on+)',
              'examples': ["chão /'floor, ground/'",
               "superfície /'surface/'",
               "face /'front, side/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<DIST>': {'umbrella': False,
              'description': 'distance (after <vt+DIST>)',
              'examples': ["légua /'mile/'"],
              'category': 'TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)'},
     '<V>': {'umbrella': False,
              'description': 'vehicle (PRODUTO) vehicle brand (cp. <V>, <Vair>, <Vwater>) Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)',
              'examples': ["carro /'car/'", "bicleta /'bicycle/'"],
              'category': 'CONCRETA, MOVING'},
     '<skib>': {'umbrella': False,
              'description': 'ship',
              'examples': ["navio /'ship/'", "iate /'yacht/'"],
              'category': 'CONCRETA, MOVING'},
     '<fly>': {'umbrella': False,
              'description': 'plane',
              'examples': ["teco-teco /'little plane/'", "pára-quedas /'parachute/'"],
              'category': 'CONCRETA, MOVING'},
     '<VV>': {'umbrella': False,
              'description': 'group of vehicles Group of vehicles (armada, convoy: frota, esquadra) Vehicle prototypes',
              'examples': ["armada /'fleet/'", "comboio /'convoy/'"],
              'category': 'CONCRETA, MOVING'},
     '<or>': {'umbrella': False,
              'description': 'machine',
              'examples': ["britadeira /'stone crusher/'"],
              'category': 'CONCRETA, MOVING'},
     '<cm>': {'umbrella': True,
              'description': 'physical mass nouns (+CONCRETE, +MASS) concrete mass/non-countable, umbrella tag, substance (cf. <mat>, terra, choça, magma) Substance prototypes',
              'examples': ["adubo /'fertilizer/'",
               "ar /'air/'",
               "breu /'tar/'",
               "espuma /'foam/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<liqu>': {'umbrella': False,
              'description': 'liquid',
              'examples': ["petróleo /'oil/'", "saliva /'saliva/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<mat>': {'umbrella': False,
              'description': 'material (SUBSTANCIA) substance Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)',
              'examples': ["madeira /'tree/'", "silicone /'silicone/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<stof>': {'umbrella': False,
              'description': 'fabric',
              'examples': ["seda /'silke/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<mad>': {'umbrella': False,
              'description': 'food, unprocessed-> <madc> (piece of unprocessed food)',
              'examples': ["carne /'meat/'", "alho /'garlic/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<kul>': {'umbrella': False,
              'description': 'food, processed-> <kulc> (piece of processed food)',
              'examples': ["chocolate /'chocolate/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<drik>': {'umbrella': False,
              'description': 'drink',
              'examples': ["chope /'draught beer/'", "leite /'milk/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<rem>': {'umbrella': False,
              'description': 'remedy (medicine)',
              'examples': ["morfina /'morphine/'",
               "penicilina /'penicilline/'",
               "vitamina /'vitamin/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<cc>': {'umbrella': True,
              'description': 'concrete objects (+CONCRETE, -MASS) Concrete countable object, umbrella tag (briquete, coágulo, normally movable things, unlike <part-build>) Thing prototypes',
              'examples': ["pedra /'stone/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<ar>': {'umbrella': False,
              'description': 'stack, heap, pile, bundle, row',
              'examples': ["feixa /'bundle/'", "fila /'row/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<er>': {'umbrella': False,
              'description': '(countable) piece or part or group member',
              'examples': ["ingrediente /'ingredient/'",
               "lasca /'chip, splinter/'",
               "parte /'part/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<sten>': {'umbrella': False,
              'description': 'stone (you can throw, cf. <mat>)',
              'examples': ["pedra /'stone/'", "rubim /'ruby/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<stok>': {'umbrella': False,
              'description': 'stick, plank, board',
              'examples': ["vara /'rod/'", "galho /'twig/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<ild>': {'umbrella': False,
              'description': '(1) fire, spark etc. (2) pipe, bonfire (all that can be lighted) -> cf. <lys> (light tools)',
              'examples': ["chama /'flame/'",
               "chispa /'spark/'",
               "fogueira /'bonfire/'",
               "relâmpago /'lightning/'",
               "cachimbo /'pipe/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<vejrc>': {'umbrella': False,
              'description': 'countable weather phenomena',
              'examples': ["nuvem /'cloud/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<madc>': {'umbrella': False,
              'description': 'piece of unprocessed food, eg. fruit',
              'examples': ["ervilha /'pea/'", "cebola /'onion/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<kulc>': {'umbrella': False,
              'description': 'piece of processed food, eg. burger',
              'examples': ["pão /'bread/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<il>': {'umbrella': False,
              'description': 'tool',
              'examples': ["garfo /'fork/'", "plectro /'plectron/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<kniv>': {'umbrella': False,
              'description': 'knife, sword, spear (bundling of features in prototype: sharp, pointed, cutting, tool)',
              'examples': ["faca /'knife/'",
               "canivete /'pocket knife/'",
               "enxada /'spade/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<fio>': {'umbrella': False,
              'description': 'thread, rope',
              'examples': ["estrém /'anchor cable/'",
               "fio /'thread/'",
               "cabo /'cable, rope/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<klud>': {'umbrella': False,
              'description': 'piece of cloth',
              'examples': ["guardanapo /'napkin/'", "toalha /'håndklæde/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<sejl>': {'umbrella': False,
              'description': 'sail',
              'examples': ["bujarrona /'jib, gib/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<paf>': {'umbrella': False,
              'description': 'gun',
              'examples': ["canhão /'canon/'", "pistola /'pistol/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<lys>': {'umbrella': False,
              'description': 'lamp, torch etc. (all that gives light)-> cf. <ild> (fire-words)',
              'examples': ["lanterna /'lantern/'", "tocha /'torch/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<ten>': {'umbrella': False,
              'description': 'handle, mouth-piece, hilt',
              'examples': ["maçaneta /'doorknob/'", "hastil /'shaft [of a lance]/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<mu>': {'umbrella': False,
              'description': 'musical instrument',
              'examples': ["violão /'guitar/'", "flauta /'flute/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<tøj>': {'umbrella': False,
              'description': 'garment (what you wear)',
              'examples': ["saia /'skirt/'", "camisa /'shirt/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<sko>': {'umbrella': False,
              'description': 'shoe',
              'examples': ["chinela /'slipper/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<hat>': {'umbrella': False,
              'description': 'hat',
              'examples': ["coroa /'crown/'", "chapéu /'hat/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<smyk>': {'umbrella': False,
              'description': 'jewels etc.',
              'examples': ["brinco /'earring/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<tøjzo>': {'umbrella': False,
              'description': 'what animals wear',
              'examples': ["brida /'bridle/'", "xairel /'saddle cloth/'"],
              'category': 'CONCRETA, NON-MOVING (MOVABLE)'},
     '<am>': {'umbrella': True,
              'description': 'quantifiable feature, abstract mass nouns (-CONCRETE, +MASS) Abstract mass/non-countable, umbrella tag (still contains many cases that could be <f-..>, e.g. habilidade, legalidade) Abstract prototypes',
              'examples': ['-eza, -idade'],
              'category': 'ABSTRACTA'},
     '<amh>': {'umbrella': False,
              'description': '+HUM quantifiable feature (ele tem mais ...ade)',
              'examples': [''],
              'category': 'ABSTRACTA'},
     '<ac>': {'umbrella': True,
              'description': 'abstract countables (-CONCRETE, -MASS, +COUNT) Abstract countable, umbrella tag (alternativa, chance, lazer) Abstract prototypes',
              'examples': ["método /'method/'", "módulo /'module/'", "onda /'wave/'"],
              'category': 'ABSTRACTA'},
     '<featc>': {'umbrella': False,
              'description': 'countable features (mark, spot)',
              'examples': ["cor /'colour/'", "listra /'stripe/'"],
              'category': 'ABSTRACTA'},
     '<anfeatc>': {'umbrella': False,
                  'description': 'countable anatomical feature',
                  'examples': ["verruga /'wart/'", "cabelo /'hair/'"],
                  'category': 'ABSTRACTA'},
     '<sygc>': {'umbrella': False,
              'description': 'countable item of disease (boil, scar)',
              'examples': ["terçolho /'sty/'", "abcesso /'abcess/'"],
              'category': 'ABSTRACTA'},
     '<p>': {'umbrella': False,
              'description': 'what yout think (thought)',
              'examples': ["idéia /'idea/'", "suspeita /'suspicion/'"],
              'category': 'ABSTRACTA'},
     '<pp>': {'umbrella': False,
              'description': 'plan, concept (product of thought)',
              'examples': ["projeto /'project/'",
               "estratégia /'strategy/'",
               "noção /'notion/'"],
              'category': 'ABSTRACTA'},
     '<reg>': {'umbrella': False,
              'description': 'rule, law',
              'examples': ["regra /'rule/'"],
              'category': 'ABSTRACTA'},
     '<right>': {'umbrella': False,
              'description': 'rights, habits (one has)(= reg +ADJECTIVAL)',
              'examples': ["direito /'right/'", "prerrogativa /'privilege/'"],
              'category': 'ABSTRACTA'},
     '<emne>': {'umbrella': False,
              'description': 'topic (both ac and ak)',
              'examples': ["assunto /'topic/'"],
              'category': 'ABSTRACTA'},
     '<l>': {'umbrella': False,
              'description': 'what you hear (e.g. natural or artificial sound, noise)',
              'examples': ["aplauso /'applause/'", "berro /'shout/'", "som /'sound/'"],
              'category': 'ABSTRACTA'},
     '<ll>': {'umbrella': False,
              'description': 'song, piece of music, type of musik (product for hearing-listening)',
              'examples': ["bossa=nova, canto /'song/'", "hino /'hymn/'"],
              'category': 'ABSTRACTA'},
     '<w>': {'umbrella': False,
          'description': 'what you see (e.g. bubble, shadow, a light)',
          'examples': ["vislumbre /'glimpse, glimmer/'",
           "ilusão /'illusion/'",
           "arco-iris /'rainbow/'"],
          'category': 'ABSTRACTA'},
     '<ww>': {'umbrella': False,
              'description': 'what you watch, e.g. movie, piece of theater (product for watching)',
              'examples': ["filme /'film/'",
               "sonho /'dream/'",
               "novela /'tv-series/'",
               'comédia'],
              'category': 'ABSTRACTA'},
     '<s>': {'umbrella': False,
              'description': 'what you say, short utterance (e.g. word, question, answer)',
              'examples': ["pergunta /'question/'", "salamaleque /'salem aleikum/'"],
              'category': 'ABSTRACTA'},
     '<ss>': {'umbrella': False,
              'description': 'speech, joke, lie, rumour, nonsense, gossip, boast (speech product)',
              'examples': ["sermão /'sermon/'", "testemunho /'testemony/'"],
              'category': 'ABSTRACTA'},
     '<sd>': {'umbrella': False,
              'description': 'speech act: tease, mockery, fun (you make of s.b.), insult, request (doing by saying), intrigue, proposal, settlement, appointment, ruling, judgement',
              'examples': ["reza /'prayer/'",
               "ultimato /'ultimatum/'",
               "veto /'veto/'",
               "queixa /'complaint/'"],
              'category': 'ABSTRACTA'},
     '<ret>': {'umbrella': False,
              'description': 'rhetoric terms (non-rhetoric, e.g parts of speech like ‘noun’, ‘subject’ -> akc)',
              'examples': ["hipérbole 'hyperbole'", "coloquialismo 'colloquialism'"],
              'category': 'ABSTRACTA'},
     '<o>': {'umbrella': False,
              'description': 'what you smell (odour) or taste',
              'examples': ["bodum 'smell of non-castrated goat'", "bufa 'fart'"],
              'category': 'ABSTRACTA'},
     '<f>': {'umbrella': True,
              'description': 'what you feel (whish, burst of pain, - where not am) feature/property, umbrella tag (problematicidade, proporcionalidade) Feature prototypes',
              'examples': ["cócegas 'itching'", "deleite 'delight'", "desejo 'wish'"],
              'category': 'ABSTRACTA'},
     '<rr>': {'umbrella': False,
              'description': 'what you read (unlike r, which is cc: livre, jornal)',
              'examples': ["romance 'novel'", "dissertação 'dissertation'"],
              'category': 'ABSTRACTA'},
     '<tegn>': {'umbrella': False,
              'description': 'what you write (sign, character, icon, printed symbol, playing card (ace, king etc.) also: <NUM> (numbers as nouns)',
              'examples': ["vírgula 'comma'",
               "dáblio 'w'",
               'emblema',
               "gatafunhos '‘scribbles’, o cinco ‘number five’"],
              'category': 'ABSTRACTA'},
     '<geom>': {'umbrella': False,
              'description': 'geometric shape (circle, globe, angle) -> +top geometry noun (circle, shape, e.g. losango, octógono, elipse) Concept prototypes',
              'examples': ["elipse 'ellipse'", 'heksaedro', "retângulo 'rectangle'"],
              'category': 'ABSTRACTA'},
     '<line>': {'umbrella': False,
              'description': 'line, stripe, streak (now: ac)',
              'examples': ["linha 'line'", "raio 'ray, radius'"],
              'category': 'ABSTRACTA'},
     '<d>': {'umbrella': False,
              'description': 'often, but not always deverbal, cp. -> ac (what you do or make)',
              'examples': ["dar uma lavadela 'light washing'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<num+>': {'umbrella': False,
              'description': '(syntactic, not semantic, therefore preferably in combination with <unit>, <qu>, <qus>)',
              'examples': [''],
              'category': 'ABSTRACTA'},
     '<unit>': {'umbrella': False,
              'description': 'unit (always with -> num+) unit noun (always implying <num+>, implied by <cur> and <dur>, e.g. caloria, centímetro, lúmen)) Tool prototypes',
              'examples': ["ano 'year'", 'cruzeiro', "acre 'acre'"],
              'category': 'ABSTRACTA'},
     '<qu>': {'umbrella': False,
              'description': 'quantity +de 263 (always with -> num+)',
              'examples': ["montes de 'lots of'", "carradas de 'loads of'"],
              'category': 'ABSTRACTA'},
     '<qus>': {'umbrella': False,
              'description': '"title"-quantity +de (always with -> num+) [in fact, a Danish MT motivated category, permitting Danish N N sequences] ',
              'examples': ["metro 'meter'", "litro 'liter'", "garaffa 'bottle'"],
              'category': 'ABSTRACTA'},
     '<mon>': {'umbrella': False,
              'description': 'quantity of money amount of money (bolsa, custo, imposto, cf. <cur>) Concept prototypes',
              'examples': ["achádego 'finder’s reward'", "bolsa 'scholarship'"],
              'category': 'ABSTRACTA'},
     '<akc>': {'umbrella': False,
              'description': 'countable category (rhetoric terms rather -> ret)',
              'examples': ["verbo 'verb'", "substantivo 'noun'", "numeral 'numeral'"],
              'category': 'ABSTRACTA'},
     '<meta>': {'umbrella': False,
              'description': 'kind of, type of, group member (semantically transparent like ar, er) meta noun (tipo, espécie) Concept prototypes',
              'examples': ["tipo 'type'", "espécie 'kind'"],
              'category': 'ABSTRACTA'},
     '<ax>': {'umbrella': False,
              'description': '(-CONCRETE, -MASS, -COUNT, +FEATURE ["adjectival content"]) Abstract/concept, neither countable nor mass (endogamia), cp. <f>, <sit> etc. Abstract prototypes',
              'examples': ["antigüidade 'antiquity'",
               "neutralidade 'neutrality'",
               "simbiose 'symbiosis'"],
              'category': 'ABSTRACTA'},
     '<state>': {'umbrella': False,
              'description': 'state (s.th. is in, distinct from -> sit - state of affairs) state (of something, otherwise <sit>), abundância, calma, baixa-mar, equilíbrio State-of-affairs prototypes',
              'examples': ["estupor 'stupor'",
               "incandescência 'incandescence'",
               "sossego 'calm'"],
              'category': 'ABSTRACTA'},
     '<sh>': {'umbrella': False,
              'description': 'human state (e.g. health, ecstasy)',
              'examples': ["apatie 'apathy'", "cansaço 'fatigue'"],
              'category': 'ABSTRACTA'},
     '<feat>': {'umbrella': False,
              'description': 'non-quantifiable feature',
              'examples': ["clima '\\climate'",
               "enormidade 'enormity'",
               "textura 'texture'"],
              'category': 'ABSTRACTA'},
     '<fh>': {'umbrella': False,
              'description': 'human non-quantifiable feature, human capacity, skill',
              'examples': ["atitude 'attitude'",
               "calvície 'baldness'",
               "inocência 'innocence'"],
              'category': 'ABSTRACTA'},
     '<featq>': {'umbrella': False,
              'description': 'quantifiable non-count non-mass (inherent) feature',
              'examples': ["tamanho 'size'",
               'massa=atómica',
               "circunferência 'circumference'",
               "cumprimento 'length'"],
              'category': 'ABSTRACTA'},
     '<fhq>': {'umbrella': False,
              'description': 'human quantifiable feature',
              'examples': ["tensão=arterial 'blood pressure'"],
              'category': 'ABSTRACTA'},
     '<ak>': {'umbrella': False,
              'description': 'category',
              'examples': ["mistura 'mixture'",
               "modo=maior 'major key'",
               "pretérito 'past tense'"],
              'category': 'ABSTRACTA'},
     '<akss>': {'umbrella': False,
              'description': 'speech product category: nonsense, gossip, boast (similar to ss, but not countable)',
              'examples': ["farelório 'chit-chat'",
               "galimatias 'gibberish'",
               "mexerico 'gossip'"],
              'category': 'ABSTRACTA'},
     '<ism>': {'umbrella': False,
              'description': 'ideology, religion ideology or other value system (anarquismo, anti-ocidentalismo, apartheid) Concept prototypes',
              'examples': ["comunismo 'communism'"],
              'category': 'ABSTRACTA'},
     '<ling>': {'umbrella': False,
              'description': 'language language (alemão, catalão, bengali) Concept prototypes',
              'examples': ['esperanto', "gíria 'slang'", "inglês 'English'", '-ês'],
              'category': 'ABSTRACTA'},
     '<syg>': {'umbrella': False,
              'description': 'disease',
              'examples': ["psitacose 'parrot fever'"],
              'category': 'ABSTRACTA'},
     '<col>': {'umbrella': False,
              'description': 'coulour color adjective OTHER SEMANTICALLY INSPIRED CATEGORIES',
              'examples': ["roxo 'violet'", "rubente 'ruby-red'"],
              'category': 'ABSTRACTA'},
     '<CI>': {'umbrella': False,
              'description': 'activity (+CONTROL, imperfective -> -PL), often derived from <vi>',
              'examples': ["cavadela 'digging'",
               "circulação 'circulation'",
               "boicote 'boycot'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<lud>': {'umbrella': False,
              'description': 'game',
              'examples': ["pôquer 'poker'", "roleta 'roulette'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<sp>': {'umbrella': False,
              'description': 'sport',
              'examples': ['badminton', "canoagem 'canoeing'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<fag>': {'umbrella': False,
              'description': 'subject (to learn), profession (to practice)',
              'examples': ["cardiologia 'cardiology'",
               "ciência 'science'",
               "culinária 'cookery'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<terapi>': {'umbrella': False,
                  'description': 'therapy',
                  'examples': ["acupunctura 'acupuncture'"],
                  'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<dans>': {'umbrella': False,
              'description': 'dance (also -> ll)',
              'examples': ['mambo', 'polca', 'samba'],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<tæsk>': {'umbrella': False,
              'description': 'beating',
              'examples': ["sova 'beating'", "surra 'thrashing'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<CP>': {'umbrella': False,
              'description': 'action (+CONTROL, perfective ), often derived from <vt> (possibly, <ve>)',
              'examples': ["pacificação 'pacification'", "partida 'departure'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<CPS>': {'umbrella': False,
              'description': '-PL -actions',
              'examples': ['-ação'],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<CPP>': {'umbrella': False,
              'description': '+PL -actions',
              'examples': ["tiro 'shot'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<kneb>': {'umbrella': False,
              'description': 'trick, cheat, fraud',
              'examples': ["jeito 'trick'",
               "dica 'tip'",
               "engano 'cheat'",
               "intriga 'intrigue'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<cI>': {'umbrella': False,
              'description': 'process (-CONTROL, imperfective -> -PL), often derived from <ve>',
              'examples': ["crescimento 'growth'",
               "decaimento 'decay'",
               "decomposição 'rotting'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<cP>': {'umbrella': False,
              'description': 'event (-CONTROL, perfective)',
              'examples': ["impacto 'impact'",
               "nascimento 'birth'",
               "ovulação 'ovulation'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<cPS>': {'umbrella': False,
              'description': '-PL -events',
              'examples': ["queda 'fall'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<cPP>': {'umbrella': False,
              'description': '+PL -events',
              'examples': ["explosão 'explosion'", "boléu 'crash'", "acidente 'accident'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<snak>': {'umbrella': False,
              'description': 'talk',
              'examples': ["debate 'debate'",
               "bate-papo 'little talk'",
               "discussão 'discussion'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<strid>': {'umbrella': False,
              'description': 'fight, quarrel',
              'examples': ["rixa 'row'", "briga 'fight'"],
              'category': 'ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)'},
     '<temp>': {'umbrella': False,
              'description': 'point in time [can be part of occ, -V event] temporal object, point in time (amanhecer, novilúnio, test: até+, cf. <dur> and <per>) State-of-affairs prototypes',
              'examples': ["início 'beginning'", "instante 'moment'"],
              'category': 'TIME FIELD'},
     '<per>': {'umbrella': False,
              'description': 'period in time [part of cI/process, time-place] period of time (prototypical test: durante, e.g. guerra, década, cf. <dur> and <temp>) Concept prototypes',
              'examples': ["fase 'phase'", "guerra 'war'"],
              'category': 'TIME FIELD'},
     '<dur>': {'umbrella': False,
              'description': 'measure of time [part of num+, time-unit] duration noun (test: durar+, implies <unit>, e.g. átimo, mês, hora) Time and event prototypes',
              'examples': ["hora 'hour'", "semana 'week'"],
              'category': 'TIME FIELD'},
     '<occ>': {'umbrella': False,
              'description': 'occasion [part of cP/event, +HUM, human place-event] (ORGANIZADO) organised event Event categories <occ>, HAREM ACONTECIMENTO (HAREM tags in parenthesis)',
              'examples': ["concerto 'concert'", "guerra 'war'", "Natal 'Christmas'"],
              'category': 'TIME FIELD'},
     '<Adom>': {'umbrella': False,
              'description': 'Domestic animal or big mammal (likely to have female forms etc.: terneiro, leão/leoa, cachorro)',
              'examples': [],
              'category': 'Animal prototypes'},
     '<AAdom>': {'umbrella': False,
              'description': 'Group of domestic animals (boiada)',
              'examples': [],
              'category': 'Animal prototypes'},
     '<Aich>': {'umbrella': False,
              'description': 'Water-animal (tubarão, delfim)',
              'examples': [],
              'category': 'Animal prototypes'},
     '<Amyth>': {'umbrella': False,
              'description': 'Mythological animal (basilisco)',
              'examples': [],
              'category': 'Animal prototypes'},
     '<Azo>': {'umbrella': False,
              'description': 'Land-animal (raposa)',
              'examples': [],
              'category': 'Animal prototypes'},
     '<Aorn>': {'umbrella': False,
              'description': 'Bird (águia, bem-te-vi)',
              'examples': [],
              'category': 'Animal prototypes'},
     '<Aent>': {'umbrella': False,
              'description': 'Insect (borboleta)',
              'examples': [],
              'category': 'Animal prototypes'},
     '<Acell>': {'umbrella': False,
              'description': 'Cell-animal (bacteria, blood cells: linfócito)',
              'examples': [],
              'category': 'Animal prototypes'},
     '<Btree>': {'umbrella': False,
              'description': 'Tree (oliveira, palmeira)',
              'examples': [],
              'category': 'Plant prototypes'},
     '<Bflo>': {'umbrella': False,
              'description': 'Flower (rosa, taraxaco)',
              'examples': [],
              'category': 'Plant prototypes'},
     '<Bbush>': {'umbrella': False,
              'description': 'Bush, shrub (rododendro, tamariz)',
              'examples': [],
              'category': 'Plant prototypes'},
     '<fruit>': {'umbrella': False,
              'description': 'fruit, berry, nut (still mostly marked as <food-c>, abricote, amora, avelã, cebola)',
              'examples': [],
              'category': 'Food prototypes'},
     '<Bveg>': {'umbrella': False,
              'description': '(vegetable espargo, funcho) - further proposed categories',
              'examples': [],
              'category': 'Plant prototypes'},
     '<Hattr>': {'umbrella': True,
              'description': 'Attributive human umbrella tag (many -ista, -ante)',
              'examples': [],
              'category': 'Human prototypes'},
     '<Hbio>': {'umbrella': False,
              'description': 'Human classified by biological criteria (race, age etc., caboclo, mestiço, bebé, adulto) ',
              'examples': [],
              'category': 'Human prototypes'},
     '<Hfam>': {'umbrella': False,
              'description': 'Human with family or other private relation (pai, noiva)',
              'examples': [],
              'category': 'Human prototypes'},
     '<Hideo>': {'umbrella': False,
              'description': 'Ideological human (comunista, implies <Hattr>), also: follower, disciple (dadaista)',
              'examples': [],
              'category': 'Human prototypes'},
     '<Hmyth>': {'umbrella': False,
              'description': 'Humanoid mythical (gods, fairy tale humanoids, curupira, duende)',
              'examples': [],
              'category': 'Human prototypes'},
     '<Hnat>': {'umbrella': False,
              'description': 'Nationality human (brasileiro, alemão), also: inhabitant (lisboeta)',
              'examples': [],
              'category': 'Human prototypes'},
     '<Hprof>': {'umbrella': False,
              'description': 'Professional human (marinheiro, implies <Hattr>), also: sport, hobby (alpinista)',
              'examples': [],
              'category': 'Human prototypes'},
     '<Hsick>': {'umbrella': False,
              'description': 'Sick human (few: asmático, diabético, cp <sick>)',
              'examples': [],
              'category': 'Human prototypes'},
     '<Htit>': {'umbrella': False,
              'description': 'Title noun (rei, senhora)',
              'examples': [],
              'category': 'Human prototypes'},
     '<L>': {'umbrella': True,
              'description': 'Place, umbrella tag',
              'examples': [],
              'category': 'Place and spatial prototypes'},
     '<Labs>': {'umbrella': False,
              'description': 'Abstract place (anverso. auge)',
              'examples': [],
              'category': 'Place and spatial prototypes'},
     '<Lciv>': {'umbrella': False,
              'description': 'Civitas, town, country, county (equals <L> + <HH>, cidade, país)',
              'examples': [],
              'category': 'Place and spatial prototypes'},
     '<Lcover>': {'umbrella': False,
              'description': 'Cover, lid (colcha, lona, tampa)',
              'examples': [],
              'category': 'Place and spatial prototypes'},
     '<Lh>': {'umbrella': False,
              'description': 'Functional place, human built or human-used (aeroporto, anfiteatro, cp. <build> for just a building)',
              'examples': [],
              'category': 'Place and spatial prototypes'},
     '<Lopening>': {'umbrella': False,
                  'description': 'opening, hole (apertura, fossa',
                  'examples': [],
                  'category': 'Place and spatial prototypes'},
     '<Lpath>': {'umbrella': False,
              'description': 'Path (road, street etc.: rua, pista)',
              'examples': [],
              'category': 'Place and spatial prototypes'},
     '<Lstar>': {'umbrella': False,
              'description': 'Star object (planets, comets: planeta, quasar)',
              'examples': [],
              'category': 'Place and spatial prototypes'},
     '<Lsurf>': {'umbrella': False,
                  'description': 'surface (face, verniz, cp. <Lcover>)',
                  'examples': [],
                  'category': 'Place and spatial prototypes'},
     '<Ltip>': {'umbrella': False,
              'description': 'tip place, edge (pico, pontinha, cp. <Labs>)',
              'examples': [],
              'category': 'Place and spatial prototypes'},
     '<Ltop>': {'umbrella': False,
              'description': 'Geographical, natural place (promontório, pântano)',
              'examples': [],
              'category': 'Place and spatial prototypes'},
     '<Ltrap>': {'umbrella': False,
              'description': 'trap place (armadilha, armazelo)',
              'examples': [],
              'category': 'Place and spatial prototypes'},
     '<Lwater>': {'umbrella': False,
                  'description': 'Water place (river, lake, sea: fonte, foz, lagoa)',
                  'examples': [],
                  'category': 'Place and spatial prototypes'},
     '<Vwater>': {'umbrella': False,
                  'description': 'Water vehicle (ship: navio, submersível, canoa)',
                  'examples': [],
                  'category': 'Vehicle prototypes'},
     '<Vair>': {'umbrella': False,
              'description': 'Air vehicle (plane: hidroplano, jatinho)',
              'examples': [],
              'category': 'Vehicle prototypes'},
     '<ac-cat>': {'umbrella': False,
                  'description': 'Category word (latinismo, número atômico)',
                  'examples': [],
                  'category': 'Abstract prototypes'},
     '<ac-sign>': {'umbrella': False,
                  'description': 'sign, symbol (parêntese, semicolcheia)',
                  'examples': [],
                  'category': 'Abstract prototypes'},
     '<act>': {'umbrella': True,
              'description': 'Action, umbrella tag (+CONTROL, PERFECTIVE)',
              'examples': [],
              'category': 'Action prototypes'},
     '<act-beat>': {'umbrella': False,
                  'description': 'beat-action (thrashing, pancada, surra)',
                  'examples': [],
                  'category': 'Action prototypes'},
     '<act-d>': {'umbrella': False,
                  'description': 'do-action (typically dar/fazer + N, tentativa, teste, homenagem)',
                  'examples': [],
                  'category': 'Action prototypes'},
     '<act-s>': {'umbrella': False,
              'description': 'speech act or communicative act (proposta, ordem)',
              'examples': [],
              'category': 'Action prototypes'},
     '<act-trick>': {'umbrella': False,
                  'description': 'trick-action (cheat, fraud, ruse, jeito, fraude, similar to <act-d>)',
                  'examples': [],
                  'category': 'Action prototypes'},
     '<activity>': {'umbrella': True,
                  'description': 'Activity, umbrella tag (+CONTROL, IMPERFECTIVE, correria, manejo)',
                  'examples': [],
                  'category': 'Action prototypes'},
     '<fight>': {'umbrella': False,
              'description': 'fight, conflict (also <activity> and +TEMP, briga, querela)',
              'examples': [],
              'category': 'Action prototypes'},
     '<cc-h>': {'umbrella': True,
              'description': 'Artifact, umbrella tag (so far empty category in PALAVRAS)',
              'examples': [],
              'category': 'Thing prototypes'},
     '<cc-beauty>': {'umbrella': False,
                  'description': 'ornamental object (few: guirlanda, rufo)',
                  'examples': [],
                  'category': 'Thing prototypes'},
     '<cc-board>': {'umbrella': False,
                  'description': 'flat long object (few: board, plank, lousa, tabla)',
                  'examples': [],
                  'category': 'Thing prototypes'},
     '<cc-fire>': {'umbrella': False,
                  'description': 'fire object (bonfire, spark, chispa, fogo, girândola)',
                  'examples': [],
                  'category': 'Thing prototypes'},
     '<cc-handle>': {'umbrella': False,
                  'description': 'handle (garra, ansa, chupadouro)',
                  'examples': [],
                  'category': 'Thing prototypes'},
     '<cc-light>': {'umbrella': False,
                  'description': 'light artifact (lampião, farol, projector)',
                  'examples': [],
                  'category': 'Thing prototypes'},
     '<cc-particle>': {'umbrella': False,
                      'description': '(atomic) particle (few: cátion, eletrônio)',
                      'examples': [],
                      'category': 'Thing prototypes'},
     '<cc-r>': {'umbrella': False,
              'description': 'read object (carteira, cupom, bilhete, carta, cf. <sem-r>)',
              'examples': [],
              'category': 'Thing prototypes'},
     '<cc-rag>': {'umbrella': False,
                  'description': 'cloth object (towel, napkin, carpet, rag) , cp. <mat-cloth>',
                  'examples': [],
                  'category': 'Thing prototypes'},
     '<cc-stone>': {'umbrella': False,
                  'description': '(= cc-round) stones and stone-sized round objects (pedra, itá, amonite, tijolo)',
                  'examples': [],
                  'category': 'Thing prototypes'},
     '<cc-stick>': {'umbrella': False,
                  'description': 'stick object (long and thin, vara, lançe, paulito)',
                  'examples': [],
                  'category': 'Thing prototypes'},
     '<furn>': {'umbrella': False,
              'description': 'furniture (cama, cadeira, tambo, quadro)',
              'examples': [],
              'category': 'Thing prototypes'},
     '<amount>': {'umbrella': False,
                  'description': 'quantity noun (bocada, teor, sem-fim)',
                  'examples': [],
                  'category': 'Thing prototypes'},
     '<build>': {'umbrella': False,
              'description': 'building (casa, citadela, garagem)',
              'examples': [],
              'category': 'Thing prototypes'},
     '<cm-h>': {'umbrella': False,
              'description': 'human-made substance (cf. <mat>, cemento)',
              'examples': [],
              'category': 'Substance prototypes'},
     '<cm-chem>': {'umbrella': False,
                  'description': 'chemical substance, also biological (acetileno, amônio, anilina, bilirrubina',
                  'examples': [],
                  'category': 'Substance prototypes'},
     '<cm-gas>': {'umbrella': False,
                  'description': 'gas substance (so far few: argônio, overlap with. <cm-chem> and <cm>)',
                  'examples': [],
                  'category': 'Substance prototypes'},
     '<cm-liq>': {'umbrella': False,
                  'description': 'liquid substance (azeite, gasolina, plasma, overlap with <food> and <cm-rem>)',
                  'examples': [],
                  'category': 'Substance prototypes'},
     '<cm-rem>': {'umbrella': False,
                  'description': 'remedy (medical or hygiene, antibiótico, canabis, quinina, part of <cm-h>, overlap with <cm-chem>)',
                  'examples': [],
                  'category': 'Substance prototypes'},
     '<cloA>': {'umbrella': False,
              'description': 'animal clothing (sela, xabraque)',
              'examples': [],
              'category': 'Clothing prototypes'},
     '<cloH>': {'umbrella': False,
              'description': 'human clothing (albornoz, anoraque, babadouro, bermudas)',
              'examples': [],
              'category': 'Clothing prototypes'},
     '<cloH-beauty>': {'umbrella': False,
                      'description': 'beauty clothing (e.g. jewelry, diadema, pendente, pulseira)',
                      'examples': [],
                      'category': 'Clothing prototypes'},
     '<cloH-hat>': {'umbrella': False,
                  'description': 'hat (sombrero, mitra, coroa)',
                  'examples': [],
                  'category': 'Clothing prototypes'},
     '<cloH-shoe>': {'umbrella': False,
                  'description': 'shoe (bota, chinela, patim)',
                  'examples': [],
                  'category': 'Clothing prototypes'},
     '<coll>': {'umbrella': False,
              'description': 'set, collective (random or systematic collection/compound/multitude of similar but distinct small parts, conjunto, série)',
              'examples': [],
              'category': 'Collective prototypes'},
     '<coll-cc>': {'umbrella': False,
                  'description': 'thing collective, pile (baralho, lanço)',
                  'examples': [],
                  'category': 'Collective prototypes'},
     '<coll-B>': {'umbrella': False,
                  'description': 'plant-part collective (buquê, folhagem)',
                  'examples': [],
                  'category': 'Collective prototypes'},
     '<coll-sem>': {'umbrella': False,
                  'description': 'semantic collective, collection (arquivo, repertório)',
                  'examples': [],
                  'category': 'Collective prototypes'},
     '<coll-tool>': {'umbrella': False,
                  'description': 'tool collective, set (intrumentário, prataria)',
                  'examples': [],
                  'category': 'Collective prototypes'},
     '<con>': {'umbrella': False,
              'description': 'container (implies <num+> quantifying, ampola, chícara, aquário)',
              'examples': [],
              'category': 'Collective prototypes'},
     '<conv>': {'umbrella': False,
              'description': 'convention (social rule or law, lei, preceito)',
              'examples': [],
              'category': 'Collective prototypes'},
     '<cord>': {'umbrella': False,
              'description': 'cord, string, rope, tape (previously <tool-tie>, arame, fio, fibrila)',
              'examples': [],
              'category': 'Collective prototypes'},
     '<cur>': {'umbrella': False,
              'description': 'currency noun (countable, implies <unit>, cf. <mon>, dirham, euro, real, dólar)',
              'examples': [],
              'category': 'Collective prototypes'},
     '<dance>': {'umbrella': False,
              'description': 'dance (both <activity>, <genre> and <sem-l>, calipso, flamenco, forró)',
              'examples': [],
              'category': 'Collective prototypes'},
     '<domain>': {'umbrella': False,
                  'description': 'domain (subject matter, profession, cf. <genre>, anatomia, citricultura, dactilografia)',
                  'examples': [],
                  'category': 'Collective prototypes'},
     '<event>': {'umbrella': False,
              'description': '(EVENTO) non-organised event',
              'examples': [],
              'category': 'Event categories <occ>, HAREM ACONTECIMENTO (HAREM tags in parenthesis)'},
     '<f-an>': {'umbrella': False,
              'description': 'anatomical "local" feature, includes countables, e.g. barbela, olheiras)',
              'examples': [],
              'category': 'Feature prototypes'},
     '<f-c>': {'umbrella': False,
              'description': 'general countable feature (vestígio, laivos, vinco)',
              'examples': [],
              'category': 'Feature prototypes'},
     '<f-h>': {'umbrella': False,
              'description': 'human physical feature, not countable (lindura, compleição, same as <f-phys-h>, cp. anatomical local features <f-an>)',
              'examples': [],
              'category': 'Feature prototypes'},
     '<f-psych>': {'umbrella': False,
                  'description': 'human psychological feature (passionalidade, pavonice, cp. passing states <state-h>)',
                  'examples': [],
                  'category': 'Feature prototypes'},
     '<f-q>': {'umbrella': False,
              'description': "quantifiable feature (e.g. circunferência, calor, DanGram's <f-phys> covers both <f> and <f-q>)",
              'examples': [],
              'category': 'Feature prototypes'},
     '<f-right>': {'umbrella': False,
                  'description': 'human social feature (right or duty): e.g. copyright, privilégio, imperativo legal)',
                  'examples': [],
                  'category': 'Feature prototypes'},
     '<food>': {'umbrella': False,
              'description': 'natural/simplex food (aveia, açúcar, carne, so far including <spice>)',
              'examples': [],
              'category': 'Food prototypes'},
     '<food-c>': {'umbrella': False,
                  'description': 'countable food (few: ovo, dente de alho, most are <fruit> or <food-c-h>)',
                  'examples': [],
                  'category': 'Food prototypes'},
     '<food-h>': {'umbrella': False,
                  'description': 'human-prepared/complex culinary food (caldo verde, lasanha)',
                  'examples': [],
                  'category': 'Food prototypes'},
     '<food-c-h>': {'umbrella': False,
                  'description': 'culinary countable food (biscoito, enchido, panetone, pastel)',
                  'examples': [],
                  'category': 'Food prototypes'},
     '<drink>': {'umbrella': False,
              'description': 'drink (cachaça, leite, guaraná, moca)',
              'examples': [],
              'category': 'Food prototypes'},
     '<game>': {'umbrella': False,
              'description': 'play, game (bilhar, ioiô, poker, also <activity>)',
              'examples': [],
              'category': 'Concept prototypes'},
     '<genre>': {'umbrella': False,
              'description': '(DISCIPLINA) subject matter',
              'examples': [],
              'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<geom-line>': {'umbrella': False,
                  'description': 'line (few: linha, percentil, curvas isobáricas)',
                  'examples': [],
                  'category': 'Concept prototypes'},
     '<mach>': {'umbrella': False,
              'description': 'machine (complex, usually with moving parts, betoneira, embrulhador, limpa-pratos, cp. <tool>)',
              'examples': [],
              'category': 'Concept prototypes'},
     '<mat-cloth>': {'umbrella': False,
                  'description': 'cloth material (seda, couro, vison, kevlar), cp. <cc-rag>',
                  'examples': [],
                  'category': 'Concept prototypes'},
     '<month>': {'umbrella': False,
              'description': 'month noun/name (agosto, julho, part of <temp>)',
              'examples': [],
              'category': 'Concept prototypes'},
     '<part>': {'umbrella': False,
              'description': 'distinctive or functional part (ingrediente, parte, trecho)',
              'examples': [],
              'category': 'Part prototypes'},
     '<part-build>': {'umbrella': False,
                      'description': 'structural part of building or vehicle (balustrada, porta, estai)',
                      'examples': [],
                      'category': 'Part prototypes'},
     '<piece>': {'umbrella': False,
              'description': 'indistinctive (little) piece (pedaço, raspa)',
              'examples': [],
              'category': 'Part prototypes'},
     '<percep-f>': {'umbrella': False,
                  'description': 'what you feel (senses or sentiment, pain, e.g. arrepio, aversão, desagrado, cócegas, some overlap with <state-h>)',
                  'examples': [],
                  'category': 'Perception prototypes'},
     '<percep-l>': {'umbrella': False,
                  'description': 'sound (what you hear, apitadela, barrulho, berro, crepitação)',
                  'examples': [],
                  'category': 'Perception prototypes'},
     '<percep-o>': {'umbrella': False,
                  'description': 'olfactory impression (what you smell, bafo, chamuscom fragrância)',
                  'examples': [],
                  'category': 'Perception prototypes'},
     '<percep-t>': {'umbrella': False,
                  'description': 'what you taste (PALAVRAS: not implemented)',
                  'examples': [],
                  'category': 'Perception prototypes'},
     '<percep-w>': {'umbrella': False,
                  'description': 'visual impression (what you see, arco-iris, réstia, vislumbre)',
                  'examples': [],
                  'category': 'Perception prototypes'},
     '<pict>': {'umbrella': False,
              'description': 'picture (combination of <cc>, <sem-w> and <L>, caricatura, cintilograma, diapositivo)',
              'examples': [],
              'category': 'Perception prototypes'},
     '<pos-an>': {'umbrella': False,
              'description': 'anatomical/body position (few: desaprumo)',
              'examples': [],
              'category': 'Perception prototypes'},
     '<pos-soc>': {'umbrella': False,
                  'description': 'social position, job (emprego, condado, capitania, presidência)',
                  'examples': [],
                  'category': 'Perception prototypes'},
     '<process>': {'umbrella': False,
                  'description': 'process (-CONTROL, -PERFECTIVE, cp. <event>, balcanização, convecção, estagnação)',
                  'examples': [],
                  'category': 'Perception prototypes'},
     '<sem>': {'umbrella': True,
              'description': 'semiotic artifact, work of art, umbrella tag (all specified in PALAVRAS)',
              'examples': [],
              'category': 'Semantic product prototypes'},
     '<sem-c>': {'umbrella': False,
                  'description': 'cognition product (concept, plan, system, conjetura, esquema, plano, prejuízo)',
                  'examples': [],
                  'category': 'Semantic product prototypes'},
     '<sem-l>': {'umbrella': False,
              'description': 'listen-work (music, cantarola, prelúdio, at the same time <genre>: bossa nova)',
              'examples': [],
              'category': 'Semantic product prototypes'},
     '<sem-nons>': {'umbrella': False,
                  'description': 'nonsense, rubbish (implies <sem-s>, galimatias, farelório)',
                  'examples': [],
                  'category': 'Semantic product prototypes'},
     '<sem-r>': {'umbrella': False,
              'description': 'read-work (biografia, dissertação, e-mail, ficha cadastral)',
              'examples': [],
              'category': 'Semantic product prototypes'},
     '<sem-s>': {'umbrella': False,
              'description': 'speak-work (palestra, piada, exposto)',
              'examples': [],
              'category': 'Semantic product prototypes'},
     '<sem-w>': {'umbrella': False,
              'description': 'watch-work (filme, esquete, mininovela)',
              'examples': [],
              'category': 'Semantic product prototypes'},
     '<sick>': {'umbrella': False,
              'description': 'disease (acne, AIDS, sida, alcoolismo, cp. <Hsick>)',
              'examples': [],
              'category': 'Semantic product prototypes'},
     '<sick-c>': {'umbrella': False,
                  'description': 'countable disease-object (abscesso, berruga, cicatriz, gangrena)',
                  'examples': [],
                  'category': 'Semantic product prototypes'},
     '<state-h>': {'umbrella': False,
                  'description': 'human state (desamparo, desesperança, dormência, euforia, febre, cp. <f-psych> and <f-phys-h>, which cover innate features)',
                  'examples': [],
                  'category': 'State-of-affairs prototypes'},
     '<sport>': {'umbrella': False,
              'description': 'sport (capoeira, futebol, golfe, also <activity> and <domain>)',
              'examples': [],
              'category': 'State-of-affairs prototypes'},
     '<talk>': {'umbrella': False,
              'description': 'speech situation, talk, discussion, quarrel (implies <activity> and <sd>, entrevista, lero-lero)',
              'examples': [],
              'category': 'State-of-affairs prototypes'},
     '<therapy>': {'umbrella': False,
                  'description': 'therapy (also <domain> and <activity>, acupuntura, balneoterapia)',
                  'examples': [],
                  'category': 'State-of-affairs prototypes'},
     '<tool>': {'umbrella': True,
              'description': 'tool, umbrella tag (abana-moscas, lápis, computador, maceta, "handable", cf. <mach>)',
              'examples': [],
              'category': 'Tool prototypes'},
     '<tool-cut>': {'umbrella': False,
                  'description': 'cutting tool, knife (canivete, espada)',
                  'examples': [],
                  'category': 'Tool prototypes'},
     '<tool-gun>': {'umbrella': False,
                  'description': 'shooting tool, gun (carabina, metralhadora, helicanão, in Dangram: <tool-shoot>)',
                  'examples': [],
                  'category': 'Tool prototypes'},
     '<tool-mus>': {'umbrella': False,
                  'description': 'musical instrument (clavicórdio, ocarina, violão)',
                  'examples': [],
                  'category': 'Tool prototypes'},
     '<tool-sail>': {'umbrella': False,
                  'description': 'sailing tool, sail (vela latina, joanete, coringa)',
                  'examples': [],
                  'category': 'Tool prototypes'},
     '<tube>': {'umbrella': False,
              'description': 'tube object (cânula, gasoduto, zarabatana, shape-category, typically with another category, like <an> or <tool>)',
              'examples': [],
              'category': 'Tool prototypes'},
     '<wea>': {'umbrella': True,
              'description': 'weather (states), umbrella tag (friagem, bruma)',
              'examples': [],
              'category': 'Weather prototypes'},
     '<wea-c>': {'umbrella': False,
                  'description': 'countable weather phenomenon (nuvem, tsunami)',
                  'examples': [],
                  'category': 'Weather prototypes'},
     '<wea-rain>': {'umbrella': False,
                  'description': "rain and other precipitation (chuvisco, tromba d'água, granizo)",
                  'examples': [],
                  'category': 'Weather prototypes'},
     '<wea-wind>': {'umbrella': False,
                  'description': 'wind, storm (brisa, furacão)',
                  'examples': [],
                  'category': 'Weather prototypes'},
     '<hum>': {'umbrella': False,
              'description': '(INDIVIDUAL) person name (cp. <H>)',
              'examples': [],
              'category': 'Person categories <hum>, HAREM PESSOA (HAREM tags in parenthesis)'},
     '<official>': {'umbrella': False,
                  'description': '(CARGO) official function (~ cp. <Htitle> and <Hprof>)',
                  'examples': [],
                  'category': 'Person categories <hum>, HAREM PESSOA (HAREM tags in parenthesis)'},
     '<member>': {'umbrella': False,
                  'description': '(MEMBRO) member',
                  'examples': [],
                  'category': 'Person categories <hum>, HAREM PESSOA (HAREM tags in parenthesis)'},
     '<admin>': {'umbrella': False,
              'description': '(ADMINISTRACAO, ORG.) administrative body (government, town administration etc.)',
              'examples': [],
              'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<org>': {'umbrella': False,
              'description': '(INSTITUICAO/EMPRESA) commercial or non-commercial, non-administrative non-party organisations (not place-bound, therefore not the same as <Linst>)',
              'examples': [],
              'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<media>': {'umbrella': False,
              'description': '(EMPRESA) media organisation (e.g. newspaper, tv channel)',
              'examples': [],
              'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<party>': {'umbrella': False,
              'description': '(INSTITUICAO) political party',
              'examples': [],
              'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<suborg>': {'umbrella': False,
                  'description': '(SUB) organized part of any of the above',
                  'examples': [],
                  'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<company>': {'umbrella': False,
                  'description': '(EMPRESA) company (not site-bound, unlike <inst>, now fused with. <org>) - currently unsupported',
                  'examples': [],
                  'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<groupind>': {'umbrella': False,
                  'description': '(GROUPOIND) people, family',
                  'examples': [],
                  'category': 'Group categories, HAREM PESSOA (HAREM tags in parenthesis)'},
     '<groupofficial>': {'umbrella': False,
                      'description': '(GROUPOCARGO) board, government (not fully implemented)',
                      'examples': [],
                      'category': 'Group categories, HAREM PESSOA (HAREM tags in parenthesis)'},
     '<grouporg>': {'umbrella': False,
                  'description': '(GROUPOMEMBRO) club, e.g. football club (now fused with <org>) - currently unsupported',
                  'examples': [],
                  'category': 'Group categories, HAREM PESSOA (HAREM tags in parenthesis)'},
     '<civ>': {'umbrella': False,
              'description': '(ADMINISTRACAO, LOC.) civitas (country, town, state, cp. <Lciv>)',
              'examples': [],
              'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<address>': {'umbrella': False,
                  'description': '(CORREIO) address (including numbers etc.)',
                  'examples': [],
                  'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<site>': {'umbrella': False,
              'description': '(ALARGADO) functional place (cp. <Lh>)',
              'examples': [],
              'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<virtual>': {'umbrella': False,
                  'description': '(VIRTUAL) virtual place',
                  'examples': [],
                  'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<astro>': {'umbrella': False,
                  'description': '(OBJECTO) astronomical place (in HAREM object, not place)',
                  'examples': [],
                  'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<road>': {'umbrella': False,
                  'description': '(ALARGADO) roads, motorway (unlike <address>) - suggested',
                  'examples': [],
                  'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<history>': {'umbrella': False,
                  'description': '(EFEMERIDE) one-time [historical] occurrence',
                  'examples': [],
                  'category': 'Event categories <occ>, HAREM ACONTECIMENTO (HAREM tags in parenthesis)'},
     '<tit>': {'umbrella': False,
                  'description': '(REPRODUZIDO) [title of] reproduced work, copy',
                  'examples': [],
                  'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<pub>': {'umbrella': False,
                  'description': '(PUBLICACAO) [scientific] publication',
                  'examples': [],
                  'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<product>': {'umbrella': False,
                  'description': '(PRODUTO) product brand',
                  'examples': [],
                  'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<artwork>': {'umbrella': False,
                  'description': '(ARTE) work of art',
                  'examples': [],
                  'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},     
     '<brand>': {'umbrella': False,
                  'description': '(MARCA) brand',
                  'examples': [],
                  'category': 'Abstract categories <brand>, HAREM ABSTRACCAO'},
     '<school>': {'umbrella': False,
                  'description': '(ESCOLA) school of thought',
                  'examples': [],
                  'category': 'Abstract categories <brand>, HAREM ABSTRACCAO'},
     '<idea>': {'umbrella': False,
                  'description': '(IDEA) idea, concept',
                  'examples': [],
                  'category': 'Abstract categories <brand>, HAREM ABSTRACCAO'},
     '<plan>': {'umbrella': False,
                  'description': '(PLANO) named plan, project',
                  'examples': [],
                  'category': 'Abstract categories <brand>, HAREM ABSTRACCAO'},
     '<author>': {'umbrella': False,
                  'description': "(OBRA) artist's name, standing for body of work",
                  'examples': [],
                  'category': 'Abstract categories <brand>, HAREM ABSTRACCAO'},
     '<absname>': {'umbrella': False,
                      'description': '(NOME)',
                      'examples': [],
                      'category': 'Abstract categories <brand>, HAREM ABSTRACCAO'},
     '<disease>': {'umbrella': False,
                  'description': '(ESTADO) physiological state, in particular: disease',
                  'examples': [],
                  'category': 'Abstract categories <brand>, HAREM ABSTRACCAO'},
     '<object>': {'umbrella': False,
                  'description': '(OBJECT) named object',
                  'examples': [],
                  'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'},
     '<common>': {'umbrella': False,
                  'description': '(OBJECT) common noun used as name',
                  'examples': [],
                  'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'},
     '<class>': {'umbrella': False,
                  'description': '(CLASSE) classification category for things',
                  'examples': [],
                  'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'},
     '<plant>': {'umbrella': False,
                  'description': '(CLASSE) plant name',
                  'examples': [],
                  'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'},
     '<currency>': {'umbrella': False,
                  'description': '(MOEDA) currency name (also marked on the unit)',
                  'examples': [],
                  'category': "Numeric value categories (marked only on the numeral, without MWE'ing, unlike HAREM VALOR)"},
     '<date>': {'umbrella': False,
              'description': '(DATA) date',
              'examples': [],
              'category': "Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO)"},
     '<hour>': {'umbrella': False,
              'description': '(HORA) hour',
              'examples': [],
              'category': "Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO)"},
     '<period>': {'umbrella': False,
                  'description': '(PERIODO) period',
                  'examples': [],
                  'category': "Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO)"},
     '<cyclic>': {'umbrella': False,
                  'description': '(CICLICO) cyclic time expression',
                  'examples': [],
                  'category': "Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO)"},
     '<quantity>': {'umbrella': False,
                  'description': '(QUANTIDADE) simple measuring numeral',
                  'examples': [],
                  'category': "Numeric value categories (marked only on the numeral, without MWE'ing, unlike HAREM VALOR)"},
     '<prednum>': {'umbrella': False,
                  'description': '(CLASSIFICADO) predicating numeral',
                  'examples': [],
                  'category': "Numeric value categories (marked only on the numeral, without MWE'ing, unlike HAREM VALOR)"},
     '<mass>': {'umbrella': False,
              'description': 'mass noun (e.g. "leite", "água")',
              'examples': [],
              'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<jh>': {'umbrella': False,
              'description': 'adjective modifying human noun',
              'examples': [],
              'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<jn>': {'umbrella': False,
              'description': 'adjective modifying inanimate noun',
              'examples': [],
              'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<ja>': {'umbrella': False,
              'description': 'adjective modifying animal',
              'examples': [],
              'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<jb>': {'umbrella': False,
              'description': 'adjective modifying plant',
              'examples': [],
              'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<nat>': {'umbrella': False,
              'description': 'nationality adjective (also: from a certain town etc.)',
              'examples': [],
              'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<vH>': {'umbrella': False,
              'description': 'verb with human subject',
              'examples': [],
              'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<vN>': {'umbrella': False,
          'description': 'verb with inanimate subject',
          'examples': [],
          'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'}

}

In [371]:
def identify_tag(tag):
    '''
    Help to identify the tag passed using dictionaries based on
    PALAVRAS rules    
    '''
    TAGSETS = [WORD_CLASS_TAGSET, WORD_SUBCLASS_TAGSET, INFLEXION_TAGSET, 
               SYNTATIC_TAGSET, TEXT_METAINFO_TAGSET, SEMANTIC_TAGSET, 
               NOMINAL_VALENCY_TAGSET, VERBAL_VALENCY_TAGSET]
    IDS_TAGSETS = ['WORD_CLASS', 'WORD_SUBCLASS', 'INFLEXION', 
                   'SYNTATIC', 'TEXT_METAINFO', 'SEMANTIC',
                   'NOMINAL_VALENCY', 'VERBAL_VALENCY']
    '''
    tag = tag.lower().replace('<','').replace('>','')
    for name_ts, ts in zip(IDS_TAGSETS,TAGSETS):
        ts_keys = [k.lower().replace('<','').replace('>','') for k in ts.keys()]
        #print(tag, ts_keys)
        if tag in ts_keys:
            return name_ts
    '''
    
    for name_ts, ts in zip(IDS_TAGSETS,TAGSETS):        
        if tag in ts.keys():
            return name_ts
        
    return "UNKNOWN"

In [363]:
MARK_START_SENTENCE = "<ß>"
MARK_END_SENTENCE = "</ß>"

def process_line(linha):
    token_1 = token_2 = token_3 = ""
        
    part_linha = linha.split('[')

    if len(part_linha) > 0:
        token_1 = part_linha[0].strip()

    part_linha = part_linha[1].split(']')
    token_2 = part_linha[0]
    #tags_linha = [ t for t in part_linha[1].split(' ') if t != '' and MARK_END_SENTENCE not in t]
    
    tags_linha = []
    ref = ""
    for t in part_linha[1].split(' '):
        if "#" in t and "->" in t:
            ref = t    
        elif t != '' and MARK_END_SENTENCE not in t:
            tags_linha.append((t, identify_tag(t)))
            
    return (token_1, token_2, tags_linha, ref)

In [364]:
def parse_file(filepath):
    html_lines = []
    try:
        with open(filepath, 'r') as f_html:
            html_lines = f_html.readlines()
    except IOError:
        print(f"Erro ao tentar abrir o arquivo {filepath}.")
        
    str_lines = " ".join(html_lines)
    dt_lines = str_lines.split('<br><dt>')

    text_sentences = []
    sentence = []
    count_sentences = 1
    count_tokens = 1

    for l in dt_lines:
        soup_line = BeautifulSoup(l, 'html.parser')    
        linha = soup_line.text        
        token_1 = token_2 = token_3 = ""

        if "[" in linha:
            #token_1, token_2, tags_linha, ref = process_line(linha)
            #sentence.append((count_tokens, token_1, token_2, tags_linha, ref))
            sentence.append((count_tokens, *process_line(linha)))
            count_tokens += 1

        if MARK_END_SENTENCE in linha:
            if len(sentence) > 0:            
                text_sentences.append(tuple([count_sentences, sentence]))        
                count_sentences += 1
                count_tokens = 1
                sentence = []
    
    return text_sentences


In [410]:
def parse_text(text):
    '''
    text it's a html text with tags and some marks of PALAVRAS parser
    '''
    
    dt_lines = text.split('<br><dt>')

    text_sentences = []
    sentence = []
    count_sentences = 1
    count_tokens = 1

    for l in dt_lines:
        soup_line = BeautifulSoup(l, 'html.parser')    
        linha = soup_line.text        
        token_1 = token_2 = token_3 = ""

        if "[" in linha:
            #token_1, token_2, tags_linha, ref = process_line(linha)
            #sentence.append((count_tokens, token_1, token_2, tags_linha, ref))
            sentence.append((count_tokens, *process_line(linha)))
            count_tokens += 1

        if MARK_END_SENTENCE in linha:
            if len(sentence) > 0:            
                text_sentences.append(tuple([count_sentences, sentence]))        
                count_sentences += 1
                count_tokens = 1
                sentence = []
    
    return text_sentences


In [390]:
texto = parse_file("../ccscore/data/visl.pt.html")

In [397]:
for num_s, s in texto:
    texto_s = " ".join([t[1] for t in s])
    print(num_s, texto_s)
    
    for t in s:
        print(50*"-"+"\n")
        print(t[1])        
        for tag in t[3]:
            #if "<" in tag[0] and ">" in tag[0]:
            if tag[1] == "UNKNOWN":
                print("\t",tag) 
        
    print(50*"+"+"\n")
        

1 Defendida por agentes de o mercado financeiro e uma de as bandeiras de a equipe econômica de o governo Jair Bolsonaro , o projeto de autonomia de o Banco Central ( BC ) deve avançar em a Câmara só após a reforma tributária andar , em o que depender de o presidente de a Casa , Rodrigo Maia ( DEM-RJ ) .
--------------------------------------------------

Defendida
	 ('<mv>', 'UNKNOWN')
--------------------------------------------------

por
--------------------------------------------------

agentes
	 ('§AG', 'UNKNOWN')
--------------------------------------------------

de
--------------------------------------------------

o
--------------------------------------------------

mercado
	 ('§TH', 'UNKNOWN')
--------------------------------------------------

financeiro
	 ('<nh>', 'UNKNOWN')
	 ('<np-close>', 'UNKNOWN')
--------------------------------------------------

e
--------------------------------------------------

uma
	 ('§AG', 'UNKNOWN')
----------------------------------------

In [404]:
import requests

def parse_PALAVRAS(text, output=None):
    parsed_text = None
    resp = None
    headers = {
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
                "Accept-Encoding": "gzip, deflate", 
                "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8", 
                "Dnt": "1", 
                "Host": "httpbin.org", 
                "Upgrade-Insecure-Requests": "1", 
                "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36", 
              }
    url_endpoint = 'https://visl.sdu.dk/cgi-bin/visl.pt.cgi'        
    
    print(f"Processing text...")        

    dic_params = {'text': text,
                     'parser':'roles',
                     'visual':'cg-dep',
                     'heads':'',
                     'symbol':'unfiltered',
                     'multisearch':'',
                     'searchtype':'',
                     'inputlang':'pt'}

    print(f"Request to parsing text")        

    try:
        resp = requests.get(url_endpoint, headers=headers, params=dic_params)
        resp.raise_for_status()
        
        if resp.status_code == 200:
            print("Success on request!")
            parsed_text = resp.text            
            if output:
                print("Writing file... ")
                try:                
                    with open(f"{output}", "w") as f_redacao:
                        f_redacao.write(resp.text)
                except IOError:
                    print("I/O Error when writing file.")        
                    
            return parsed_text        
        else:
            print("Error on request. No data parsed.")
            
    except requests.exceptions.HTTPError as errh:
        print ("Http Error: ",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Connection Error: ",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout: ",errt)        
    
    return parsed_text
    


In [405]:
text_to_parse="O ser humano constrói seu caráter em sociedade. Primeiro no lar, na família e depois na Igreja ou qualquer outra instituição."
r_parsed = parse_PALAVRAS(text_to_parse)


Processing text...
Request to parsing text
Success on request!
Writing file... 


In [407]:
l_sin = ['estado',
 'controle',
 'domínio',
 'regência',
 'governança',
 'administração',
 'regime',
 'governação',
 'regimento',
 'regímen',
 'norteamento',
 'direção',
 'orientação',
 'norteio']

s_sin = ",".join(l_sin)
r_parsed = parse_PALAVRAS(s_sin)

Processing text...
Request to parsing text
Success on request!
Writing file... 


In [409]:
s_parsed = BeautifulSoup(r_parsed)
s_parsed.text

'<ß>estado [estado]  <state> N M S  @TOP #1->27 , [,]  PU @PU #2->0 controle [controle]  <am> N M S  @TOP #3->27 , [,]  PU @PU #4->0 domínio [domínio]  <f-psych> <np-close> N M S  @N<PRED §ATR #5->3 , [,]  PU @PU #6->0 regência [regência]  <am> N F S  @PRED> §ATR #7->27 , [,]  PU @PU #8->0 governança [governança]  <HH> <np-close> N F S  @N<PRED §ATR #9->7 , [,]  PU @PU #10->0 administração [administração]  <activity> <HH> <np-close> N F S  @N<PRED §ATR #11->9 , [,]  PU @PU #12->0 regime [regime]  <ac> <np-close> N M S  @N<PRED §ATR #13->11 , [,]  PU @PU #14->0 governação [governação]  <activity> <np-close> N F S  @N<PRED §ATR #15->13 , [,]  PU @PU #16->0 regimento [regimento]  <act> <np-close> N M S  @N<PRED §ATR #17->15 , [,]  PU @PU #18->0 regímen [regímen]  <ac> <np-close> N M S  @N<PRED §ATR #19->17 , [,]  PU @PU #20->0 norteamento [norteamento]  <DERS> <np-close> N M S  @N<PRED §ATR #21->19 , [,]  PU @PU #22->0 direção [direção]  <HH> <dir> <Ltop> <np-close> N F S  @N<PRED §LOC #2

In [411]:
parse_text(r_parsed)

[(1,
  [(1,
    'estado',
    'estado',
    [('<state>', 'SEMANTIC'),
     ('N', 'WORD_CLASS'),
     ('M', 'INFLEXION'),
     ('S', 'INFLEXION'),
     ('@TOP', 'SYNTATIC')],
    '#1->27'),
   (2, ',', ',', [('PU', 'UNKNOWN'), ('@PU', 'UNKNOWN')], '#2->0'),
   (3,
    'controle',
    'controle',
    [('<am>', 'SEMANTIC'),
     ('N', 'WORD_CLASS'),
     ('M', 'INFLEXION'),
     ('S', 'INFLEXION'),
     ('@TOP', 'SYNTATIC')],
    '#3->27'),
   (4, ',', ',', [('PU', 'UNKNOWN'), ('@PU', 'UNKNOWN')], '#4->0'),
   (5,
    'domínio',
    'domínio',
    [('<f-psych>', 'SEMANTIC'),
     ('<np-close>', 'UNKNOWN'),
     ('N', 'WORD_CLASS'),
     ('M', 'INFLEXION'),
     ('S', 'INFLEXION'),
     ('@N<PRED', 'SYNTATIC'),
     ('§ATR', 'UNKNOWN')],
    '#5->3'),
   (6, ',', ',', [('PU', 'UNKNOWN'), ('@PU', 'UNKNOWN')], '#6->0'),
   (7,
    'regência',
    'regência',
    [('<am>', 'SEMANTIC'),
     ('N', 'WORD_CLASS'),
     ('F', 'INFLEXION'),
     ('S', 'INFLEXION'),
     ('@PRED>', 'SYNTATIC'),
   

In [296]:
from urllib.parse import quote
from urllib.request import urlretrieve

text_to_parse="O ser humano constrói seu caráter em sociedade. Primeiro no lar, na família e depois na Igreja ou qualquer outra instituição."
text_to_parse = quote(text_to_parse)
URL_PARSER="https://visl.sdu.dk/cgi-bin/visl.pt.cgi?text="+text_to_parse+"&parser=parse&visual=niceline&heads=&symbol=&multisearch=&searchtype=&inputlang=pt"

print(URL_PARSER)



https://visl.sdu.dk/cgi-bin/visl.pt.cgi?text=O%20ser%20humano%20constr%C3%B3i%20seu%20car%C3%A1ter%20em%20sociedade.%20Primeiro%20no%20lar%2C%20na%20fam%C3%ADlia%20e%20depois%20na%20Igreja%20ou%20qualquer%20outra%20institui%C3%A7%C3%A3o.&parser=parse&visual=niceline&heads=&symbol=&multisearch=&searchtype=&inputlang=pt


In [297]:
from bs4 import BeautifulSoup
html_text = """
<dl>

<dt><b><font color="maroon">o</font></b> 

<font color="maroon">[o]</font>  &lt;*&gt; &lt;artd&gt; <font color="blue"><b>DET</b> M S </font> <font color="darkgreen">@&gt;N</font>
<dt><b><font color="maroon">ser humano</font></b> 

<font color="maroon">[ser=humano]</font>  &lt;H&gt; <font color="blue"><b>N</b> M S </font> <font color="darkgreen">@SUBJ&gt;</font>
<dt><b><font color="maroon">constrói</font></b> 

<font color="maroon">[construir]</font>  &lt;fmc&gt; &lt;vt&gt; <font color="blue"><b>V</b> PR 3S IND VFIN </font> <font color="darkgreen">@FMV</font>
<dt><b><font color="maroon">seu</font></b> 

<font color="maroon">[seu]</font>  &lt;poss 3S&gt; &lt;si&gt; <font color="blue"><b>DET</b> M S </font> <font color="darkgreen">@&gt;N</font>
<dt><b><font color="maroon">caráter</font></b> 

<font color="maroon">[caráter]</font>  &lt;f-psych&gt; <font color="blue"><b>N</b> M S </font> <font color="darkgreen">@&lt;ACC</font>
<dt><b><font color="maroon">em</font></b> 

<font color="maroon">[em]</font>  <font color="blue"><b>PRP</b> </font> <font color="darkgreen">@&lt;ADVL</font>
<dt><b><font color="maroon">sociedade</font></b> 

<font color="maroon">[sociedade]</font>  &lt;HH&gt; <font color="blue"><b>N</b> F S </font> <font color="darkgreen">@P&lt;</font>
<dt><b><font color="maroon">.</font></b> 


<dt><b><font color="maroon">primeiro</font></b> 

<font color="maroon">[primeiro]</font>  &lt;*&gt; <font color="blue"><b>ADV</b> </font> <font color="darkgreen">@ADVL</font>
<dt><b><font color="maroon">em</font></b> 

<font color="maroon">[em]</font>  &lt;sam-&gt; <font color="blue"><b>PRP</b> </font> <font color="darkgreen">@ADVL</font>
<dt><b><font color="maroon">o</font></b> 

<font color="maroon">[o]</font>  &lt;-sam&gt; &lt;artd&gt; <font color="blue"><b>DET</b> M S </font> <font color="darkgreen">@&gt;N</font>
<dt><b><font color="maroon">lar</font></b> 

<font color="maroon">[lar]</font>  &lt;Lh&gt; <font color="blue"><b>N</b> M S </font> <font color="darkgreen">@P&lt;</font>
<dt><b><font color="maroon">,</font></b> 

<dt><b><font color="maroon">em</font></b> 

<font color="maroon">[em]</font>  &lt;sam-&gt; <font color="blue"><b>PRP</b> </font> <font color="darkgreen">@ADVL</font>
<dt><b><font color="maroon">a</font></b> 

<font color="maroon">[o]</font>  &lt;-sam&gt; &lt;artd&gt; <font color="blue"><b>DET</b> F S </font> <font color="darkgreen">@&gt;N</font>
<dt><b><font color="maroon">família</font></b> 

<font color="maroon">[família]</font>  &lt;HH&gt; <font color="blue"><b>N</b> F S </font> <font color="darkgreen">@P&lt;</font>
<dt><b><font color="maroon">e</font></b> 

<font color="maroon">[e]</font>  <font color="blue"><b>KC</b> </font> <font color="darkgreen">@CO</font>
<dt><b><font color="maroon">depois</font></b> 

<font color="maroon">[depois]</font>  &lt;atemp&gt; <font color="blue"><b>ADV</b> </font> <font color="darkgreen">@ADVL</font>
<dt><b><font color="maroon">em</font></b> 

<font color="maroon">[em]</font>  &lt;sam-&gt; <font color="blue"><b>PRP</b> </font> <font color="darkgreen">@ADVL</font>
<dt><b><font color="maroon">a</font></b> 

<font color="maroon">[o]</font>  &lt;-sam&gt; &lt;artd&gt; <font color="blue"><b>DET</b> F S </font> <font color="darkgreen">@&gt;N</font>
<dt><b><font color="maroon">igreja</font></b> 

<font color="maroon">[igreja]</font>  &lt;prop&gt; &lt;*&gt; &lt;inst&gt; <font color="blue"><b>N</b> F S </font> <font color="darkgreen">@P&lt;</font>
<dt><b><font color="maroon">ou</font></b> 

<font color="maroon">[ou]</font>  <font color="blue"><b>KC</b> </font> <font color="darkgreen">@CO</font>
<dt><b><font color="maroon">qualquer</font></b> 

<font color="maroon">[qualquer]</font>  &lt;quant&gt; <font color="blue"><b>DET</b> F S </font> <font color="darkgreen">@&gt;N</font>
<dt><b><font color="maroon">outra</font></b> 

<font color="maroon">[outro]</font>  &lt;diff&gt; &lt;KOMP&gt; <font color="blue"><b>DET</b> F S </font> <font color="darkgreen">@&gt;N</font>
<dt><b><font color="maroon">instituição</font></b> 

<font color="maroon">[instituição]</font>  &lt;inst&gt; &lt;act&gt; <font color="blue"><b>N</b> F S </font> <font color="darkgreen">@NPHR</font>
<dt><b><font color="maroon">.</font></b> 


<dt><b><font color="maroon">¶</font></b> 


</dl>
"""

soup = BeautifulSoup(html_text)
soup.text

'\n\no\n[o]  <*> <artd> DET M S  @>N\nser humano\n[ser=humano]  <H> N M S  @SUBJ>\nconstrói\n[construir]  <fmc> <vt> V PR 3S IND VFIN  @FMV\nseu\n[seu]  <poss 3S> <si> DET M S  @>N\ncaráter\n[caráter]  <f-psych> N M S  @<ACC\nem\n[em] PRP  @<ADVL\nsociedade\n[sociedade]  <HH> N F S  @P<\n.\nprimeiro\n[primeiro]  <*> ADV  @ADVL\nem\n[em]  <sam-> PRP  @ADVL\no\n[o]  <-sam> <artd> DET M S  @>N\nlar\n[lar]  <Lh> N M S  @P<\n,\nem\n[em]  <sam-> PRP  @ADVL\na\n[o]  <-sam> <artd> DET F S  @>N\nfamília\n[família]  <HH> N F S  @P<\ne\n[e] KC  @CO\ndepois\n[depois]  <atemp> ADV  @ADVL\nem\n[em]  <sam-> PRP  @ADVL\na\n[o]  <-sam> <artd> DET F S  @>N\nigreja\n[igreja]  <prop> <*> <inst> N F S  @P<\nou\n[ou] KC  @CO\nqualquer\n[qualquer]  <quant> DET F S  @>N\noutra\n[outro]  <diff> <KOMP> DET F S  @>N\ninstituição\n[instituição]  <inst> <act> N F S  @NPHR\n.\n¶\n\n'

Celulas de montagem dos dicionarios - apagar futuramente

In [344]:
s1 = """
Animal prototypes:

<A> Animal, umbrella tag (clone, fêmea, fóssil, parasito, predador)
<AA> Group of animals (cardume, enxame, passarada, ninhada)
<Adom> Domestic animal or big mammal (likely to have female forms etc.: terneiro, leão/leoa, cachorro)
<AAdom> Group of domestic animals (boiada)
<Aich> Water-animal (tubarão, delfim)
<Amyth> Mythological animal (basilisco)
<Azo> Land-animal (raposa)
<Aorn> Bird (águia, bem-te-vi)
<Aent> Insect (borboleta)
<Acell> Cell-animal (bacteria, blood cells: linfócito)

Plant prototypes:

<B> Plant, umbrella tag
<BB> Group of plants, plantation (field, forest etc.: mata, nabal)
<Btree> Tree (oliveira, palmeira)
<Bflo> Flower (rosa, taraxaco)
<Bbush> Bush, shrub (rododendro, tamariz)

<fruit> (fruit, berries, nuts: maçã, morango, avelã, melancia) - cp. also 
<Bveg> (vegetable espargo, funcho) - further proposed categories

Human prototypes:

<H> Human, umbrella tag
<HH> Group of humans (organisations, teams, companies, e.g. editora)
<Hattr> Attributive human umbrella tag (many -ista, -ante)
<Hbio> Human classified by biological criteria (race, age etc., caboclo, mestiço, bebé, adulto) 
<Hfam> Human with family or other private relation (pai, noiva)
<Hideo> Ideological human (comunista, implies <Hattr>), also: follower, disciple (dadaista)
<Hmyth> Humanoid mythical (gods, fairy tale humanoids, curupira, duende)
<Hnat> Nationality human (brasileiro, alemão), also: inhabitant (lisboeta)
<Hprof> Professional human (marinheiro, implies <Hattr>), also: sport, hobby (alpinista)
<Hsick> Sick human (few: asmático, diabético, cp <sick>)
<Htit> Title noun (rei, senhora)

Place and spatial prototypes:

<L> Place, umbrella tag
<Labs> Abstract place (anverso. auge)
<Lciv> Civitas, town, country, county (equals <L> + <HH>, cidade, país)
<Lcover> Cover, lid (colcha, lona, tampa)
<Lh> Functional place, human built or human-used (aeroporto, anfiteatro, cp. <build> for just a building)
<Lopening> opening, hole (apertura, fossa
<Lpath> Path (road, street etc.: rua, pista)
<Lstar> Star object (planets, comets: planeta, quasar)
<Lsurf> surface (face, verniz, cp. <Lcover>)
<Ltip> tip place, edge (pico, pontinha, cp. <Labs>)
<Ltop> Geographical, natural place (promontório, pântano)
<Ltrap> trap place (armadilha, armazelo)
<Lwater> Water place (river, lake, sea: fonte, foz, lagoa)


Vehicle prototypes:

<V> Vehicle, umbrella tag and ground vehicle (car, train: carro, comboio, tanque, teleférico)
<VV> Group of vehicles (armada, convoy: frota, esquadra)
<Vwater> Water vehicle (ship: navio, submersível, canoa)
<Vair> Air vehicle (plane: hidroplano, jatinho)

Abstract prototypes:

<ac> Abstract countable, umbrella tag (alternativa, chance, lazer)
<ac-cat> Category word (latinismo, número atômico)
<ac-sign> sign, symbol (parêntese, semicolcheia)
<am> Abstract mass/non-countable, umbrella tag (still contains many cases that could be <f-..>, e.g. habilidade, legalidade)
<ax> Abstract/concept, neither countable nor mass (endogamia), cp. <f>, <sit> etc.


Action prototypes:

<act> Action, umbrella tag (+CONTROL, PERFECTIVE)
<act-beat> beat-action (thrashing, pancada, surra)
<act-d> do-action (typically dar/fazer + N, tentativa, teste, homenagem)
<act-s> speech act or communicative act (proposta, ordem)
<act-trick> trick-action (cheat, fraud, ruse, jeito, fraude, similar to <act-d>)
<activity> Activity, umbrella tag (+CONTROL, IMPERFECTIVE, correria, manejo)
<fight> fight, conflict (also <activity> and +TEMP, briga, querela)


Anatomical prototypes:

<an> Anatomical noun, umbrella tag (carótida, clítoris, dorso)
<anmov> Movable anatomy (arm, leg, braço, bíceps, cotovelo)
<anorg> Organ (heart, liver, hipófise, coração, testículo)
<anost> Bone (calcâneo, fíbula, vértebra)
<anzo> Animal anatomy (rúmen, carapaça, chifres, tromba)
<anorn> Bird anatomy (bico, pluma)
<anich> Fish anatomy (few: bránquias, siba)
<anent> Insect anatomy (few: tentáculo, olho composto)
<anbo> Plant anatomy (bulbo, caule, folha)


Thing prototypes:

<cc> Concrete countable object, umbrella tag (briquete, coágulo, normally movable things, unlike <part-build>)
<cc-h> Artifact, umbrella tag (so far empty category in PALAVRAS)
<cc-beauty> ornamental object (few: guirlanda, rufo)
<cc-board> flat long object (few: board, plank, lousa, tabla)
<cc-fire> fire object (bonfire, spark, chispa, fogo, girândola)
<cc-handle> handle (garra, ansa, chupadouro)
<cc-light> light artifact (lampião, farol, projector)
<cc-particle> (atomic) particle (few: cátion, eletrônio)
<cc-r> read object (carteira, cupom, bilhete, carta, cf. <sem-r>)
<cc-rag> cloth object (towel, napkin, carpet, rag) , cp. <mat-cloth>
<cc-stone> (= cc-round) stones and stone-sized round objects (pedra, itá, amonite, tijolo)
<cc-stick> stick object (long and thin, vara, lançe, paulito)
<furn> furniture (cama, cadeira, tambo, quadro)
<amount> quantity noun (bocada, teor, sem-fim)
<bar> barrier noun (dique, limite, muralha)
<build> building (casa, citadela, garagem)

Substance prototypes:

<cm> concrete mass/non-countable, umbrella tag, substance (cf. <mat>, terra, choça, magma)
<cm-h> human-made substance (cf. <mat>, cemento)
<cm-chem> chemical substance, also biological (acetileno, amônio, anilina, bilirrubina
<cm-gas> gas substance (so far few: argônio, overlap with. <cm-chem> and <cm>)
<cm-liq> liquid substance (azeite, gasolina, plasma, overlap with <food> and <cm-rem>)
<cm-rem> remedy (medical or hygiene, antibiótico, canabis, quinina, part of <cm-h>, overlap with <cm-chem>)

Clothing prototypes:

<cloA> animal clothing (sela, xabraque)
<cloH> human clothing (albornoz, anoraque, babadouro, bermudas)
<cloH-beauty> beauty clothing (e.g. jewelry, diadema, pendente, pulseira)
<cloH-hat> hat (sombrero, mitra, coroa)
<cloH-shoe> shoe (bota, chinela, patim)

Collective prototypes:

<coll> set, collective (random or systematic collection/compound/multitude of similar but distinct small parts, conjunto, série)
<coll-cc> thing collective, pile (baralho, lanço)
<coll-B> plant-part collective (buquê, folhagem)
<coll-sem> semantic collective, collection (arquivo, repertório)
<coll-tool> tool collective, set (intrumentário, prataria)

<col> coulor (amarelo, carmesim, verde-mar)
<con> container (implies <num+> quantifying, ampola, chícara, aquário)
<conv> convention (social rule or law, lei, preceito)
<cord> cord, string, rope, tape (previously <tool-tie>, arame, fio, fibrila)
<cur> currency noun (countable, implies <unit>, cf. <mon>, dirham, euro, real, dólar)
<dance> dance (both <activity>, <genre> and <sem-l>, calipso, flamenco, forró)
<dir> direction noun (estibordo, contrasenso, norte)
<domain> domain (subject matter, profession, cf. <genre>, anatomia, citricultura, dactilografia)


Time and event prototypes:

<dur> duration noun (test: durar+, implies <unit>, e.g. átimo, mês, hora)
<per> (period) and <temp> (time point)
<event> event (-CONTROL, PERFECTIVE, milagre, morte)
<occ> (organized event), <process>, <act...> and <activity>

Feature prototypes:

<f> feature/property, umbrella tag (problematicidade, proporcionalidade)
<f-an> anatomical "local" feature, includes countables, e.g. barbela, olheiras)
<f-c> general countable feature (vestígio, laivos, vinco)
<f-h> human physical feature, not countable (lindura, compleição, same as <f-phys-h>, cp. anatomical local features <f-an>)
<f-psych> human psychological feature (passionalidade, pavonice, cp. passing states <state-h>)
<f-q> quantifiable feature (e.g. circunferência, calor, DanGram's <f-phys> covers both <f> and <f-q>)
<f-right> human social feature (right or duty): e.g. copyright, privilégio, imperativo legal)


Food prototypes:

<food> natural/simplex food (aveia, açúcar, carne, so far including <spice>)
<food-c> countable food (few: ovo, dente de alho, most are <fruit> or <food-c-h>)
<food-h> human-prepared/complex culinary food (caldo verde, lasanha)
<food-c-h> culinary countable food (biscoito, enchido, panetone, pastel)
<fruit> fruit, berry, nut (still mostly marked as <food-c>, abricote, amora, avelã, cebola)
<drink> drink (cachaça, leite, guaraná, moca)

Concept prototypes:

<game> play, game (bilhar, ioiô, poker, also <activity>)
<genre> genre (especially art genre, cf. <domain>, modernismo, tropicalismo)

<geom> geometry noun (circle, shape, e.g. losango, octógono, elipse)
<geom-line> line (few: linha, percentil, curvas isobáricas)

<inst> "institution", functional structure (+PLACE, +HUM, auto-escola, bolsa, cinemateca) , cp. <Lh> (human-made place) and <HH> (group, organisation)
<ism> ideology or other value system (anarquismo, anti-ocidentalismo, apartheid)
<ling> language (alemão, catalão, bengali)
<mach> machine (complex, usually with moving parts, betoneira, embrulhador, limpa-pratos, cp. <tool>)

<mat> material (argila, bronze, granito, cf. <cm>)
<mat-cloth> cloth material (seda, couro, vison, kevlar), cp. <cc-rag>

<meta> meta noun (tipo, espécie)
<mon> amount of money (bolsa, custo, imposto, cf. <cur>)
<month> month noun/name (agosto, julho, part of <temp>)
<occ> occasion, human/social event (copa do mundo, aniversário, jantar, desfile, cp. unorganized <event>)
<per> period of time (prototypical test: durante, e.g. guerra, década, cf. <dur> and <temp>)

Part prototypes:

<part> distinctive or functional part (ingrediente, parte, trecho)
<part-build> structural part of building or vehicle (balustrada, porta, estai)
<piece> indistinctive (little) piece (pedaço, raspa)


Perception prototypes:

<percep-f> what you feel (senses or sentiment, pain, e.g. arrepio, aversão, desagrado, cócegas, some overlap with <state-h>)
<percep-l> sound (what you hear, apitadela, barrulho, berro, crepitação)
<percep-o> olfactory impression (what you smell, bafo, chamuscom fragrância)
<percep-t> what you taste (PALAVRAS: not implemented)
<percep-w> visual impression (what you see, arco-iris, réstia, vislumbre)

<pict> picture (combination of <cc>, <sem-w> and <L>, caricatura, cintilograma, diapositivo)

<pos-an> anatomical/body position (few: desaprumo)
<pos-soc> social position, job (emprego, condado, capitania, presidência)
<process> process (-CONTROL, -PERFECTIVE, cp. <event>, balcanização, convecção, estagnação)

Semantic product prototypes:

<sem> semiotic artifact, work of art, umbrella tag (all specified in PALAVRAS)
<sem-c> cognition product (concept, plan, system, conjetura, esquema, plano, prejuízo)
<sem-l> listen-work (music, cantarola, prelúdio, at the same time <genre>: bossa nova)
<sem-nons> nonsense, rubbish (implies <sem-s>, galimatias, farelório)
<sem-r> read-work (biografia, dissertação, e-mail, ficha cadastral)
<sem-s> speak-work (palestra, piada, exposto)
<sem-w> watch-work (filme, esquete, mininovela)

<sick> disease (acne, AIDS, sida, alcoolismo, cp. <Hsick>)
<sick-c> countable disease-object (abscesso, berruga, cicatriz, gangrena)

State-of-affairs prototypes:

<sit> psychological situation or physical state of affairs (reclusão, arruaça, ilegalidade, more complex and more "locative" than <state> and <state-h>
<state> state (of something, otherwise <sit>), abundância, calma, baixa-mar, equilíbrio
<state-h> human state (desamparo, desesperança, dormência, euforia, febre, cp. <f-psych> and <f-phys-h>, which cover innate features)

<sport> sport (capoeira, futebol, golfe, also <activity> and <domain>)

<talk> speech situation, talk, discussion, quarrel (implies <activity> and <sd>, entrevista, lero-lero)
<temp> temporal object, point in time (amanhecer, novilúnio, test: até+, cf. <dur> and <per>)
<therapy> therapy (also <domain> and <activity>, acupuntura, balneoterapia)

Tool prototypes:

<tool> tool, umbrella tag (abana-moscas, lápis, computador, maceta, "handable", cf. <mach>)
<tool-cut> cutting tool, knife (canivete, espada)
<tool-gun> shooting tool, gun (carabina, metralhadora, helicanão, in Dangram: <tool-shoot>)
<tool-mus> musical instrument (clavicórdio, ocarina, violão)
<tool-sail> sailing tool, sail (vela latina, joanete, coringa)

<tube> tube object (cânula, gasoduto, zarabatana, shape-category, typically with another category, like <an> or <tool>)
<unit> unit noun (always implying <num+>, implied by <cur> and <dur>, e.g. caloria, centímetro, lúmen))

Weather prototypes:

<wea> weather (states), umbrella tag (friagem, bruma)
<wea-c> countable weather phenomenon (nuvem, tsunami)
<wea-rain> rain and other precipitation (chuvisco, tromba d'água, granizo)
<wea-wind> wind, storm (brisa, furacão)


Person categories <hum>, HAREM PESSOA (HAREM tags in parenthesis):

<hum> (INDIVIDUAL) person name (cp. <H>)
<official> (CARGO) official function (~ cp. <Htitle> and <Hprof>)
<member> (MEMBRO) member

Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis):

<admin> (ADMINISTRACAO, ORG.) administrative body (government, town administration etc.)
<org> (INSTITUICAO/EMPRESA) commercial or non-commercial, non-administrative non-party organisations (not place-bound, therefore not the same as <Linst>)
<inst> (EMPRESA) organized site (e.g. restaurant, cp. <Linst>)
<media> (EMPRESA) media organisation (e.g. newspaper, tv channel)
<party> (INSTITUICAO) political party
<suborg> (SUB) organized part of any of the above

<company> (EMPRESA) company (not site-bound, unlike <inst>, now fused with. <org>) - currently unsupported

Group categories, HAREM PESSOA (HAREM tags in parenthesis):

<groupind> (GROUPOIND) people, family
<groupofficial> (GROUPOCARGO) board, government (not fully implemented)

<grouporg> (GROUPOMEMBRO) club, e.g. football club (now fused with <org>) - currently unsupported

Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis):

<top> (GEOGRAFICO) geographical location (cp. <Ltop>)
<civ> (ADMINISTRACAO, LOC.) civitas (country, town, state, cp. <Lciv>)
<address> (CORREIO) address (including numbers etc.)
<site> (ALARGADO) functional place (cp. <Lh>)
<virtual> (VIRTUAL) virtual place
<astro> (OBJECTO) astronomical place (in HAREM object, not place)

<road> (ALARGADO) roads, motorway (unlike <address>) - suggested

Event categories <occ>, HAREM ACONTECIMENTO (HAREM tags in parenthesis):

<occ> (ORGANIZADO) organised event
<event> (EVENTO) non-organised event
<history> (EFEMERIDE) one-time [historical] occurrence

Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis):

<tit> (REPRODUZIDO) [title of] reproduced work, copy
<pub> (PUBLICACAO) [scientific] publication
<product> (PRODUTO) product brand
<V> (PRODUTO) vehicle brand (cp. <V>, <Vair>, <Vwater>)
<artwork> (ARTE) work of art

Abstract categories <brand>, HAREM ABSTRACCAO (HAREM tags in parenthesis): 

<brand> (MARCA) brand
<genre> (DISCIPLINA) subject matter
<school> (ESCOLA) school of thought
<idea> (IDEA) idea, concept
<plan> (PLANO) named plan, project
<author> (OBRA) artist's name, standing for body of work
<absname> (NOME)
<disease> (ESTADO) physiological state, in particular: disease

Thing categories <common>, HAREM COISA (HAREM tags in parenthesis):

<object> (OBJECT) named object
<common> (OBJECT) common noun used as name
<mat> (SUBSTANCIA) substance
<class> (CLASSE) classification category for things
<plant> (CLASSE) plant name
<currency> (MOEDA) currency name (also marked on the number)


Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO):

<date> (DATA) date
<hour> (HORA) hour
<period> (PERIODO) period
<cyclic> (CICLICO) cyclic time expression

Numeric value categories (marked only on the numeral, without MWE'ing, unlike HAREM VALOR):

<quantity> (QUANTIDADE) simple measuring numeral
<prednum> (CLASSIFICADO) predicating numeral
<currency> (MOEDA) currency name (also marked on the unit)

OTHER SEMANTICALLY INSPIRED CATEGORIES:

<mass> mass noun (e.g. "leite", "a'gua")

<jh> adjective modifying human noun
<jn> adjective modifying inanimate noun
<ja> adjective modifying animal
<jb> adjective modifying plant
<col> color adjective
<nat> nationality adjective (also: from a certain town etc.)
<attr> (human) attributive adjective (not fully implemented, cp. <Hattr>, e.g. "um presidente COMUNISTA")

<vH> verb with human subject
<vN> verb with inanimate subject
"""

l1s = s1.split('\n')
category = ""
tag = ""
description = ""
examples = []
dic_s = {}
for l in l1s:
    is_umbrella = False
    if l[-1:] == ':':
        category = l[:-1]
    else:
        if l != "":
            if "umbrella tag" in l:
                is_umbrella = True
            l_s = l.split(' ')
            tag = l_s[0]
            description = " ".join(l_s[1:])
            
            dic_s[tag] = {'umbrella': is_umbrella, 
                          'description': description,
                          'examples': examples,
                          'category': category}
                
            
#for k,v in dic_s.items():
#    print(k,v)
        
print(dic_s)        
#template 
'''
'H': {'umbrella': True, 
          'description': "+HUM-noun, human being -> cf. more specific categories below",
          'examples': ["achacador", "'kidnapper'","acionário", "'shareholder'", "inimigo","'enemy'"],
          'category': "ANIMATE HUMAN"}
'''

{'<A>': {'umbrella': True, 'description': 'Animal, umbrella tag (clone, fêmea, fóssil, parasito, predador)', 'examples': [], 'category': 'Animal prototypes'}, '<AA>': {'umbrella': False, 'description': 'Group of animals (cardume, enxame, passarada, ninhada)', 'examples': [], 'category': 'Animal prototypes'}, '<Adom>': {'umbrella': False, 'description': 'Domestic animal or big mammal (likely to have female forms etc.: terneiro, leão/leoa, cachorro)', 'examples': [], 'category': 'Animal prototypes'}, '<AAdom>': {'umbrella': False, 'description': 'Group of domestic animals (boiada)', 'examples': [], 'category': 'Animal prototypes'}, '<Aich>': {'umbrella': False, 'description': 'Water-animal (tubarão, delfim)', 'examples': [], 'category': 'Animal prototypes'}, '<Amyth>': {'umbrella': False, 'description': 'Mythological animal (basilisco)', 'examples': [], 'category': 'Animal prototypes'}, '<Azo>': {'umbrella': False, 'description': 'Land-animal (raposa)', 'examples': [], 'category': 'Anima

'\n\'H\': {\'umbrella\': True, \n          \'description\': "+HUM-noun, human being -> cf. more specific categories below",\n          \'examples\': ["achacador", "\'kidnapper\'","acionário", "\'shareholder\'", "inimigo","\'enemy\'"],\n          \'category\': "ANIMATE HUMAN"}\n'

In [345]:
ss_dic = """{'<A>': {'umbrella': True, 'description': 'Animal, umbrella tag (clone, fêmea, fóssil, parasito, predador)', 'examples': [], 'category': 'Animal prototypes'}, '<AA>': {'umbrella': False, 'description': 'Group of animals (cardume, enxame, passarada, ninhada)', 'examples': [], 'category': 'Animal prototypes'}, '<Adom>': {'umbrella': False, 'description': 'Domestic animal or big mammal (likely to have female forms etc.: terneiro, leão/leoa, cachorro)', 'examples': [], 'category': 'Animal prototypes'}, '<AAdom>': {'umbrella': False, 'description': 'Group of domestic animals (boiada)', 'examples': [], 'category': 'Animal prototypes'}, '<Aich>': {'umbrella': False, 'description': 'Water-animal (tubarão, delfim)', 'examples': [], 'category': 'Animal prototypes'}, '<Amyth>': {'umbrella': False, 'description': 'Mythological animal (basilisco)', 'examples': [], 'category': 'Animal prototypes'}, '<Azo>': {'umbrella': False, 'description': 'Land-animal (raposa)', 'examples': [], 'category': 'Animal prototypes'}, '<Aorn>': {'umbrella': False, 'description': 'Bird (águia, bem-te-vi)', 'examples': [], 'category': 'Animal prototypes'}, '<Aent>': {'umbrella': False, 'description': 'Insect (borboleta)', 'examples': [], 'category': 'Animal prototypes'}, '<Acell>': {'umbrella': False, 'description': 'Cell-animal (bacteria, blood cells: linfócito)', 'examples': [], 'category': 'Animal prototypes'}, '<B>': {'umbrella': True, 'description': 'Plant, umbrella tag', 'examples': [], 'category': 'Plant prototypes'}, '<BB>': {'umbrella': False, 'description': 'Group of plants, plantation (field, forest etc.: mata, nabal)', 'examples': [], 'category': 'Plant prototypes'}, '<Btree>': {'umbrella': False, 'description': 'Tree (oliveira, palmeira)', 'examples': [], 'category': 'Plant prototypes'}, '<Bflo>': {'umbrella': False, 'description': 'Flower (rosa, taraxaco)', 'examples': [], 'category': 'Plant prototypes'}, '<Bbush>': {'umbrella': False, 'description': 'Bush, shrub (rododendro, tamariz)', 'examples': [], 'category': 'Plant prototypes'}, '<fruit>': {'umbrella': False, 'description': 'fruit, berry, nut (still mostly marked as <food-c>, abricote, amora, avelã, cebola)', 'examples': [], 'category': 'Food prototypes'}, '<Bveg>': {'umbrella': False, 'description': '(vegetable espargo, funcho) - further proposed categories', 'examples': [], 'category': 'Plant prototypes'}, '<H>': {'umbrella': True, 'description': 'Human, umbrella tag', 'examples': [], 'category': 'Human prototypes'}, '<HH>': {'umbrella': False, 'description': 'Group of humans (organisations, teams, companies, e.g. editora)', 'examples': [], 'category': 'Human prototypes'}, '<Hattr>': {'umbrella': True, 'description': 'Attributive human umbrella tag (many -ista, -ante)', 'examples': [], 'category': 'Human prototypes'}, '<Hbio>': {'umbrella': False, 'description': 'Human classified by biological criteria (race, age etc., caboclo, mestiço, bebé, adulto) ', 'examples': [], 'category': 'Human prototypes'}, '<Hfam>': {'umbrella': False, 'description': 'Human with family or other private relation (pai, noiva)', 'examples': [], 'category': 'Human prototypes'}, '<Hideo>': {'umbrella': False, 'description': 'Ideological human (comunista, implies <Hattr>), also: follower, disciple (dadaista)', 'examples': [], 'category': 'Human prototypes'}, '<Hmyth>': {'umbrella': False, 'description': 'Humanoid mythical (gods, fairy tale humanoids, curupira, duende)', 'examples': [], 'category': 'Human prototypes'}, '<Hnat>': {'umbrella': False, 'description': 'Nationality human (brasileiro, alemão), also: inhabitant (lisboeta)', 'examples': [], 'category': 'Human prototypes'}, '<Hprof>': {'umbrella': False, 'description': 'Professional human (marinheiro, implies <Hattr>), also: sport, hobby (alpinista)', 'examples': [], 'category': 'Human prototypes'}, '<Hsick>': {'umbrella': False, 'description': 'Sick human (few: asmático, diabético, cp <sick>)', 'examples': [], 'category': 'Human prototypes'}, '<Htit>': {'umbrella': False, 'description': 'Title noun (rei, senhora)', 'examples': [], 'category': 'Human prototypes'}, '<L>': {'umbrella': True, 'description': 'Place, umbrella tag', 'examples': [], 'category': 'Place and spatial prototypes'}, '<Labs>': {'umbrella': False, 'description': 'Abstract place (anverso. auge)', 'examples': [], 'category': 'Place and spatial prototypes'}, '<Lciv>': {'umbrella': False, 'description': 'Civitas, town, country, county (equals <L> + <HH>, cidade, país)', 'examples': [], 'category': 'Place and spatial prototypes'}, '<Lcover>': {'umbrella': False, 'description': 'Cover, lid (colcha, lona, tampa)', 'examples': [], 'category': 'Place and spatial prototypes'}, '<Lh>': {'umbrella': False, 'description': 'Functional place, human built or human-used (aeroporto, anfiteatro, cp. <build> for just a building)', 'examples': [], 'category': 'Place and spatial prototypes'}, '<Lopening>': {'umbrella': False, 'description': 'opening, hole (apertura, fossa', 'examples': [], 'category': 'Place and spatial prototypes'}, '<Lpath>': {'umbrella': False, 'description': 'Path (road, street etc.: rua, pista)', 'examples': [], 'category': 'Place and spatial prototypes'}, '<Lstar>': {'umbrella': False, 'description': 'Star object (planets, comets: planeta, quasar)', 'examples': [], 'category': 'Place and spatial prototypes'}, '<Lsurf>': {'umbrella': False, 'description': 'surface (face, verniz, cp. <Lcover>)', 'examples': [], 'category': 'Place and spatial prototypes'}, '<Ltip>': {'umbrella': False, 'description': 'tip place, edge (pico, pontinha, cp. <Labs>)', 'examples': [], 'category': 'Place and spatial prototypes'}, '<Ltop>': {'umbrella': False, 'description': 'Geographical, natural place (promontório, pântano)', 'examples': [], 'category': 'Place and spatial prototypes'}, '<Ltrap>': {'umbrella': False, 'description': 'trap place (armadilha, armazelo)', 'examples': [], 'category': 'Place and spatial prototypes'}, '<Lwater>': {'umbrella': False, 'description': 'Water place (river, lake, sea: fonte, foz, lagoa)', 'examples': [], 'category': 'Place and spatial prototypes'}, '<V>': {'umbrella': False, 'description': '(PRODUTO) vehicle brand (cp. <V>, <Vair>, <Vwater>)', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<VV>': {'umbrella': False, 'description': 'Group of vehicles (armada, convoy: frota, esquadra)', 'examples': [], 'category': 'Vehicle prototypes'}, '<Vwater>': {'umbrella': False, 'description': 'Water vehicle (ship: navio, submersível, canoa)', 'examples': [], 'category': 'Vehicle prototypes'}, '<Vair>': {'umbrella': False, 'description': 'Air vehicle (plane: hidroplano, jatinho)', 'examples': [], 'category': 'Vehicle prototypes'}, '<ac>': {'umbrella': True, 'description': 'Abstract countable, umbrella tag (alternativa, chance, lazer)', 'examples': [], 'category': 'Abstract prototypes'}, '<ac-cat>': {'umbrella': False, 'description': 'Category word (latinismo, número atômico)', 'examples': [], 'category': 'Abstract prototypes'}, '<ac-sign>': {'umbrella': False, 'description': 'sign, symbol (parêntese, semicolcheia)', 'examples': [], 'category': 'Abstract prototypes'}, '<am>': {'umbrella': True, 'description': 'Abstract mass/non-countable, umbrella tag (still contains many cases that could be <f-..>, e.g. habilidade, legalidade)', 'examples': [], 'category': 'Abstract prototypes'}, '<ax>': {'umbrella': False, 'description': 'Abstract/concept, neither countable nor mass (endogamia), cp. <f>, <sit> etc.', 'examples': [], 'category': 'Abstract prototypes'}, '<act>': {'umbrella': True, 'description': 'Action, umbrella tag (+CONTROL, PERFECTIVE)', 'examples': [], 'category': 'Action prototypes'}, '<act-beat>': {'umbrella': False, 'description': 'beat-action (thrashing, pancada, surra)', 'examples': [], 'category': 'Action prototypes'}, '<act-d>': {'umbrella': False, 'description': 'do-action (typically dar/fazer + N, tentativa, teste, homenagem)', 'examples': [], 'category': 'Action prototypes'}, '<act-s>': {'umbrella': False, 'description': 'speech act or communicative act (proposta, ordem)', 'examples': [], 'category': 'Action prototypes'}, '<act-trick>': {'umbrella': False, 'description': 'trick-action (cheat, fraud, ruse, jeito, fraude, similar to <act-d>)', 'examples': [], 'category': 'Action prototypes'}, '<activity>': {'umbrella': True, 'description': 'Activity, umbrella tag (+CONTROL, IMPERFECTIVE, correria, manejo)', 'examples': [], 'category': 'Action prototypes'}, '<fight>': {'umbrella': False, 'description': 'fight, conflict (also <activity> and +TEMP, briga, querela)', 'examples': [], 'category': 'Action prototypes'}, '<an>': {'umbrella': True, 'description': 'Anatomical noun, umbrella tag (carótida, clítoris, dorso)', 'examples': [], 'category': 'Anatomical prototypes'}, '<anmov>': {'umbrella': False, 'description': 'Movable anatomy (arm, leg, braço, bíceps, cotovelo)', 'examples': [], 'category': 'Anatomical prototypes'}, '<anorg>': {'umbrella': False, 'description': 'Organ (heart, liver, hipófise, coração, testículo)', 'examples': [], 'category': 'Anatomical prototypes'}, '<anost>': {'umbrella': False, 'description': 'Bone (calcâneo, fíbula, vértebra)', 'examples': [], 'category': 'Anatomical prototypes'}, '<anzo>': {'umbrella': False, 'description': 'Animal anatomy (rúmen, carapaça, chifres, tromba)', 'examples': [], 'category': 'Anatomical prototypes'}, '<anorn>': {'umbrella': False, 'description': 'Bird anatomy (bico, pluma)', 'examples': [], 'category': 'Anatomical prototypes'}, '<anich>': {'umbrella': False, 'description': 'Fish anatomy (few: bránquias, siba)', 'examples': [], 'category': 'Anatomical prototypes'}, '<anent>': {'umbrella': False, 'description': 'Insect anatomy (few: tentáculo, olho composto)', 'examples': [], 'category': 'Anatomical prototypes'}, '<anbo>': {'umbrella': False, 'description': 'Plant anatomy (bulbo, caule, folha)', 'examples': [], 'category': 'Anatomical prototypes'}, '<cc>': {'umbrella': True, 'description': 'Concrete countable object, umbrella tag (briquete, coágulo, normally movable things, unlike <part-build>)', 'examples': [], 'category': 'Thing prototypes'}, '<cc-h>': {'umbrella': True, 'description': 'Artifact, umbrella tag (so far empty category in PALAVRAS)', 'examples': [], 'category': 'Thing prototypes'}, '<cc-beauty>': {'umbrella': False, 'description': 'ornamental object (few: guirlanda, rufo)', 'examples': [], 'category': 'Thing prototypes'}, '<cc-board>': {'umbrella': False, 'description': 'flat long object (few: board, plank, lousa, tabla)', 'examples': [], 'category': 'Thing prototypes'}, '<cc-fire>': {'umbrella': False, 'description': 'fire object (bonfire, spark, chispa, fogo, girândola)', 'examples': [], 'category': 'Thing prototypes'}, '<cc-handle>': {'umbrella': False, 'description': 'handle (garra, ansa, chupadouro)', 'examples': [], 'category': 'Thing prototypes'}, '<cc-light>': {'umbrella': False, 'description': 'light artifact (lampião, farol, projector)', 'examples': [], 'category': 'Thing prototypes'}, '<cc-particle>': {'umbrella': False, 'description': '(atomic) particle (few: cátion, eletrônio)', 'examples': [], 'category': 'Thing prototypes'}, '<cc-r>': {'umbrella': False, 'description': 'read object (carteira, cupom, bilhete, carta, cf. <sem-r>)', 'examples': [], 'category': 'Thing prototypes'}, '<cc-rag>': {'umbrella': False, 'description': 'cloth object (towel, napkin, carpet, rag) , cp. <mat-cloth>', 'examples': [], 'category': 'Thing prototypes'}, '<cc-stone>': {'umbrella': False, 'description': '(= cc-round) stones and stone-sized round objects (pedra, itá, amonite, tijolo)', 'examples': [], 'category': 'Thing prototypes'}, '<cc-stick>': {'umbrella': False, 'description': 'stick object (long and thin, vara, lançe, paulito)', 'examples': [], 'category': 'Thing prototypes'}, '<furn>': {'umbrella': False, 'description': 'furniture (cama, cadeira, tambo, quadro)', 'examples': [], 'category': 'Thing prototypes'}, '<amount>': {'umbrella': False, 'description': 'quantity noun (bocada, teor, sem-fim)', 'examples': [], 'category': 'Thing prototypes'}, '<bar>': {'umbrella': False, 'description': 'barrier noun (dique, limite, muralha)', 'examples': [], 'category': 'Thing prototypes'}, '<build>': {'umbrella': False, 'description': 'building (casa, citadela, garagem)', 'examples': [], 'category': 'Thing prototypes'}, '<cm>': {'umbrella': True, 'description': 'concrete mass/non-countable, umbrella tag, substance (cf. <mat>, terra, choça, magma)', 'examples': [], 'category': 'Substance prototypes'}, '<cm-h>': {'umbrella': False, 'description': 'human-made substance (cf. <mat>, cemento)', 'examples': [], 'category': 'Substance prototypes'}, '<cm-chem>': {'umbrella': False, 'description': 'chemical substance, also biological (acetileno, amônio, anilina, bilirrubina', 'examples': [], 'category': 'Substance prototypes'}, '<cm-gas>': {'umbrella': False, 'description': 'gas substance (so far few: argônio, overlap with. <cm-chem> and <cm>)', 'examples': [], 'category': 'Substance prototypes'}, '<cm-liq>': {'umbrella': False, 'description': 'liquid substance (azeite, gasolina, plasma, overlap with <food> and <cm-rem>)', 'examples': [], 'category': 'Substance prototypes'}, '<cm-rem>': {'umbrella': False, 'description': 'remedy (medical or hygiene, antibiótico, canabis, quinina, part of <cm-h>, overlap with <cm-chem>)', 'examples': [], 'category': 'Substance prototypes'}, '<cloA>': {'umbrella': False, 'description': 'animal clothing (sela, xabraque)', 'examples': [], 'category': 'Clothing prototypes'}, '<cloH>': {'umbrella': False, 'description': 'human clothing (albornoz, anoraque, babadouro, bermudas)', 'examples': [], 'category': 'Clothing prototypes'}, '<cloH-beauty>': {'umbrella': False, 'description': 'beauty clothing (e.g. jewelry, diadema, pendente, pulseira)', 'examples': [], 'category': 'Clothing prototypes'}, '<cloH-hat>': {'umbrella': False, 'description': 'hat (sombrero, mitra, coroa)', 'examples': [], 'category': 'Clothing prototypes'}, '<cloH-shoe>': {'umbrella': False, 'description': 'shoe (bota, chinela, patim)', 'examples': [], 'category': 'Clothing prototypes'}, '<coll>': {'umbrella': False, 'description': 'set, collective (random or systematic collection/compound/multitude of similar but distinct small parts, conjunto, série)', 'examples': [], 'category': 'Collective prototypes'}, '<coll-cc>': {'umbrella': False, 'description': 'thing collective, pile (baralho, lanço)', 'examples': [], 'category': 'Collective prototypes'}, '<coll-B>': {'umbrella': False, 'description': 'plant-part collective (buquê, folhagem)', 'examples': [], 'category': 'Collective prototypes'}, '<coll-sem>': {'umbrella': False, 'description': 'semantic collective, collection (arquivo, repertório)', 'examples': [], 'category': 'Collective prototypes'}, '<coll-tool>': {'umbrella': False, 'description': 'tool collective, set (intrumentário, prataria)', 'examples': [], 'category': 'Collective prototypes'}, '<col>': {'umbrella': False, 'description': 'color adjective', 'examples': [], 'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'}, '<con>': {'umbrella': False, 'description': 'container (implies <num+> quantifying, ampola, chícara, aquário)', 'examples': [], 'category': 'Collective prototypes'}, '<conv>': {'umbrella': False, 'description': 'convention (social rule or law, lei, preceito)', 'examples': [], 'category': 'Collective prototypes'}, '<cord>': {'umbrella': False, 'description': 'cord, string, rope, tape (previously <tool-tie>, arame, fio, fibrila)', 'examples': [], 'category': 'Collective prototypes'}, '<cur>': {'umbrella': False, 'description': 'currency noun (countable, implies <unit>, cf. <mon>, dirham, euro, real, dólar)', 'examples': [], 'category': 'Collective prototypes'}, '<dance>': {'umbrella': False, 'description': 'dance (both <activity>, <genre> and <sem-l>, calipso, flamenco, forró)', 'examples': [], 'category': 'Collective prototypes'}, '<dir>': {'umbrella': False, 'description': 'direction noun (estibordo, contrasenso, norte)', 'examples': [], 'category': 'Collective prototypes'}, '<domain>': {'umbrella': False, 'description': 'domain (subject matter, profession, cf. <genre>, anatomia, citricultura, dactilografia)', 'examples': [], 'category': 'Collective prototypes'}, '<dur>': {'umbrella': False, 'description': 'duration noun (test: durar+, implies <unit>, e.g. átimo, mês, hora)', 'examples': [], 'category': 'Time and event prototypes'}, '<per>': {'umbrella': False, 'description': 'period of time (prototypical test: durante, e.g. guerra, década, cf. <dur> and <temp>)', 'examples': [], 'category': 'Concept prototypes'}, '<event>': {'umbrella': False, 'description': '(EVENTO) non-organised event', 'examples': [], 'category': 'Event categories <occ>, HAREM ACONTECIMENTO (HAREM tags in parenthesis)'}, '<occ>': {'umbrella': False, 'description': '(ORGANIZADO) organised event', 'examples': [], 'category': 'Event categories <occ>, HAREM ACONTECIMENTO (HAREM tags in parenthesis)'}, '<f>': {'umbrella': True, 'description': 'feature/property, umbrella tag (problematicidade, proporcionalidade)', 'examples': [], 'category': 'Feature prototypes'}, '<f-an>': {'umbrella': False, 'description': 'anatomical "local" feature, includes countables, e.g. barbela, olheiras)', 'examples': [], 'category': 'Feature prototypes'}, '<f-c>': {'umbrella': False, 'description': 'general countable feature (vestígio, laivos, vinco)', 'examples': [], 'category': 'Feature prototypes'}, '<f-h>': {'umbrella': False, 'description': 'human physical feature, not countable (lindura, compleição, same as <f-phys-h>, cp. anatomical local features <f-an>)', 'examples': [], 'category': 'Feature prototypes'}, '<f-psych>': {'umbrella': False, 'description': 'human psychological feature (passionalidade, pavonice, cp. passing states <state-h>)', 'examples': [], 'category': 'Feature prototypes'}, '<f-q>': {'umbrella': False, 'description': "quantifiable feature (e.g. circunferência, calor, DanGram's <f-phys> covers both <f> and <f-q>)", 'examples': [], 'category': 'Feature prototypes'}, '<f-right>': {'umbrella': False, 'description': 'human social feature (right or duty): e.g. copyright, privilégio, imperativo legal)', 'examples': [], 'category': 'Feature prototypes'}, '<food>': {'umbrella': False, 'description': 'natural/simplex food (aveia, açúcar, carne, so far including <spice>)', 'examples': [], 'category': 'Food prototypes'}, '<food-c>': {'umbrella': False, 'description': 'countable food (few: ovo, dente de alho, most are <fruit> or <food-c-h>)', 'examples': [], 'category': 'Food prototypes'}, '<food-h>': {'umbrella': False, 'description': 'human-prepared/complex culinary food (caldo verde, lasanha)', 'examples': [], 'category': 'Food prototypes'}, '<food-c-h>': {'umbrella': False, 'description': 'culinary countable food (biscoito, enchido, panetone, pastel)', 'examples': [], 'category': 'Food prototypes'}, '<drink>': {'umbrella': False, 'description': 'drink (cachaça, leite, guaraná, moca)', 'examples': [], 'category': 'Food prototypes'}, '<game>': {'umbrella': False, 'description': 'play, game (bilhar, ioiô, poker, also <activity>)', 'examples': [], 'category': 'Concept prototypes'}, '<genre>': {'umbrella': False, 'description': '(DISCIPLINA) subject matter', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<geom>': {'umbrella': False, 'description': 'geometry noun (circle, shape, e.g. losango, octógono, elipse)', 'examples': [], 'category': 'Concept prototypes'}, '<geom-line>': {'umbrella': False, 'description': 'line (few: linha, percentil, curvas isobáricas)', 'examples': [], 'category': 'Concept prototypes'}, '<inst>': {'umbrella': False, 'description': '(EMPRESA) organized site (e.g. restaurant, cp. <Linst>)', 'examples': [], 'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'}, '<ism>': {'umbrella': False, 'description': 'ideology or other value system (anarquismo, anti-ocidentalismo, apartheid)', 'examples': [], 'category': 'Concept prototypes'}, '<ling>': {'umbrella': False, 'description': 'language (alemão, catalão, bengali)', 'examples': [], 'category': 'Concept prototypes'}, '<mach>': {'umbrella': False, 'description': 'machine (complex, usually with moving parts, betoneira, embrulhador, limpa-pratos, cp. <tool>)', 'examples': [], 'category': 'Concept prototypes'}, '<mat>': {'umbrella': False, 'description': '(SUBSTANCIA) substance', 'examples': [], 'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'}, '<mat-cloth>': {'umbrella': False, 'description': 'cloth material (seda, couro, vison, kevlar), cp. <cc-rag>', 'examples': [], 'category': 'Concept prototypes'}, '<meta>': {'umbrella': False, 'description': 'meta noun (tipo, espécie)', 'examples': [], 'category': 'Concept prototypes'}, '<mon>': {'umbrella': False, 'description': 'amount of money (bolsa, custo, imposto, cf. <cur>)', 'examples': [], 'category': 'Concept prototypes'}, '<month>': {'umbrella': False, 'description': 'month noun/name (agosto, julho, part of <temp>)', 'examples': [], 'category': 'Concept prototypes'}, '<part>': {'umbrella': False, 'description': 'distinctive or functional part (ingrediente, parte, trecho)', 'examples': [], 'category': 'Part prototypes'}, '<part-build>': {'umbrella': False, 'description': 'structural part of building or vehicle (balustrada, porta, estai)', 'examples': [], 'category': 'Part prototypes'}, '<piece>': {'umbrella': False, 'description': 'indistinctive (little) piece (pedaço, raspa)', 'examples': [], 'category': 'Part prototypes'}, '<percep-f>': {'umbrella': False, 'description': 'what you feel (senses or sentiment, pain, e.g. arrepio, aversão, desagrado, cócegas, some overlap with <state-h>)', 'examples': [], 'category': 'Perception prototypes'}, '<percep-l>': {'umbrella': False, 'description': 'sound (what you hear, apitadela, barrulho, berro, crepitação)', 'examples': [], 'category': 'Perception prototypes'}, '<percep-o>': {'umbrella': False, 'description': 'olfactory impression (what you smell, bafo, chamuscom fragrância)', 'examples': [], 'category': 'Perception prototypes'}, '<percep-t>': {'umbrella': False, 'description': 'what you taste (PALAVRAS: not implemented)', 'examples': [], 'category': 'Perception prototypes'}, '<percep-w>': {'umbrella': False, 'description': 'visual impression (what you see, arco-iris, réstia, vislumbre)', 'examples': [], 'category': 'Perception prototypes'}, '<pict>': {'umbrella': False, 'description': 'picture (combination of <cc>, <sem-w> and <L>, caricatura, cintilograma, diapositivo)', 'examples': [], 'category': 'Perception prototypes'}, '<pos-an>': {'umbrella': False, 'description': 'anatomical/body position (few: desaprumo)', 'examples': [], 'category': 'Perception prototypes'}, '<pos-soc>': {'umbrella': False, 'description': 'social position, job (emprego, condado, capitania, presidência)', 'examples': [], 'category': 'Perception prototypes'}, '<process>': {'umbrella': False, 'description': 'process (-CONTROL, -PERFECTIVE, cp. <event>, balcanização, convecção, estagnação)', 'examples': [], 'category': 'Perception prototypes'}, '<sem>': {'umbrella': True, 'description': 'semiotic artifact, work of art, umbrella tag (all specified in PALAVRAS)', 'examples': [], 'category': 'Semantic product prototypes'}, '<sem-c>': {'umbrella': False, 'description': 'cognition product (concept, plan, system, conjetura, esquema, plano, prejuízo)', 'examples': [], 'category': 'Semantic product prototypes'}, '<sem-l>': {'umbrella': False, 'description': 'listen-work (music, cantarola, prelúdio, at the same time <genre>: bossa nova)', 'examples': [], 'category': 'Semantic product prototypes'}, '<sem-nons>': {'umbrella': False, 'description': 'nonsense, rubbish (implies <sem-s>, galimatias, farelório)', 'examples': [], 'category': 'Semantic product prototypes'}, '<sem-r>': {'umbrella': False, 'description': 'read-work (biografia, dissertação, e-mail, ficha cadastral)', 'examples': [], 'category': 'Semantic product prototypes'}, '<sem-s>': {'umbrella': False, 'description': 'speak-work (palestra, piada, exposto)', 'examples': [], 'category': 'Semantic product prototypes'}, '<sem-w>': {'umbrella': False, 'description': 'watch-work (filme, esquete, mininovela)', 'examples': [], 'category': 'Semantic product prototypes'}, '<sick>': {'umbrella': False, 'description': 'disease (acne, AIDS, sida, alcoolismo, cp. <Hsick>)', 'examples': [], 'category': 'Semantic product prototypes'}, '<sick-c>': {'umbrella': False, 'description': 'countable disease-object (abscesso, berruga, cicatriz, gangrena)', 'examples': [], 'category': 'Semantic product prototypes'}, '<sit>': {'umbrella': False, 'description': 'psychological situation or physical state of affairs (reclusão, arruaça, ilegalidade, more complex and more "locative" than <state> and <state-h>', 'examples': [], 'category': 'State-of-affairs prototypes'}, '<state>': {'umbrella': False, 'description': 'state (of something, otherwise <sit>), abundância, calma, baixa-mar, equilíbrio', 'examples': [], 'category': 'State-of-affairs prototypes'}, '<state-h>': {'umbrella': False, 'description': 'human state (desamparo, desesperança, dormência, euforia, febre, cp. <f-psych> and <f-phys-h>, which cover innate features)', 'examples': [], 'category': 'State-of-affairs prototypes'}, '<sport>': {'umbrella': False, 'description': 'sport (capoeira, futebol, golfe, also <activity> and <domain>)', 'examples': [], 'category': 'State-of-affairs prototypes'}, '<talk>': {'umbrella': False, 'description': 'speech situation, talk, discussion, quarrel (implies <activity> and <sd>, entrevista, lero-lero)', 'examples': [], 'category': 'State-of-affairs prototypes'}, '<temp>': {'umbrella': False, 'description': 'temporal object, point in time (amanhecer, novilúnio, test: até+, cf. <dur> and <per>)', 'examples': [], 'category': 'State-of-affairs prototypes'}, '<therapy>': {'umbrella': False, 'description': 'therapy (also <domain> and <activity>, acupuntura, balneoterapia)', 'examples': [], 'category': 'State-of-affairs prototypes'}, '<tool>': {'umbrella': True, 'description': 'tool, umbrella tag (abana-moscas, lápis, computador, maceta, "handable", cf. <mach>)', 'examples': [], 'category': 'Tool prototypes'}, '<tool-cut>': {'umbrella': False, 'description': 'cutting tool, knife (canivete, espada)', 'examples': [], 'category': 'Tool prototypes'}, '<tool-gun>': {'umbrella': False, 'description': 'shooting tool, gun (carabina, metralhadora, helicanão, in Dangram: <tool-shoot>)', 'examples': [], 'category': 'Tool prototypes'}, '<tool-mus>': {'umbrella': False, 'description': 'musical instrument (clavicórdio, ocarina, violão)', 'examples': [], 'category': 'Tool prototypes'}, '<tool-sail>': {'umbrella': False, 'description': 'sailing tool, sail (vela latina, joanete, coringa)', 'examples': [], 'category': 'Tool prototypes'}, '<tube>': {'umbrella': False, 'description': 'tube object (cânula, gasoduto, zarabatana, shape-category, typically with another category, like <an> or <tool>)', 'examples': [], 'category': 'Tool prototypes'}, '<unit>': {'umbrella': False, 'description': 'unit noun (always implying <num+>, implied by <cur> and <dur>, e.g. caloria, centímetro, lúmen))', 'examples': [], 'category': 'Tool prototypes'}, '<wea>': {'umbrella': True, 'description': 'weather (states), umbrella tag (friagem, bruma)', 'examples': [], 'category': 'Weather prototypes'}, '<wea-c>': {'umbrella': False, 'description': 'countable weather phenomenon (nuvem, tsunami)', 'examples': [], 'category': 'Weather prototypes'}, '<wea-rain>': {'umbrella': False, 'description': "rain and other precipitation (chuvisco, tromba d'água, granizo)", 'examples': [], 'category': 'Weather prototypes'}, '<wea-wind>': {'umbrella': False, 'description': 'wind, storm (brisa, furacão)', 'examples': [], 'category': 'Weather prototypes'}, '<hum>': {'umbrella': False, 'description': '(INDIVIDUAL) person name (cp. <H>)', 'examples': [], 'category': 'Person categories <hum>, HAREM PESSOA (HAREM tags in parenthesis)'}, '<official>': {'umbrella': False, 'description': '(CARGO) official function (~ cp. <Htitle> and <Hprof>)', 'examples': [], 'category': 'Person categories <hum>, HAREM PESSOA (HAREM tags in parenthesis)'}, '<member>': {'umbrella': False, 'description': '(MEMBRO) member', 'examples': [], 'category': 'Person categories <hum>, HAREM PESSOA (HAREM tags in parenthesis)'}, '<admin>': {'umbrella': False, 'description': '(ADMINISTRACAO, ORG.) administrative body (government, town administration etc.)', 'examples': [], 'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'}, '<org>': {'umbrella': False, 'description': '(INSTITUICAO/EMPRESA) commercial or non-commercial, non-administrative non-party organisations (not place-bound, therefore not the same as <Linst>)', 'examples': [], 'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'}, '<media>': {'umbrella': False, 'description': '(EMPRESA) media organisation (e.g. newspaper, tv channel)', 'examples': [], 'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'}, '<party>': {'umbrella': False, 'description': '(INSTITUICAO) political party', 'examples': [], 'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'}, '<suborg>': {'umbrella': False, 'description': '(SUB) organized part of any of the above', 'examples': [], 'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'}, '<company>': {'umbrella': False, 'description': '(EMPRESA) company (not site-bound, unlike <inst>, now fused with. <org>) - currently unsupported', 'examples': [], 'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'}, '<groupind>': {'umbrella': False, 'description': '(GROUPOIND) people, family', 'examples': [], 'category': 'Group categories, HAREM PESSOA (HAREM tags in parenthesis)'}, '<groupofficial>': {'umbrella': False, 'description': '(GROUPOCARGO) board, government (not fully implemented)', 'examples': [], 'category': 'Group categories, HAREM PESSOA (HAREM tags in parenthesis)'}, '<grouporg>': {'umbrella': False, 'description': '(GROUPOMEMBRO) club, e.g. football club (now fused with <org>) - currently unsupported', 'examples': [], 'category': 'Group categories, HAREM PESSOA (HAREM tags in parenthesis)'}, '<top>': {'umbrella': False, 'description': '(GEOGRAFICO) geographical location (cp. <Ltop>)', 'examples': [], 'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'}, '<civ>': {'umbrella': False, 'description': '(ADMINISTRACAO, LOC.) civitas (country, town, state, cp. <Lciv>)', 'examples': [], 'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'}, '<address>': {'umbrella': False, 'description': '(CORREIO) address (including numbers etc.)', 'examples': [], 'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'}, '<site>': {'umbrella': False, 'description': '(ALARGADO) functional place (cp. <Lh>)', 'examples': [], 'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'}, '<virtual>': {'umbrella': False, 'description': '(VIRTUAL) virtual place', 'examples': [], 'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'}, '<astro>': {'umbrella': False, 'description': '(OBJECTO) astronomical place (in HAREM object, not place)', 'examples': [], 'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'}, '<road>': {'umbrella': False, 'description': '(ALARGADO) roads, motorway (unlike <address>) - suggested', 'examples': [], 'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'}, '<history>': {'umbrella': False, 'description': '(EFEMERIDE) one-time [historical] occurrence', 'examples': [], 'category': 'Event categories <occ>, HAREM ACONTECIMENTO (HAREM tags in parenthesis)'}, '<tit>': {'umbrella': False, 'description': '(REPRODUZIDO) [title of] reproduced work, copy', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<pub>': {'umbrella': False, 'description': '(PUBLICACAO) [scientific] publication', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<product>': {'umbrella': False, 'description': '(PRODUTO) product brand', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<artwork>': {'umbrella': False, 'description': '(ARTE) work of art', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, 'Abstract': {'umbrella': False, 'description': 'categories <brand>, HAREM ABSTRACCAO (HAREM tags in parenthesis): ', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<brand>': {'umbrella': False, 'description': '(MARCA) brand', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<school>': {'umbrella': False, 'description': '(ESCOLA) school of thought', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<idea>': {'umbrella': False, 'description': '(IDEA) idea, concept', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<plan>': {'umbrella': False, 'description': '(PLANO) named plan, project', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<author>': {'umbrella': False, 'description': "(OBRA) artist's name, standing for body of work", 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<absname>': {'umbrella': False, 'description': '(NOME)', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<disease>': {'umbrella': False, 'description': '(ESTADO) physiological state, in particular: disease', 'examples': [], 'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'}, '<object>': {'umbrella': False, 'description': '(OBJECT) named object', 'examples': [], 'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'}, '<common>': {'umbrella': False, 'description': '(OBJECT) common noun used as name', 'examples': [], 'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'}, '<class>': {'umbrella': False, 'description': '(CLASSE) classification category for things', 'examples': [], 'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'}, '<plant>': {'umbrella': False, 'description': '(CLASSE) plant name', 'examples': [], 'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'}, '<currency>': {'umbrella': False, 'description': '(MOEDA) currency name (also marked on the unit)', 'examples': [], 'category': "Numeric value categories (marked only on the numeral, without MWE'ing, unlike HAREM VALOR)"}, '<date>': {'umbrella': False, 'description': '(DATA) date', 'examples': [], 'category': "Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO)"}, '<hour>': {'umbrella': False, 'description': '(HORA) hour', 'examples': [], 'category': "Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO)"}, '<period>': {'umbrella': False, 'description': '(PERIODO) period', 'examples': [], 'category': "Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO)"}, '<cyclic>': {'umbrella': False, 'description': '(CICLICO) cyclic time expression', 'examples': [], 'category': "Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO)"}, '<quantity>': {'umbrella': False, 'description': '(QUANTIDADE) simple measuring numeral', 'examples': [], 'category': "Numeric value categories (marked only on the numeral, without MWE'ing, unlike HAREM VALOR)"}, '<prednum>': {'umbrella': False, 'description': '(CLASSIFICADO) predicating numeral', 'examples': [], 'category': "Numeric value categories (marked only on the numeral, without MWE'ing, unlike HAREM VALOR)"}, '<mass>': {'umbrella': False, 'description': 'mass noun (e.g. "leite", "a\'gua")', 'examples': [], 'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'}, '<jh>': {'umbrella': False, 'description': 'adjective modifying human noun', 'examples': [], 'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'}, '<jn>': {'umbrella': False, 'description': 'adjective modifying inanimate noun', 'examples': [], 'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'}, '<ja>': {'umbrella': False, 'description': 'adjective modifying animal', 'examples': [], 'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'}, '<jb>': {'umbrella': False, 'description': 'adjective modifying plant', 'examples': [], 'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'}, '<nat>': {'umbrella': False, 'description': 'nationality adjective (also: from a certain town etc.)', 'examples': [], 'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'}, '<attr>': {'umbrella': False, 'description': '(human) attributive adjective (not fully implemented, cp. <Hattr>, e.g. "um presidente COMUNISTA")', 'examples': [], 'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'}, '<vH>': {'umbrella': False, 'description': 'verb with human subject', 'examples': [], 'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'}, '<vN>': {'umbrella': False, 'description': 'verb with inanimate subject', 'examples': [], 'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'}}"""

print(ss_dic.replace("},","},\n").replace(", '", ", \n\t'"))



{'<A>': {'umbrella': True, 
	'description': 'Animal, umbrella tag (clone, fêmea, fóssil, parasito, predador)', 
	'examples': [], 
	'category': 'Animal prototypes'},
 '<AA>': {'umbrella': False, 
	'description': 'Group of animals (cardume, enxame, passarada, ninhada)', 
	'examples': [], 
	'category': 'Animal prototypes'},
 '<Adom>': {'umbrella': False, 
	'description': 'Domestic animal or big mammal (likely to have female forms etc.: terneiro, leão/leoa, cachorro)', 
	'examples': [], 
	'category': 'Animal prototypes'},
 '<AAdom>': {'umbrella': False, 
	'description': 'Group of domestic animals (boiada)', 
	'examples': [], 
	'category': 'Animal prototypes'},
 '<Aich>': {'umbrella': False, 
	'description': 'Water-animal (tubarão, delfim)', 
	'examples': [], 
	'category': 'Animal prototypes'},
 '<Amyth>': {'umbrella': False, 
	'description': 'Mythological animal (basilisco)', 
	'examples': [], 
	'category': 'Animal prototypes'},
 '<Azo>': {'umbrella': False, 
	'description': 'Land-animal (

In [347]:
n_semantic = {
    '<A>': {'umbrella': True, 
            'description': 'Animal, umbrella tag (clone, fêmea, fóssil, parasito, predador)', 
            'examples': [], 
            'category': 'Animal prototypes'},
     '<AA>': {'umbrella': False, 
              'description': 'Group of animals (cardume, enxame, passarada, ninhada)', 
              'examples': [], 
              'category': 'Animal prototypes'},
     '<Adom>': {'umbrella': False, 
                'description': 'Domestic animal or big mammal (likely to have female forms etc.: terneiro, leão/leoa, cachorro)', 
                'examples': [], 
                'category': 'Animal prototypes'},
     '<AAdom>': {'umbrella': False, 
                'description': 'Group of domestic animals (boiada)', 
                'examples': [], 
                'category': 'Animal prototypes'},
     '<Aich>': {'umbrella': False, 
                'description': 'Water-animal (tubarão, delfim)', 
                'examples': [], 
                'category': 'Animal prototypes'},
     '<Amyth>': {'umbrella': False, 
                'description': 'Mythological animal (basilisco)', 
                'examples': [], 
                'category': 'Animal prototypes'},
     '<Azo>': {'umbrella': False, 
                'description': 'Land-animal (raposa)', 
                'examples': [], 
                'category': 'Animal prototypes'},
     '<Aorn>': {'umbrella': False, 
                'description': 'Bird (águia, bem-te-vi)', 
                'examples': [], 
                'category': 'Animal prototypes'},
     '<Aent>': {'umbrella': False, 
                'description': 'Insect (borboleta)', 
                'examples': [], 
                'category': 'Animal prototypes'},
     '<Acell>': {'umbrella': False, 
                'description': 'Cell-animal (bacteria, blood cells: linfócito)', 
                'examples': [], 
                'category': 'Animal prototypes'},
     '<B>': {'umbrella': True, 
            'description': 'Plant, umbrella tag', 
            'examples': [], 
            'category': 'Plant prototypes'},
     '<BB>': {'umbrella': False, 
            'description': 'Group of plants, plantation (field, forest etc.: mata, nabal)', 
            'examples': [], 
            'category': 'Plant prototypes'},
     '<Btree>': {'umbrella': False, 
                'description': 'Tree (oliveira, palmeira)', 
                'examples': [], 
                'category': 'Plant prototypes'},
     '<Bflo>': {'umbrella': False, 
                'description': 'Flower (rosa, taraxaco)', 
                'examples': [], 
                'category': 'Plant prototypes'},
     '<Bbush>': {'umbrella': False, 
                'description': 'Bush, shrub (rododendro, tamariz)', 
                'examples': [], 
                'category': 'Plant prototypes'},
     '<fruit>': {'umbrella': False, 
                'description': 'fruit, berry, nut (still mostly marked as <food-c>, abricote, amora, avelã, cebola)', 
                'examples': [], 
                'category': 'Food prototypes'},
     '<Bveg>': {'umbrella': False, 
                'description': '(vegetable espargo, funcho) - further proposed categories', 
                'examples': [], 
                'category': 'Plant prototypes'},
     '<H>': {'umbrella': True, 
            'description': 'Human, umbrella tag', 
            'examples': [], 
            'category': 'Human prototypes'},
     '<HH>': {'umbrella': False, 
            'description': 'Group of humans (organisations, teams, companies, e.g. editora)', 
            'examples': [], 
            'category': 'Human prototypes'},
     '<Hattr>': {'umbrella': True, 
                'description': 'Attributive human umbrella tag (many -ista, -ante)', 
                'examples': [], 
                'category': 'Human prototypes'},
     '<Hbio>': {'umbrella': False, 
                'description': 'Human classified by biological criteria (race, age etc., caboclo, mestiço, bebé, adulto) ', 
                'examples': [], 
                'category': 'Human prototypes'},
     '<Hfam>': {'umbrella': False, 
                'description': 'Human with family or other private relation (pai, noiva)', 
                'examples': [], 
                'category': 'Human prototypes'},
     '<Hideo>': {'umbrella': False, 
                'description': 'Ideological human (comunista, implies <Hattr>), also: follower, disciple (dadaista)', 
                'examples': [], 
                'category': 'Human prototypes'},
     '<Hmyth>': {'umbrella': False, 
                'description': 'Humanoid mythical (gods, fairy tale humanoids, curupira, duende)', 
                'examples': [], 
                'category': 'Human prototypes'},
     '<Hnat>': {'umbrella': False, 
                'description': 'Nationality human (brasileiro, alemão), also: inhabitant (lisboeta)', 
                'examples': [], 
                'category': 'Human prototypes'},
     '<Hprof>': {'umbrella': False, 
                'description': 'Professional human (marinheiro, implies <Hattr>), also: sport, hobby (alpinista)', 
                'examples': [], 
                'category': 'Human prototypes'},
     '<Hsick>': {'umbrella': False, 
                'description': 'Sick human (few: asmático, diabético, cp <sick>)', 
                'examples': [], 
                'category': 'Human prototypes'},
     '<Htit>': {'umbrella': False, 
                'description': 'Title noun (rei, senhora)', 
                'examples': [], 
                'category': 'Human prototypes'},
     '<L>': {'umbrella': True, 
            'description': 'Place, umbrella tag', 
            'examples': [], 
            'category': 'Place and spatial prototypes'},
     '<Labs>': {'umbrella': False, 
                'description': 'Abstract place (anverso. auge)', 
                'examples': [], 
                'category': 'Place and spatial prototypes'},
     '<Lciv>': {'umbrella': False, 
                'description': 'Civitas, town, country, county (equals <L> + <HH>, cidade, país)', 
                'examples': [], 
                'category': 'Place and spatial prototypes'},
     '<Lcover>': {'umbrella': False, 
                'description': 'Cover, lid (colcha, lona, tampa)', 
                'examples': [], 
                'category': 'Place and spatial prototypes'},
     '<Lh>': {'umbrella': False, 
                'description': 'Functional place, human built or human-used (aeroporto, anfiteatro, cp. <build> for just a building)', 
                'examples': [], 
                'category': 'Place and spatial prototypes'},
     '<Lopening>': {'umbrella': False, 
                    'description': 'opening, hole (apertura, fossa', 
                    'examples': [], 
                    'category': 'Place and spatial prototypes'},
     '<Lpath>': {'umbrella': False, 
                'description': 'Path (road, street etc.: rua, pista)', 
                'examples': [], 
                'category': 'Place and spatial prototypes'},
     '<Lstar>': {'umbrella': False, 
                'description': 'Star object (planets, comets: planeta, quasar)', 
                'examples': [], 
                'category': 'Place and spatial prototypes'},
     '<Lsurf>': {'umbrella': False, 
                'description': 'surface (face, verniz, cp. <Lcover>)', 
                'examples': [], 
                'category': 'Place and spatial prototypes'},
     '<Ltip>': {'umbrella': False, 
                'description': 'tip place, edge (pico, pontinha, cp. <Labs>)', 
                'examples': [], 
                'category': 'Place and spatial prototypes'},
     '<Ltop>': {'umbrella': False, 
                'description': 'Geographical, natural place (promontório, pântano)', 
                'examples': [], 
                'category': 'Place and spatial prototypes'},
     '<Ltrap>': {'umbrella': False, 
                'description': 'trap place (armadilha, armazelo)', 
                'examples': [], 
                'category': 'Place and spatial prototypes'},
     '<Lwater>': {'umbrella': False, 
                'description': 'Water place (river, lake, sea: fonte, foz, lagoa)', 
                'examples': [], 
                'category': 'Place and spatial prototypes'},
     '<V>': {'umbrella': False, 
            'description': '(PRODUTO) vehicle brand (cp. <V>, <Vair>, <Vwater>)', 
            'examples': [], 
            'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<VV>': {'umbrella': False, 
            'description': 'Group of vehicles (armada, convoy: frota, esquadra)', 
            'examples': [], 
            'category': 'Vehicle prototypes'},
     '<Vwater>': {'umbrella': False, 
                'description': 'Water vehicle (ship: navio, submersível, canoa)', 
                'examples': [], 
                'category': 'Vehicle prototypes'},
     '<Vair>': {'umbrella': False, 
                'description': 'Air vehicle (plane: hidroplano, jatinho)', 
                'examples': [], 
                'category': 'Vehicle prototypes'},
     '<ac>': {'umbrella': True, 
            'description': 'Abstract countable, umbrella tag (alternativa, chance, lazer)', 
            'examples': [], 
            'category': 'Abstract prototypes'},
     '<ac-cat>': {'umbrella': False, 
                'description': 'Category word (latinismo, número atômico)', 
                'examples': [], 
                'category': 'Abstract prototypes'},
     '<ac-sign>': {'umbrella': False, 
                'description': 'sign, symbol (parêntese, semicolcheia)', 
                'examples': [], 
                'category': 'Abstract prototypes'},
     '<am>': {'umbrella': True, 
            'description': 'Abstract mass/non-countable, umbrella tag (still contains many cases that could be <f-..>, e.g. habilidade, legalidade)', 
            'examples': [], 
            'category': 'Abstract prototypes'},
     '<ax>': {'umbrella': False, 
            'description': 'Abstract/concept, neither countable nor mass (endogamia), cp. <f>, <sit> etc.', 
            'examples': [], 
            'category': 'Abstract prototypes'},
     '<act>': {'umbrella': True, 
                'description': 'Action, umbrella tag (+CONTROL, PERFECTIVE)', 
                'examples': [], 
                'category': 'Action prototypes'},
     '<act-beat>': {'umbrella': False, 
                'description': 'beat-action (thrashing, pancada, surra)', 
                'examples': [], 
                'category': 'Action prototypes'},
     '<act-d>': {'umbrella': False, 
                'description': 'do-action (typically dar/fazer + N, tentativa, teste, homenagem)', 
                'examples': [], 
                'category': 'Action prototypes'},
     '<act-s>': {'umbrella': False, 
                'description': 'speech act or communicative act (proposta, ordem)', 
                'examples': [], 
                'category': 'Action prototypes'},
     '<act-trick>': {'umbrella': False, 
                    'description': 'trick-action (cheat, fraud, ruse, jeito, fraude, similar to <act-d>)', 
                    'examples': [], 
                    'category': 'Action prototypes'},
     '<activity>': {'umbrella': True, 
                    'description': 'Activity, umbrella tag (+CONTROL, IMPERFECTIVE, correria, manejo)', 
                    'examples': [], 
                    'category': 'Action prototypes'},
     '<fight>': {'umbrella': False, 
                'description': 'fight, conflict (also <activity> and +TEMP, briga, querela)', 
                'examples': [], 
                'category': 'Action prototypes'},
     '<an>': {'umbrella': True, 
            'description': 'Anatomical noun, umbrella tag (carótida, clítoris, dorso)', 
            'examples': [], 
            'category': 'Anatomical prototypes'},
     '<anmov>': {'umbrella': False, 
                'description': 'Movable anatomy (arm, leg, braço, bíceps, cotovelo)', 
                'examples': [], 
                'category': 'Anatomical prototypes'},
     '<anorg>': {'umbrella': False, 
                'description': 'Organ (heart, liver, hipófise, coração, testículo)', 
                'examples': [], 
                'category': 'Anatomical prototypes'},
     '<anost>': {'umbrella': False, 
                'description': 'Bone (calcâneo, fíbula, vértebra)', 
                'examples': [], 
                'category': 'Anatomical prototypes'},
     '<anzo>': {'umbrella': False, 
                'description': 'Animal anatomy (rúmen, carapaça, chifres, tromba)', 
                'examples': [], 
                'category': 'Anatomical prototypes'},
     '<anorn>': {'umbrella': False, 
                'description': 'Bird anatomy (bico, pluma)', 
                'examples': [], 
                'category': 'Anatomical prototypes'},
     '<anich>': {'umbrella': False, 
                'description': 'Fish anatomy (few: bránquias, siba)', 
                'examples': [], 
                'category': 'Anatomical prototypes'},
     '<anent>': {'umbrella': False, 
                'description': 'Insect anatomy (few: tentáculo, olho composto)', 
                'examples': [], 
                'category': 'Anatomical prototypes'},
     '<anbo>': {'umbrella': False, 
                'description': 'Plant anatomy (bulbo, caule, folha)', 
                'examples': [], 
                'category': 'Anatomical prototypes'},
     '<cc>': {'umbrella': True, 
            'description': 'Concrete countable object, umbrella tag (briquete, coágulo, normally movable things, unlike <part-build>)', 
            'examples': [], 
            'category': 'Thing prototypes'},
     '<cc-h>': {'umbrella': True, 
                'description': 'Artifact, umbrella tag (so far empty category in PALAVRAS)', 
                'examples': [], 
                'category': 'Thing prototypes'},
     '<cc-beauty>': {'umbrella': False, 
                    'description': 'ornamental object (few: guirlanda, rufo)', 
                    'examples': [], 
                    'category': 'Thing prototypes'},
     '<cc-board>': {'umbrella': False, 
                    'description': 'flat long object (few: board, plank, lousa, tabla)', 
                    'examples': [], 
                    'category': 'Thing prototypes'},
     '<cc-fire>': {'umbrella': False, 
                'description': 'fire object (bonfire, spark, chispa, fogo, girândola)', 
                'examples': [], 
                'category': 'Thing prototypes'},
     '<cc-handle>': {'umbrella': False, 
                    'description': 'handle (garra, ansa, chupadouro)', 
                    'examples': [], 
                    'category': 'Thing prototypes'},
     '<cc-light>': {'umbrella': False, 
                    'description': 'light artifact (lampião, farol, projector)', 
                    'examples': [], 
                    'category': 'Thing prototypes'},
     '<cc-particle>': {'umbrella': False, 
                    'description': '(atomic) particle (few: cátion, eletrônio)', 
                    'examples': [], 
                    'category': 'Thing prototypes'},
     '<cc-r>': {'umbrella': False, 
                'description': 'read object (carteira, cupom, bilhete, carta, cf. <sem-r>)', 
                'examples': [], 
                'category': 'Thing prototypes'},
     '<cc-rag>': {'umbrella': False, 
                'description': 'cloth object (towel, napkin, carpet, rag) , cp. <mat-cloth>', 
                'examples': [], 
                'category': 'Thing prototypes'},
     '<cc-stone>': {'umbrella': False, 
                'description': '(= cc-round) stones and stone-sized round objects (pedra, itá, amonite, tijolo)', 
                'examples': [], 
                'category': 'Thing prototypes'},
     '<cc-stick>': {'umbrella': False, 
                'description': 'stick object (long and thin, vara, lançe, paulito)', 
                'examples': [], 
                'category': 'Thing prototypes'},
     '<furn>': {'umbrella': False, 
                'description': 'furniture (cama, cadeira, tambo, quadro)', 
                'examples': [], 
                'category': 'Thing prototypes'},
     '<amount>': {'umbrella': False, 
                'description': 'quantity noun (bocada, teor, sem-fim)', 
                'examples': [], 
                'category': 'Thing prototypes'},
     '<bar>': {'umbrella': False, 
            'description': 'barrier noun (dique, limite, muralha)', 
            'examples': [], 
            'category': 'Thing prototypes'},
     '<build>': {'umbrella': False, 
                'description': 'building (casa, citadela, garagem)', 
                'examples': [], 
                'category': 'Thing prototypes'},
     '<cm>': {'umbrella': True, 
            'description': 'concrete mass/non-countable, umbrella tag, substance (cf. <mat>, terra, choça, magma)', 
            'examples': [], 
            'category': 'Substance prototypes'},
     '<cm-h>': {'umbrella': False, 
                'description': 'human-made substance (cf. <mat>, cemento)', 
                'examples': [], 
                'category': 'Substance prototypes'},
     '<cm-chem>': {'umbrella': False, 
                'description': 'chemical substance, also biological (acetileno, amônio, anilina, bilirrubina', 
                'examples': [], 
                'category': 'Substance prototypes'},
     '<cm-gas>': {'umbrella': False, 
                'description': 'gas substance (so far few: argônio, overlap with. <cm-chem> and <cm>)', 
                'examples': [], 
                'category': 'Substance prototypes'},
     '<cm-liq>': {'umbrella': False, 
                'description': 'liquid substance (azeite, gasolina, plasma, overlap with <food> and <cm-rem>)', 
                'examples': [], 
                'category': 'Substance prototypes'},
     '<cm-rem>': {'umbrella': False, 
                'description': 'remedy (medical or hygiene, antibiótico, canabis, quinina, part of <cm-h>, overlap with <cm-chem>)', 
                'examples': [], 
                'category': 'Substance prototypes'},
     '<cloA>': {'umbrella': False, 
                'description': 'animal clothing (sela, xabraque)', 
                'examples': [], 
                'category': 'Clothing prototypes'},
     '<cloH>': {'umbrella': False, 
                'description': 'human clothing (albornoz, anoraque, babadouro, bermudas)', 
                'examples': [], 
                'category': 'Clothing prototypes'},
     '<cloH-beauty>': {'umbrella': False, 
                'description': 'beauty clothing (e.g. jewelry, diadema, pendente, pulseira)', 
                'examples': [], 
                'category': 'Clothing prototypes'},
     '<cloH-hat>': {'umbrella': False, 
                    'description': 'hat (sombrero, mitra, coroa)', 
                    'examples': [], 
                    'category': 'Clothing prototypes'},
     '<cloH-shoe>': {'umbrella': False, 
                    'description': 'shoe (bota, chinela, patim)', 
                    'examples': [], 
                    'category': 'Clothing prototypes'},
     '<coll>': {'umbrella': False, 
                'description': 'set, collective (random or systematic collection/compound/multitude of similar but distinct small parts, conjunto, série)', 
                'examples': [], 
                'category': 'Collective prototypes'},
     '<coll-cc>': {'umbrella': False, 
                'description': 'thing collective, pile (baralho, lanço)', 
                'examples': [], 
                'category': 'Collective prototypes'},
     '<coll-B>': {'umbrella': False, 
                'description': 'plant-part collective (buquê, folhagem)', 
                'examples': [], 
                'category': 'Collective prototypes'},
     '<coll-sem>': {'umbrella': False, 
                'description': 'semantic collective, collection (arquivo, repertório)', 
                'examples': [], 
                'category': 'Collective prototypes'},
     '<coll-tool>': {'umbrella': False, 
                    'description': 'tool collective, set (intrumentário, prataria)', 
                    'examples': [], 
                    'category': 'Collective prototypes'},
     '<col>': {'umbrella': False, 
            'description': 'color adjective', 
            'examples': [], 
            'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<con>': {'umbrella': False, 
            'description': 'container (implies <num+> quantifying, ampola, chícara, aquário)', 
            'examples': [], 
            'category': 'Collective prototypes'},
     '<conv>': {'umbrella': False, 
                'description': 'convention (social rule or law, lei, preceito)', 
                'examples': [], 
                'category': 'Collective prototypes'},
     '<cord>': {'umbrella': False, 
                'description': 'cord, string, rope, tape (previously <tool-tie>, arame, fio, fibrila)', 
                'examples': [], 
                'category': 'Collective prototypes'},
     '<cur>': {'umbrella': False, 
            'description': 'currency noun (countable, implies <unit>, cf. <mon>, dirham, euro, real, dólar)', 
            'examples': [], 
            'category': 'Collective prototypes'},
     '<dance>': {'umbrella': False, 
                'description': 'dance (both <activity>, <genre> and <sem-l>, calipso, flamenco, forró)', 
                'examples': [], 
                'category': 'Collective prototypes'},
     '<dir>': {'umbrella': False, 
            'description': 'direction noun (estibordo, contrasenso, norte)', 
            'examples': [], 
            'category': 'Collective prototypes'},
     '<domain>': {'umbrella': False, 
                'description': 'domain (subject matter, profession, cf. <genre>, anatomia, citricultura, dactilografia)', 
                'examples': [], 
                'category': 'Collective prototypes'},
     '<dur>': {'umbrella': False, 
            'description': 'duration noun (test: durar+, implies <unit>, e.g. átimo, mês, hora)', 
            'examples': [], 
            'category': 'Time and event prototypes'},
     '<per>': {'umbrella': False, 
            'description': 'period of time (prototypical test: durante, e.g. guerra, década, cf. <dur> and <temp>)', 
            'examples': [], 
            'category': 'Concept prototypes'},
     '<event>': {'umbrella': False, 
                'description': '(EVENTO) non-organised event', 
                'examples': [], 
                'category': 'Event categories <occ>, HAREM ACONTECIMENTO (HAREM tags in parenthesis)'},
     '<occ>': {'umbrella': False, 
            'description': '(ORGANIZADO) organised event', 
            'examples': [], 
            'category': 'Event categories <occ>, HAREM ACONTECIMENTO (HAREM tags in parenthesis)'},
     '<f>': {'umbrella': True, 
            'description': 'feature/property, umbrella tag (problematicidade, proporcionalidade)', 
            'examples': [], 
            'category': 'Feature prototypes'},
     '<f-an>': {'umbrella': False, 
                'description': 'anatomical "local" feature, includes countables, e.g. barbela, olheiras)', 
                'examples': [], 
                'category': 'Feature prototypes'},
     '<f-c>': {'umbrella': False, 
            'description': 'general countable feature (vestígio, laivos, vinco)', 
            'examples': [], 
            'category': 'Feature prototypes'},
     '<f-h>': {'umbrella': False, 
            'description': 'human physical feature, not countable (lindura, compleição, same as <f-phys-h>, cp. anatomical local features <f-an>)', 
            'examples': [], 
            'category': 'Feature prototypes'},
     '<f-psych>': {'umbrella': False, 
                'description': 'human psychological feature (passionalidade, pavonice, cp. passing states <state-h>)', 
                'examples': [], 
                'category': 'Feature prototypes'},
     '<f-q>': {'umbrella': False, 
            'description': "quantifiable feature (e.g. circunferência, calor, DanGram's <f-phys> covers both <f> and <f-q>)", 
            'examples': [], 
            'category': 'Feature prototypes'},
     '<f-right>': {'umbrella': False, 
                'description': 'human social feature (right or duty): e.g. copyright, privilégio, imperativo legal)', 
                'examples': [], 
                'category': 'Feature prototypes'},
     '<food>': {'umbrella': False, 
                'description': 'natural/simplex food (aveia, açúcar, carne, so far including <spice>)', 
                'examples': [], 
                'category': 'Food prototypes'},
     '<food-c>': {'umbrella': False, 
                'description': 'countable food (few: ovo, dente de alho, most are <fruit> or <food-c-h>)', 
                'examples': [], 
                'category': 'Food prototypes'},
     '<food-h>': {'umbrella': False, 
                'description': 'human-prepared/complex culinary food (caldo verde, lasanha)', 
                'examples': [], 
                'category': 'Food prototypes'},
     '<food-c-h>': {'umbrella': False, 
                'description': 'culinary countable food (biscoito, enchido, panetone, pastel)', 
                'examples': [], 
                'category': 'Food prototypes'},
     '<drink>': {'umbrella': False, 
                'description': 'drink (cachaça, leite, guaraná, moca)', 
                'examples': [], 
                'category': 'Food prototypes'},
     '<game>': {'umbrella': False, 
                'description': 'play, game (bilhar, ioiô, poker, also <activity>)', 
                'examples': [], 
                'category': 'Concept prototypes'},
     '<genre>': {'umbrella': False, 
                'description': '(DISCIPLINA) subject matter', 
                'examples': [], 
                'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<geom>': {'umbrella': False, 
                'description': 'geometry noun (circle, shape, e.g. losango, octógono, elipse)', 
                'examples': [], 
                'category': 'Concept prototypes'},
     '<geom-line>': {'umbrella': False, 
                    'description': 'line (few: linha, percentil, curvas isobáricas)', 
                    'examples': [], 
                    'category': 'Concept prototypes'},
     '<inst>': {'umbrella': False, 
                'description': '(EMPRESA) organized site (e.g. restaurant, cp. <Linst>)', 
                'examples': [], 
                'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<ism>': {'umbrella': False, 
                'description': 'ideology or other value system (anarquismo, anti-ocidentalismo, apartheid)', 
                'examples': [], 
                'category': 'Concept prototypes'},
     '<ling>': {'umbrella': False, 
                'description': 'language (alemão, catalão, bengali)', 
                'examples': [], 
                'category': 'Concept prototypes'},
     '<mach>': {'umbrella': False, 
                'description': 'machine (complex, usually with moving parts, betoneira, embrulhador, limpa-pratos, cp. <tool>)', 
                'examples': [], 
                'category': 'Concept prototypes'},
     '<mat>': {'umbrella': False, 
                'description': '(SUBSTANCIA) substance', 
                'examples': [], 
                'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'},
     '<mat-cloth>': {'umbrella': False, 
                    'description': 'cloth material (seda, couro, vison, kevlar), cp. <cc-rag>', 
                    'examples': [], 
                    'category': 'Concept prototypes'},
     '<meta>': {'umbrella': False, 
                'description': 'meta noun (tipo, espécie)', 
                'examples': [], 
                'category': 'Concept prototypes'},
     '<mon>': {'umbrella': False, 
                'description': 'amount of money (bolsa, custo, imposto, cf. <cur>)', 
                'examples': [], 
                'category': 'Concept prototypes'},
     '<month>': {'umbrella': False, 
                'description': 'month noun/name (agosto, julho, part of <temp>)', 
                'examples': [], 
                'category': 'Concept prototypes'},
     '<part>': {'umbrella': False, 
                'description': 'distinctive or functional part (ingrediente, parte, trecho)', 
                'examples': [], 
                'category': 'Part prototypes'},
     '<part-build>': {'umbrella': False, 
                    'description': 'structural part of building or vehicle (balustrada, porta, estai)', 
                    'examples': [], 
                    'category': 'Part prototypes'},
     '<piece>': {'umbrella': False, 
                'description': 'indistinctive (little) piece (pedaço, raspa)', 
                'examples': [], 
                'category': 'Part prototypes'},
     '<percep-f>': {'umbrella': False, 
                    'description': 'what you feel (senses or sentiment, pain, e.g. arrepio, aversão, desagrado, cócegas, some overlap with <state-h>)', 
                    'examples': [], 
                    'category': 'Perception prototypes'},
     '<percep-l>': {'umbrella': False, 
                    'description': 'sound (what you hear, apitadela, barrulho, berro, crepitação)', 
                    'examples': [], 
                    'category': 'Perception prototypes'},
     '<percep-o>': {'umbrella': False, 
                    'description': 'olfactory impression (what you smell, bafo, chamuscom fragrância)', 
                    'examples': [], 
                    'category': 'Perception prototypes'},
     '<percep-t>': {'umbrella': False, 
                    'description': 'what you taste (PALAVRAS: not implemented)', 
                    'examples': [], 
                    'category': 'Perception prototypes'},
     '<percep-w>': {'umbrella': False, 
                    'description': 'visual impression (what you see, arco-iris, réstia, vislumbre)', 
                    'examples': [], 
                    'category': 'Perception prototypes'},
     '<pict>': {'umbrella': False, 
                'description': 'picture (combination of <cc>, <sem-w> and <L>, caricatura, cintilograma, diapositivo)', 
                'examples': [], 
                'category': 'Perception prototypes'},
     '<pos-an>': {'umbrella': False, 
                'description': 'anatomical/body position (few: desaprumo)', 
                'examples': [], 
                'category': 'Perception prototypes'},
     '<pos-soc>': {'umbrella': False, 
                'description': 'social position, job (emprego, condado, capitania, presidência)', 
                'examples': [], 
                'category': 'Perception prototypes'},
     '<process>': {'umbrella': False, 
                'description': 'process (-CONTROL, -PERFECTIVE, cp. <event>, balcanização, convecção, estagnação)', 
                'examples': [], 
                'category': 'Perception prototypes'},
     '<sem>': {'umbrella': True, 
            'description': 'semiotic artifact, work of art, umbrella tag (all specified in PALAVRAS)', 
            'examples': [], 
            'category': 'Semantic product prototypes'},
     '<sem-c>': {'umbrella': False, 
                'description': 'cognition product (concept, plan, system, conjetura, esquema, plano, prejuízo)', 
                'examples': [], 
                'category': 'Semantic product prototypes'},
     '<sem-l>': {'umbrella': False, 
                'description': 'listen-work (music, cantarola, prelúdio, at the same time <genre>: bossa nova)', 
                'examples': [], 
                'category': 'Semantic product prototypes'},
     '<sem-nons>': {'umbrella': False, 
                    'description': 'nonsense, rubbish (implies <sem-s>, galimatias, farelório)', 
                    'examples': [], 
                    'category': 'Semantic product prototypes'},
     '<sem-r>': {'umbrella': False, 
                'description': 'read-work (biografia, dissertação, e-mail, ficha cadastral)', 
                'examples': [], 
                'category': 'Semantic product prototypes'},
     '<sem-s>': {'umbrella': False, 
                'description': 'speak-work (palestra, piada, exposto)', 
                'examples': [], 
                'category': 'Semantic product prototypes'},
     '<sem-w>': {'umbrella': False, 
                'description': 'watch-work (filme, esquete, mininovela)', 
                'examples': [], 
                'category': 'Semantic product prototypes'},
     '<sick>': {'umbrella': False, 
                'description': 'disease (acne, AIDS, sida, alcoolismo, cp. <Hsick>)', 
                'examples': [], 
                'category': 'Semantic product prototypes'},
     '<sick-c>': {'umbrella': False, 
                'description': 'countable disease-object (abscesso, berruga, cicatriz, gangrena)', 
                'examples': [], 
                'category': 'Semantic product prototypes'},
     '<sit>': {'umbrella': False, 
            'description': 'psychological situation or physical state of affairs (reclusão, arruaça, ilegalidade, more complex and more "locative" than <state> and <state-h>', 
            'examples': [], 
            'category': 'State-of-affairs prototypes'},
     '<state>': {'umbrella': False, 
                'description': 'state (of something, otherwise <sit>), abundância, calma, baixa-mar, equilíbrio', 
                'examples': [], 
                'category': 'State-of-affairs prototypes'},
     '<state-h>': {'umbrella': False, 
                'description': 'human state (desamparo, desesperança, dormência, euforia, febre, cp. <f-psych> and <f-phys-h>, which cover innate features)', 
                'examples': [], 
                'category': 'State-of-affairs prototypes'},
     '<sport>': {'umbrella': False, 
                'description': 'sport (capoeira, futebol, golfe, also <activity> and <domain>)', 
                'examples': [], 
                'category': 'State-of-affairs prototypes'},
     '<talk>': {'umbrella': False, 
                'description': 'speech situation, talk, discussion, quarrel (implies <activity> and <sd>, entrevista, lero-lero)', 
                'examples': [], 
                'category': 'State-of-affairs prototypes'},
     '<temp>': {'umbrella': False, 
                'description': 'temporal object, point in time (amanhecer, novilúnio, test: até+, cf. <dur> and <per>)', 
                'examples': [], 
                'category': 'State-of-affairs prototypes'},
     '<therapy>': {'umbrella': False, 
                'description': 'therapy (also <domain> and <activity>, acupuntura, balneoterapia)', 
                'examples': [], 
                'category': 'State-of-affairs prototypes'},
     '<tool>': {'umbrella': True, 
                'description': 'tool, umbrella tag (abana-moscas, lápis, computador, maceta, "handable", cf. <mach>)', 
                'examples': [], 
                'category': 'Tool prototypes'},
     '<tool-cut>': {'umbrella': False, 
                'description': 'cutting tool, knife (canivete, espada)', 
                'examples': [], 
                'category': 'Tool prototypes'},
     '<tool-gun>': {'umbrella': False, 
                    'description': 'shooting tool, gun (carabina, metralhadora, helicanão, in Dangram: <tool-shoot>)', 
                    'examples': [], 
                    'category': 'Tool prototypes'},
     '<tool-mus>': {'umbrella': False, 
                    'description': 'musical instrument (clavicórdio, ocarina, violão)', 
                    'examples': [], 
                    'category': 'Tool prototypes'},
     '<tool-sail>': {'umbrella': False, 
                    'description': 'sailing tool, sail (vela latina, joanete, coringa)', 
                    'examples': [], 
                    'category': 'Tool prototypes'},
     '<tube>': {'umbrella': False, 
                'description': 'tube object (cânula, gasoduto, zarabatana, shape-category, typically with another category, like <an> or <tool>)', 
                'examples': [], 
                'category': 'Tool prototypes'},
     '<unit>': {'umbrella': False, 
                'description': 'unit noun (always implying <num+>, implied by <cur> and <dur>, e.g. caloria, centímetro, lúmen))', 
                'examples': [], 
                'category': 'Tool prototypes'},
     '<wea>': {'umbrella': True, 
            'description': 'weather (states), umbrella tag (friagem, bruma)', 
            'examples': [], 
            'category': 'Weather prototypes'},
     '<wea-c>': {'umbrella': False, 
                'description': 'countable weather phenomenon (nuvem, tsunami)', 
                'examples': [], 
                'category': 'Weather prototypes'},
     '<wea-rain>': {'umbrella': False, 
                    'description': "rain and other precipitation (chuvisco, tromba d'água, granizo)", 
                    'examples': [], 
                    'category': 'Weather prototypes'},
     '<wea-wind>': {'umbrella': False, 
                    'description': 'wind, storm (brisa, furacão)', 
                    'examples': [], 
                    'category': 'Weather prototypes'},
     '<hum>': {'umbrella': False, 
            'description': '(INDIVIDUAL) person name (cp. <H>)', 
            'examples': [], 
            'category': 'Person categories <hum>, HAREM PESSOA (HAREM tags in parenthesis)'},
     '<official>': {'umbrella': False, 
                    'description': '(CARGO) official function (~ cp. <Htitle> and <Hprof>)', 
                    'examples': [], 
                    'category': 'Person categories <hum>, HAREM PESSOA (HAREM tags in parenthesis)'},
     '<member>': {'umbrella': False, 
                'description': '(MEMBRO) member', 
                'examples': [], 
                'category': 'Person categories <hum>, HAREM PESSOA (HAREM tags in parenthesis)'},
     '<admin>': {'umbrella': False, 
                'description': '(ADMINISTRACAO, ORG.) administrative body (government, town administration etc.)', 
                'examples': [], 
                'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<org>': {'umbrella': False, 
            'description': '(INSTITUICAO/EMPRESA) commercial or non-commercial, non-administrative non-party organisations (not place-bound, therefore not the same as <Linst>)', 
            'examples': [], 
            'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<media>': {'umbrella': False, 
                'description': '(EMPRESA) media organisation (e.g. newspaper, tv channel)', 
                'examples': [], 
                'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<party>': {'umbrella': False, 
                'description': '(INSTITUICAO) political party', 
                'examples': [], 
                'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<suborg>': {'umbrella': False, 
                'description': '(SUB) organized part of any of the above', 
                'examples': [], 
                'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<company>': {'umbrella': False, 
                'description': '(EMPRESA) company (not site-bound, unlike <inst>, now fused with. <org>) - currently unsupported', 
                'examples': [], 
                'category': 'Organisation/Group categories <org>, HAREM ORGANIZACAO (HAREM tags in parenthesis)'},
     '<groupind>': {'umbrella': False, 
                    'description': '(GROUPOIND) people, family', 
                    'examples': [], 
                    'category': 'Group categories, HAREM PESSOA (HAREM tags in parenthesis)'},
     '<groupofficial>': {'umbrella': False, 
                        'description': '(GROUPOCARGO) board, government (not fully implemented)', 
                        'examples': [], 
                        'category': 'Group categories, HAREM PESSOA (HAREM tags in parenthesis)'},
     '<grouporg>': {'umbrella': False, 
                    'description': '(GROUPOMEMBRO) club, e.g. football club (now fused with <org>) - currently unsupported', 
                    'examples': [], 
                    'category': 'Group categories, HAREM PESSOA (HAREM tags in parenthesis)'},
     '<top>': {'umbrella': False, 
            'description': '(GEOGRAFICO) geographical location (cp. <Ltop>)', 
            'examples': [], 
            'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<civ>': {'umbrella': False, 
            'description': '(ADMINISTRACAO, LOC.) civitas (country, town, state, cp. <Lciv>)', 
            'examples': [], 
            'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<address>': {'umbrella': False, 
                'description': '(CORREIO) address (including numbers etc.)', 
                'examples': [], 
                'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<site>': {'umbrella': False, 
                'description': '(ALARGADO) functional place (cp. <Lh>)', 
                'examples': [], 
                'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<virtual>': {'umbrella': False, 
                'description': '(VIRTUAL) virtual place', 
                'examples': [], 
                'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<astro>': {'umbrella': False, 
                'description': '(OBJECTO) astronomical place (in HAREM object, not place)', 
                'examples': [], 
                'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<road>': {'umbrella': False, 
                'description': '(ALARGADO) roads, motorway (unlike <address>) - suggested', 
                'examples': [], 
                'category': 'Place categories <top>, HAREM LOCAL (HAREM tags in parenthesis)'},
     '<history>': {'umbrella': False, 
                'description': '(EFEMERIDE) one-time [historical] occurrence', 
                'examples': [], 
                'category': 'Event categories <occ>, HAREM ACONTECIMENTO (HAREM tags in parenthesis)'},
     '<tit>': {'umbrella': False, 
            'description': '(REPRODUZIDO) [title of] reproduced work, copy', 
            'examples': [], 
            'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<pub>': {'umbrella': False, 
            'description': '(PUBLICACAO) [scientific] publication', 
            'examples': [], 
            'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<product>': {'umbrella': False, 
                'description': '(PRODUTO) product brand', 
                'examples': [], 
                'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<artwork>': {'umbrella': False, 
                'description': '(ARTE) work of art', 
                'examples': [], 
                'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     'Abstract': {'umbrella': False, 
                'description': 'categories <brand>, HAREM ABSTRACCAO (HAREM tags in parenthesis): ', 
                'examples': [], 
                'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<brand>': {'umbrella': False, 
                'description': '(MARCA) brand', 
                'examples': [], 
                'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<school>': {'umbrella': False, 
                'description': '(ESCOLA) school of thought', 
                'examples': [], 
                'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<idea>': {'umbrella': False, 
                'description': '(IDEA) idea, concept', 
                'examples': [], 
                'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<plan>': {'umbrella': False, 
                'description': '(PLANO) named plan, project', 
                'examples': [], 
                'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<author>': {'umbrella': False, 
                'description': "(OBRA) artist's name, standing for body of work", 
                'examples': [], 
                'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<absname>': {'umbrella': False, 
                'description': '(NOME)', 
                'examples': [], 
                'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<disease>': {'umbrella': False, 
                'description': '(ESTADO) physiological state, in particular: disease', 
                'examples': [], 
                'category': 'Work of art/product categories <tit>, HAREM OBRA (HAREM tags in parenthesis)'},
     '<object>': {'umbrella': False, 
                'description': '(OBJECT) named object', 
                'examples': [], 
                'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'},
     '<common>': {'umbrella': False, 
                'description': '(OBJECT) common noun used as name', 
                'examples': [], 
                'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'},
     '<class>': {'umbrella': False, 
                'description': '(CLASSE) classification category for things', 
                'examples': [], 
                'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'},
     '<plant>': {'umbrella': False, 
                'description': '(CLASSE) plant name', 
                'examples': [], 
                'category': 'Thing categories <common>, HAREM COISA (HAREM tags in parenthesis)'},
     '<currency>': {'umbrella': False, 
                    'description': '(MOEDA) currency name (also marked on the unit)', 
                    'examples': [], 
                    'category': "Numeric value categories (marked only on the numeral, without MWE'ing, unlike HAREM VALOR)"},
     '<date>': {'umbrella': False, 
                'description': '(DATA) date', 
                'examples': [], 
                'category': "Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO)"},
     '<hour>': {'umbrella': False, 
            'description': '(HORA) hour', 
            'examples': [], 
            'category': "Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO)"},
     '<period>': {'umbrella': False, 
            'description': '(PERIODO) period', 
            'examples': [], 
            'category': "Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO)"},
     '<cyclic>': {'umbrella': False, 
            'description': '(CICLICO) cyclic time expression', 
            'examples': [], 
            'category': "Time categories (if used for NUM or N rather than PROP, marked only on the numeral or noun, without MWE'ing, unlike HAREM TEMPO)"},
     '<quantity>': {'umbrella': False, 
                    'description': '(QUANTIDADE) simple measuring numeral', 
                    'examples': [], 
                    'category': "Numeric value categories (marked only on the numeral, without MWE'ing, unlike HAREM VALOR)"},
     '<prednum>': {'umbrella': False, 
                'description': '(CLASSIFICADO) predicating numeral', 
                'examples': [], 
                'category': "Numeric value categories (marked only on the numeral, without MWE'ing, unlike HAREM VALOR)"},
     '<mass>': {'umbrella': False, 
                'description': 'mass noun (e.g. "leite", "água")', 
                'examples': [], 
                'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<jh>': {'umbrella': False, 
            'description': 'adjective modifying human noun', 
            'examples': [], 
            'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<jn>': {'umbrella': False, 
            'description': 'adjective modifying inanimate noun', 
            'examples': [], 
            'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<ja>': {'umbrella': False, 
            'description': 'adjective modifying animal', 
            'examples': [], 
            'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<jb>': {'umbrella': False, 
            'description': 'adjective modifying plant', 
            'examples': [], 
            'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<nat>': {'umbrella': False, 
            'description': 'nationality adjective (also: from a certain town etc.)', 
            'examples': [], 
            'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<attr>': {'umbrella': False, 
                'description': '(human) attributive adjective (not fully implemented, cp. <Hattr>, e.g. "um presidente COMUNISTA")', 
                'examples': [], 
                'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<vH>': {'umbrella': False, 
            'description': 'verb with human subject', 
            'examples': [], 
            'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'},
     '<vN>': {'umbrella': False, 
            'description': 'verb with inanimate subject', 
            'examples': [], 
            'category': 'OTHER SEMANTICALLY INSPIRED CATEGORIES'}
}

In [348]:
SEMANTIC_TAGSET = {
    'H': {'umbrella': True, 
          'description': "+HUM-noun, human being -> cf. more specific categories below",
          'examples': ["achacador", "'kidnapper'","acionário", "'shareholder'", "inimigo","'enemy'"],
          'category': "ANIMATE HUMAN"},
    'HM': {'umbrella': False, 
           'description': "mystical or religious entity, constellations in anjo ‘angel’, duende ‘goblin’, astrology",
           'examples': ["anjo 'angel'", "duende 'goblin'","hidra 'Hydra'", "tauro 'Taurus'"],
           'category': "ANIMATE HUMAN"},
    'N': {'umbrella': False, 
          'description': "",
          'examples': ["cigano 'gypsy'", "escandinavo 'Scandinavian'"],
          'category': "ANIMATE HUMAN"},        
    'prof': {'umbrella': False, 
          'description': "professional",
          'examples': ["-or: escritor 'author'", "filósofo 'philosopher'"],
          'category': "ANIMATE HUMAN"},
    'fam': {'umbrella': False, 
          'description': "family member",
          'examples': ["pai 'father'", "mãe 'mother'"],
          'category': "ANIMATE HUMAN"},
    'title': {'umbrella': False, 
          'description': "+HUM, often governing name (a) regular title (b) others, apart from <prof> or <fam>",
          'examples': ["(a) rei \'king\', presidente \'president\', senhor \'mister\'", "(b) moça \'girl\', colega \'collegue\'"],
          'category': "ANIMATE HUMAN"},
    'n+': {'umbrella': False, 
          'description': "-HUM, potencially governing name",
          'examples': ["restaurante \'restaurant\'", "plano \'plan\'", "rua \'street\'"],
          'category': "ANIMATE HUMAN"},
    'attr': {'umbrella': False, 
          'description': "attributive +HUM noun, often used as @N<",
          'examples': ["comunista /'communist/'"],
          'category': "ANIMATE HUMAN"},
    'HH': {'umbrella': False, 
          'description': "group of H", 
           'examples': ["companhia \'company\'", "equipe \'team\'"],
          'category': "ANIMATE HUMAN"},
    'parti': {'umbrella': False, 
          'description': "(political) party",
          'examples': ["PT \'Labour\'"],
          'category': "ANIMATE HUMAN"}, 
    'inst': {'umbrella': False, 
          'description': "institution [also topological]",
          'examples': ["igreja /'the church/'", "polícia /'the police/'", "-ria: padaria \'bakery\'"],
          'category': "ANIMATE HUMAN"},
    'h': {'umbrella': False, 
          'description': "+HUM-adjective",
          'examples': ["jubiloso \'jubilant\'", "louco \'crazy\'"],
          'category': "ANIMATE HUMAN"},
    'n': {'umbrella': False, 
          'description': "nationalaty-adective",
          'examples': ["dinamarquês /'Danish/'"],
          'category': "ANIMATE HUMAN"},                   
    'A': {'umbrella': True, 
          'description': "+ANIM,-HUM",
          'examples': [],
          'category': "ANIMATE NON-HUMAN, MOVING"},
    'AM': {'umbrella': False, 
          'description': "mythological animal",
          'examples': ["pégaso \'Pegasus\'", "licorne \'unicorn\'"],
          'category': "ANIMATE NON-HUMAN, MOVING"},                
    'AB': {'umbrella': False, 
          'description': "bacteria, cells",
          'examples': ["macrófago \'macrophage\'"],
          'category': "ANIMATE NON-HUMAN, MOVING"},                   
    'zo': {'umbrella': False, 
          'description': "animal, including mammals",
          'examples': ["aligátor \'alligator\'", "tênia \'tape-worm\'", "babuíno \'baboon\'"],
          'category': "ANIMATE NON-HUMAN, MOVING"},
    'D': {'umbrella': False, 
          'description': "mammal, especially domestic",
          'examples': ["cavalo \'horse\'", "vaca \'cow\'"],
          'category': "ANIMATE NON-HUMAN, MOVING"},                   
    'orn': {'umbrella': False, 
          'description': "bird",
          'examples': ["bem-te-vi \'Pitangus-bird\'", "canário \'canary\'"],
          'category': "ANIMATE NON-HUMAN, MOVING"},
    'ent': {'umbrella': False, 
          'description': "insect",
          'examples': ["saúva \'ant\'", "formiga \'ant\'"],
          'category': "ANIMATE NON-HUMAN, MOVING"},
    'ich': {'umbrella': False, 
          'description': "fish",
          'examples': ["perca \'perch\'", "lobo-marinho \'sea-lion\'"],
          'category': "ANIMATE NON-HUMAN, MOVING"},
    'AA': {'umbrella': False, 
          'description': "group of animals (experimental: 4 AAorn, 5 DD)",
          'examples': ["manada \'herd\'", "matilha \'pack\'", "cria \'brood\'", "vacada \'cow-herd\'"],
          'category': "ANIMATE NON-HUMAN, MOVING"},
    'bo': {'umbrella': False, 
          'description': "plant (general)",
          'examples': ["madressilva /'honeysuckle/'"],
          'category': "ANIMATE NON-HUMAN, NON-MOVING"},
    'B': {'umbrella': False, 
          'description': "tree or bush (under, near, in)",
          'examples': ["tree or bush (under, near, in)"],
          'category': "ANIMATE NON-HUMAN, NON-MOVING"},                       
    'BB': {'umbrella': False, 
          'description': "group of plants (cp. topologica), place as defined by what grows on it",
          'examples': ["faial \'beech forest\'", "floresta \'forest\'", "roça \'field\'"],
          'category': "ANIMATE NON-HUMAN, NON-MOVING"},                       
    'top': {'umbrella': False, 
          'description': "toponym or natural topological",
          'examples': ["Brasília, monte /'mountain/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},                   
    'agua': {'umbrella': False, 
          'description': "body of water (where one swims)",
          'examples': ["rio \'river\'", "mar \'sea\'", "laguna \'lagoon\'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"}, 
    'sky': {'umbrella': False, 
          'description': "sky, space, air space (where one flies)",
          'examples': ["céu \'sky\'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"}, 
    'vej': {'umbrella': False, 
          'description': "path, road (where one walks or drives)",
          'examples': ["rua \'street\'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},                   
    'topabs': {'umbrella': False, 
          'description': "abstract topological",
          'examples': ["fim /'end/'", "curvatura /'curvature/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},                
    'spids': {'umbrella': False, 
          'description': "point, tip",
          'examples': ["farpa /'barb/'", "pico /'sharp point, peak/'", "cume /'peak/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},                   
    'area': {'umbrella': False, 
          'description': "area, region",
          'examples': ["área /'area/'", "terreno /'area/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},                    
    'hul': {'umbrella': False, 
          'description': "hole or cavity, notch or groove",
          'examples': ["poro /'pore/'", "valeta /'gutter/'", "rasgão /'tear, gash/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},                  
    'ejo': {'umbrella': False, 
          'description': "functional place",
          'examples': ["quarto /'room/'", "banheiro /'bathroom/'", "-douro"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},                  
    'hus': {'umbrella': False, 
          'description': "building [also cc]",
          'examples': ["casa /'house/'", "casebre /'hut/'", "torre /'tower/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'by': {'umbrella': False, 
          'description': "group of houses, town, administrative unit country, state",
          'examples': ["vila /'village/'", "cidade /'town/'", "estado /'state/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'inst': {'umbrella': False, 
          'description': "institution [also animate human]",
          'examples': ["igreja /'church/'", "polícia /'police/'", "-ria: padaria /'bakery/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'ta': {'umbrella': False, 
          'description': "arquitectural feature",
          'examples': ["trave /'beam/'", "janela /'window/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},                   
    'tm': {'umbrella': False, 
          'description': "piece of furniture [also cc]",
          'examples': ["cadeira ‘chair’, mesa ‘table’"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},                   
    'fælde': {'umbrella': False, 
          'description': "trap, snare",
          'examples': ["nassa /'wicker basket/'", "ratoeira /'mouse trap/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'kovr': {'umbrella': False, 
          'description': "blanket, carpet, curtain, cover, lid (what things can be under)",
          'examples': ["tampa /'lid/'", "manta^259 /'blanket/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'ujo': {'umbrella': False, 
          'description': "container",
          'examples': ["copo /'cup/'", "garrafa /'bottle/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'rør': {'umbrella': False, 
          'description': "tube",
          'examples': ["tubo /'tube/'", "oleoduto /'pipeline/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'bild': {'umbrella': False, 
          'description': "picture",
          'examples': ["pintura /'picture/'", "grafiti /'graffiti/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'r': {'umbrella': False, 
          'description': "things you can read and touch. All r (touchable readables) are also rr (readables), but not vice versa, as with poema \'poem\'.",
          'examples': ["livro /'book/'", "jornal /'newspaper/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'bar': {'umbrella': False, 
          'description': "fence or hedge, dike, dam (s.th. you pass over)",
          'examples': ["fronteira /'border/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'dir': {'umbrella': False, 
          'description': "direction",
          'examples': ["lés-sueste \'eastsoutheast\'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},                   
     'stil': {'umbrella': False, 
          'description': "position (you hold)",
          'examples': ["presidência /'presidency/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},                  
    'sit': {'umbrella': False, 
          'description': "situation, state of affairs (in+)",
          'examples': ["caos /'chaos/'", "circunstâncias /'circumstances/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},                   
      'vejr': {'umbrella': False, 
          'description': "weather",
          'examples': ["gelada /'hoar frost/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
     'vind': {'umbrella': False, 
          'description': "wind",
          'examples': ["furacão /'hurricane/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
     'regn': {'umbrella': False, 
          'description': "rain",
          'examples': ["chuvisco /'drizzle/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
     'an': {'umbrella': True, 
          'description': "anatomical site (HUM)",
          'examples': ["dorso /'back/'", "entranhas /'guts/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
     'anmov': {'umbrella': False, 
          'description': "movable anatomical part-> +PL (hand, finger, head)",
          'examples': ["sobrancelha /'eyebrow/'", "tentáculo /'tentacle/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
     'anorg': {'umbrella': False, 
          'description': "anatomical organ-> +PL (in+)",
          'examples': ["coração /'heart/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},               
     'anost': {'umbrella': False, 
          'description': "bone",
          'examples': ["fêmur /'thighbone/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
      'anfeat': {'umbrella': False, 
          'description': "(uncountable) anatomical \"wearable\" (grimace, tan, hair)-> <anfeatc> (countable anatomical feature=",
          'examples': ["queixo-duplo /'double chin/'","riso=amarelo /'forced smile/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
      'anzo': {'umbrella': False, 
          'description': "animal anatomy",
          'examples': ["colmilho /'tusk/'", "focinha /'snout/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
      'anorn': {'umbrella': False, 
          'description': "bird anatomy",
          'examples': ["pluma /'feather/'", "rostro /'beak/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
      'anich': {'umbrella': False, 
          'description': "",
          'examples': [""],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'anent': {'umbrella': False, 
          'description': "insect anatomy",
          'examples': ["rostro /'proboscis/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
      'anbo': {'umbrella': False, 
          'description': "plant anatomy",
          'examples': ["drupa /'drupe/'", "estame /'stamen/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'star': {'umbrella': False, 
          'description': "star",
          'examples': ["planeta /'planet/'", "Venus /'Venus/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
      'surf': {'umbrella': False, 
          'description': "surface (2-dimensional, on+)",
          'examples': ["chão /'floor, ground/'", "superfície /'surface/'", "face /'front, side/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'DIST': {'umbrella': False, 
          'description': "distance (after <vt+DIST>)",
          'examples': ["légua /'mile/'"],
          'category': "TOPOLOGICALS (mostly, CONCRETA, NON-MOVABLE)"},
    'V': {'umbrella': False, 
          'description': "vehicle",
          'examples': ["carro /'car/'", "bicleta /'bicycle/'"],
          'category': "CONCRETA, MOVING"},
    'skib': {'umbrella': False, 
          'description': "ship",
          'examples': ["navio /'ship/'", "iate /'yacht/'"],
          'category': "CONCRETA, MOVING"},
    'fly': {'umbrella': False, 
          'description': "plane",
          'examples': ["teco-teco /'little plane/'", "pára-quedas /'parachute/'"],
          'category': "CONCRETA, MOVING"},
    'VV': {'umbrella': False, 
          'description': "group of vehicles",
          'examples': ["armada /'fleet/'", "comboio /'convoy/'"],
          'category': "CONCRETA, MOVING"},
    'or': {'umbrella': False, 
          'description': "machine",
          'examples': ["britadeira /'stone crusher/'"],
          'category': "CONCRETA, MOVING"},
    'cm': {'umbrella': False, 
          'description': "physical mass nouns (+CONCRETE, +MASS)",
          'examples': ["adubo /'fertilizer/'", "ar /'air/'", "breu /'tar/'", "espuma /'foam/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'liqu': {'umbrella': False, 
          'description': "liquid",
          'examples': ["petróleo /'oil/'", "saliva /'saliva/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'mat': {'umbrella': False, 
          'description': "material",
          'examples': ["madeira /'tree/'", "silicone /'silicone/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'stof': {'umbrella': False, 
          'description': "fabric",
          'examples': ["seda /'silke/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'mad': {'umbrella': False, 
          'description': "food, unprocessed-> <madc> (piece of unprocessed food)",
          'examples': ["carne /'meat/'", "alho /'garlic/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'kul': {'umbrella': False, 
          'description': "food, processed-> <kulc> (piece of processed food)",
          'examples': ["chocolate /'chocolate/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'drik': {'umbrella': False, 
          'description': "drink",
          'examples': ["chope /'draught beer/'", "leite /'milk/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'rem': {'umbrella': False, 
          'description': "remedy (medicine)",
          'examples': ["morfina /'morphine/'", "penicilina /'penicilline/'", "vitamina /'vitamin/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},                       
    'cc': {'umbrella': False, 
          'description': "concrete objects (+CONCRETE, -MASS)",
          'examples': ["pedra /'stone/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},                       
    'ar': {'umbrella': False, 
          'description': "stack, heap, pile, bundle, row",
          'examples': ["feixa /'bundle/'", "fila /'row/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},                       
    'er': {'umbrella': False, 
          'description': "(countable) piece or part or group member",
          'examples': ["ingrediente /'ingredient/'", "lasca /'chip, splinter/'", "parte /'part/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},                       
    'sten': {'umbrella': False, 
          'description': "stone (you can throw, cf. <mat>)",
          'examples': ["pedra /'stone/'", "rubim /'ruby/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},                       
    'stok': {'umbrella': False, 
          'description': "stick, plank, board",
          'examples': ["vara /'rod/'", "galho /'twig/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},                       
    'star': {'umbrella': False, 
          'description': "star (cf. topologica)",
          'examples': ["estrela /'star/'", "planeta /'planet/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},                       
    'ild': {'umbrella': False, 
          'description': "(1) fire, spark etc. (2) pipe, bonfire (all that can be lighted) -> cf. <lys> (light tools)",
          'examples': ["chama /'flame/'", "chispa /'spark/'", "fogueira /'bonfire/'", "relâmpago /'lightning/'", "cachimbo /'pipe/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},                       
    'vejrc': {'umbrella': False, 
          'description': "countable weather phenomena",
          'examples': ["nuvem /'cloud/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},                       
    'madc': {'umbrella': False, 
          'description': "piece of unprocessed food, eg. fruit",
          'examples': ["ervilha /'pea/'", "cebola /'onion/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'kulc': {'umbrella': False, 
          'description': "piece of processed food, eg. burger",
          'examples': ["pão /'bread/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'il': {'umbrella': False, 
          'description': "tool",
          'examples': ["garfo /'fork/'", "plectro /'plectron/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'kniv': {'umbrella': False, 
          'description': "knife, sword, spear (bundling of features in prototype: sharp, pointed, cutting, tool)",
          'examples': ["faca /'knife/'", "canivete /'pocket knife/'", "enxada /'spade/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'fio': {'umbrella': False, 
          'description': "thread, rope",
          'examples': ["estrém /'anchor cable/'", "fio /'thread/'", "cabo /'cable, rope/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},                       
    'klud': {'umbrella': False, 
          'description': "piece of cloth",
          'examples': ["guardanapo /'napkin/'", "toalha /'håndklæde/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'sejl': {'umbrella': False, 
          'description': "sail",
          'examples': ["bujarrona /'jib, gib/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'paf': {'umbrella': False, 
          'description': "gun",
          'examples': ["canhão /'canon/'", "pistola /'pistol/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'lys': {'umbrella': False, 
          'description': "lamp, torch etc. (all that gives light)-> cf. <ild> (fire-words)",
          'examples': ["lanterna /'lantern/'", "tocha /'torch/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
    'ten': {'umbrella': False, 
          'description': "handle, mouth-piece, hilt",
          'examples': ["maçaneta /'doorknob/'", "hastil /'shaft [of a lance]/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
     'mu': {'umbrella': False, 
          'description': "musical instrument",
          'examples': ["violão /'guitar/'", "flauta /'flute/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},                  
     'tøj': {'umbrella': False, 
          'description': "garment (what you wear)",
          'examples': ["saia /'skirt/'", "camisa /'shirt/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
     'sko': {'umbrella': False, 
          'description': "shoe",
          'examples': ["chinela /'slipper/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
     'hat': {'umbrella': False, 
          'description': "hat",
          'examples': ["coroa /'crown/'", "chapéu /'hat/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
     'smyk': {'umbrella': False, 
          'description': "jewels etc.",
          'examples': ["brinco /'earring/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},
     'tøjzo': {'umbrella': False, 
          'description': "what animals wear",
          'examples': ["brida /'bridle/'", "xairel /'saddle cloth/'"],
          'category': "CONCRETA, NON-MOVING (MOVABLE)"},                       
    'am': {'umbrella': False, 
          'description': "quantifiable feature, abstract mass nouns (-CONCRETE, +MASS)",
          'examples': ["-eza, -idade"],
          'category': "ABSTRACTA"},
    'amh': {'umbrella': False, 
          'description': "+HUM quantifiable feature (ele tem mais ...ade)",
          'examples': [""],
          'category': "ABSTRACTA"},                       
    'ac': {'umbrella': False, 
          'description': "abstract countables (-CONCRETE, -MASS, +COUNT)",
          'examples': ["método /'method/'", "módulo /'module/'", "onda /'wave/'"],
          'category': "ABSTRACTA"},
    'featc': {'umbrella': False, 
          'description': "countable features (mark, spot)",
          'examples': ["cor /'colour/'", "listra /'stripe/'"],
          'category': "ABSTRACTA"},                       
    'anfeatc': {'umbrella': False, 
          'description': "countable anatomical feature",
          'examples': ["verruga /'wart/'", "cabelo /'hair/'"],
          'category': "ABSTRACTA"},
    'sygc': {'umbrella': False, 
          'description': "countable item of disease (boil, scar)",
          'examples': ["terçolho /'sty/'", "abcesso /'abcess/'"],
          'category': "ABSTRACTA"},                       
    'p': {'umbrella': False, 
          'description': "what yout think (thought)",
          'examples': ["idéia /'idea/'", "suspeita /'suspicion/'"],
          'category': "ABSTRACTA"},
    'pp': {'umbrella': False, 
          'description': "plan, concept (product of thought)",
          'examples': ["projeto /'project/'", "estratégia /'strategy/'", "noção /'notion/'"],
          'category': "ABSTRACTA"},                       
    'reg': {'umbrella': False, 
          'description': "rule, law",
          'examples': ["regra /'rule/'"],
          'category': "ABSTRACTA"},
    'right': {'umbrella': False, 
          'description': "rights, habits (one has)(= reg +ADJECTIVAL)",
          'examples': ["direito /'right/'", "prerrogativa /'privilege/'"],
          'category': "ABSTRACTA"},
    'emne': {'umbrella': False, 
          'description': "topic (both ac and ak)",
          'examples': ["assunto /'topic/'"],
          'category': "ABSTRACTA"},
    'l': {'umbrella': False, 
          'description': "what you hear (e.g. natural or artificial sound, noise)",
          'examples': ["aplauso /'applause/'", "berro /'shout/'", "som /'sound/'"],
          'category': "ABSTRACTA"},                       
    'll': {'umbrella': False, 
          'description': "song, piece of music, type of musik (product for hearing-listening)",
          'examples': ["bossa=nova, canto /'song/'", "hino /'hymn/'"],
          'category': "ABSTRACTA"},
    'w': {'umbrella': False, 
          'description': "what you see (e.g. bubble, shadow, a light)",
          'examples': ["vislumbre /'glimpse, glimmer/'", "ilusão /'illusion/'", "arco-iris /'rainbow/'"],
          'category': "ABSTRACTA"},                       
    'ww': {'umbrella': False, 
          'description': "what you watch, e.g. movie, piece of theater (product for watching)",
          'examples': ["filme /'film/'", "sonho /'dream/'", "novela /'tv-series/'", "comédia"],
          'category': "ABSTRACTA"},
    's': {'umbrella': False, 
          'description': "what you say, short utterance (e.g. word, question, answer)",
          'examples': ["pergunta /'question/'", "salamaleque /'salem aleikum/'"],
          'category': "ABSTRACTA"},
    'ss': {'umbrella': False, 
          'description': "speech, joke, lie, rumour, nonsense, gossip, boast (speech product)",
          'examples': ["sermão /'sermon/'", "testemunho /'testemony/'"],
          'category': "ABSTRACTA"},
    'sd': {'umbrella': False, 
          'description': "speech act: tease, mockery, fun (you make of s.b.), insult, request (doing by saying), intrigue, proposal, settlement, appointment, ruling, judgement",
          'examples': ["reza /'prayer/'", "ultimato /'ultimatum/'", "veto /'veto/'", "queixa /'complaint/'"],
          'category': "ABSTRACTA"},                       
    'ret': {'umbrella': False, 
          'description': "rhetoric terms (non-rhetoric, e.g parts of speech like ‘noun’, ‘subject’ -> akc)",
          'examples': ["hipérbole \'hyperbole\'", "coloquialismo \'colloquialism\'"],
          'category': "ABSTRACTA"},
    'o': {'umbrella': False, 
          'description': "what you smell (odour) or taste",
          'examples': ["bodum \'smell of non-castrated goat\'", "bufa \'fart\'"],
          'category': "ABSTRACTA"},   
    'f': {'umbrella': False, 
          'description': "what you feel (whish, burst of pain, - where not am)",
          'examples': ["cócegas \'itching\'", "deleite \'delight\'", "desejo \'wish\'"],
          'category': "ABSTRACTA"},
    'rr': {'umbrella': False, 
          'description': "what you read (unlike r, which is cc: livre, jornal)",
          'examples': ["romance \'novel\'", "dissertação \'dissertation\'"],
          'category': "ABSTRACTA"},
    'tegn': {'umbrella': False, 
          'description': "what you write (sign, character, icon, printed symbol, playing card (ace, king etc.) also: <NUM> (numbers as nouns)",
          'examples': ["vírgula \'comma\'", "dáblio \'w\'", "emblema", "gatafunhos \'‘scribbles’, o cinco ‘number five’"],
          'category': "ABSTRACTA"},
    'geom': {'umbrella': False, 
          'description': "geometric shape (circle, globe, angle) -> +top",
          'examples': ["elipse \'ellipse\'", "heksaedro", "retângulo \'rectangle\'"],
          'category': "ABSTRACTA"},                       
    'line': {'umbrella': False, 
          'description': "line, stripe, streak (now: ac)",
          'examples': ["linha \'line\'", "raio \'ray, radius\'"],
          'category': "ABSTRACTA"},
    'd': {'umbrella': False, 
          'description': "what you do or make: mistake, error (test: dar/fazer, e.g. dar uma lavadela), also ->CP",
          'examples': ["crime \'crime\'", "espiada \'glance\'", "gafe \'blunder\'"],
          'category': "ABSTRACTA"},   
    'num+': {'umbrella': False, 
          'description': "(syntactic, not semantic, therefore preferably in combination with <unit>, <qu>, <qus>)",
          'examples': [""],
          'category': "ABSTRACTA"},
    'unit': {'umbrella': False, 
          'description': "unit (always with -> num+)",
          'examples': ["ano \'year\'", "cruzeiro", "acre \'acre\'"],
          'category': "ABSTRACTA"},
    'qu': {'umbrella': False, 
          'description': "quantity +de 263 (always with -> num+)",
          'examples': ["montes de \'lots of\'", "carradas de \'loads of\'"],
          'category': "ABSTRACTA"},
    'qus': {'umbrella': False, 
          'description': "\"title\"-quantity +de (always with -> num+) [in fact, a Danish MT motivated category, permitting Danish N N sequences] ",
          'examples': ["metro \'meter\'", "litro \'liter\'", "garaffa \'bottle\'"],
          'category': "ABSTRACTA"},
    'mon': {'umbrella': False, 
          'description': "quantity of money",
          'examples': ["achádego \'finder’s reward\'", "bolsa \'scholarship\'"],
          'category': "ABSTRACTA"},
    'akc': {'umbrella': False, 
          'description': "countable category (rhetoric terms rather -> ret)",
          'examples': ["verbo \'verb\'", "substantivo \'noun\'", "numeral \'numeral\'"],
          'category': "ABSTRACTA"},
    'meta': {'umbrella': False, 
          'description': "kind of, type of, group member (semantically transparent like ar, er)",
          'examples': ["tipo \'type\'", "espécie \'kind\'"],
          'category': "ABSTRACTA"},
    'ax': {'umbrella': False, 
          'description': "(-CONCRETE, -MASS, -COUNT, +FEATURE [\"adjectival content\"])",
          'examples': ["antigüidade \'antiquity\'", "neutralidade \'neutrality\'", "simbiose \'symbiosis\'"],
          'category': "ABSTRACTA"},
    'state': {'umbrella': False, 
          'description': "state (s.th. is in, distinct from -> sit - state of affairs)",
          'examples': ["estupor \'stupor\'", "incandescência \'incandescence\'", "sossego \'calm\'"],
          'category': "ABSTRACTA"},                       
    'sh': {'umbrella': False, 
          'description': "human state (e.g. health, ecstasy)",
          'examples': ["apatie \'apathy\'", "cansaço \'fatigue\'"],
          'category': "ABSTRACTA"},
    'feat': {'umbrella': False, 
          'description': "non-quantifiable feature",
          'examples': ["clima '\climate\'", "enormidade \'enormity\'", "textura \'texture\'"],
          'category': "ABSTRACTA"},                       
    'fh': {'umbrella': False, 
          'description': "human non-quantifiable feature, human capacity, skill",
          'examples': ["atitude \'attitude\'", "calvície \'baldness\'", "inocência \'innocence\'"],
          'category': "ABSTRACTA"},
    'featq': {'umbrella': False, 
          'description': "quantifiable non-count non-mass (inherent) feature",
          'examples': ["tamanho \'size\'", "massa=atómica", "circunferência \'circumference\'", "cumprimento \'length\'"],
          'category': "ABSTRACTA"},                       
    'fhq': {'umbrella': False, 
          'description': "human quantifiable feature",
          'examples': ["tensão=arterial \'blood pressure\'"],
          'category': "ABSTRACTA"},
    'ak': {'umbrella': False, 
          'description': "category",
          'examples': ["mistura \'mixture\'", "modo=maior \'major key\'", "pretérito \'past tense\'"],
          'category': "ABSTRACTA"},                       
    'akss': {'umbrella': False, 
          'description': "speech product category: nonsense, gossip, boast (similar to ss, but not countable)",
          'examples': ["farelório \'chit-chat\'", "galimatias \'gibberish\'", "mexerico \'gossip\'"],
          'category': "ABSTRACTA"},
    'ism': {'umbrella': False, 
          'description': "ideology, religion",
          'examples': ["comunismo \'communism\'"],
          'category': "ABSTRACTA"},                       
    'ling': {'umbrella': False, 
          'description': "language",
          'examples': ["esperanto", "gíria \'slang\'", "inglês \'English\'", "-ês"],
          'category': "ABSTRACTA"},
    'syg': {'umbrella': False, 
          'description': "disease",
          'examples': ["psitacose \'parrot fever\'"],
          'category': "ABSTRACTA"},                       
    'col': {'umbrella': False, 
          'description': "coulour",
          'examples': ["roxo \'violet\'", "rubente \'ruby-red\'"],
          'category': "ABSTRACTA"},
    'CI': {'umbrella': False, 
          'description': "activity (+CONTROL, imperfective -> -PL), often derived from <vi>", 
          'examples': ["cavadela \'digging\'", "circulação \'circulation\'", "boicote \'boycot\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'lud': {'umbrella': False, 
          'description': "game",
          'examples': ["pôquer \'poker\'", "roleta \'roulette\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'sp': {'umbrella': False, 
          'description': "sport",
          'examples': ["badminton", "canoagem \'canoeing\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},                       
    'fag': {'umbrella': False, 
          'description': "subject (to learn), profession (to practice)",
          'examples': ["cardiologia \'cardiology\'", "ciência \'science\'", "culinária \'cookery\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'terapi': {'umbrella': False, 
          'description': "therapy",
          'examples': ["acupunctura \'acupuncture\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},                       
    'dans': {'umbrella': False, 
          'description': "dance (also -> ll)",
          'examples': ["mambo", "polca", "samba"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'tæsk': {'umbrella': False, 
          'description': "beating",
          'examples': ["sova \'beating\'", "surra \'thrashing\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},                       
    'CP': {'umbrella': False, 
          'description': "action (+CONTROL, perfective ), often derived from <vt> (possibly, <ve>)",
          'examples': ["pacificação \'pacification\'", "partida \'departure\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'CPS': {'umbrella': False, 
          'description': "-PL -actions",
          'examples': ["-ação"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'CPP': {'umbrella': False, 
          'description': "+PL -actions",
          'examples': ["tiro \'shot\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'd': {'umbrella': False, 
          'description': "often, but not always deverbal, cp. -> ac (what you do or make)",
          'examples': ["dar uma lavadela \'light washing\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'kneb': {'umbrella': False, 
          'description': "trick, cheat, fraud",
          'examples': ["jeito \'trick\'", "dica \'tip\'", "engano \'cheat\'", "intriga \'intrigue\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'cI': {'umbrella': False, 
          'description': "process (-CONTROL, imperfective -> -PL), often derived from <ve>",
          'examples': ["crescimento \'growth\'", "decaimento \'decay\'", "decomposição \'rotting\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'cP': {'umbrella': False, 
          'description': "event (-CONTROL, perfective)",
          'examples': ["impacto \'impact\'", "nascimento \'birth\'", "ovulação \'ovulation\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'cPS': {'umbrella': False, 
          'description': "-PL -events",
          'examples': ["queda \'fall\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'cPP': {'umbrella': False, 
          'description': "+PL -events",
          'examples': ["explosão \'explosion\'", "boléu \'crash\'", "acidente \'accident\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'snak': {'umbrella': False, 
          'description': "talk",
          'examples': ["debate \'debate\'", "bate-papo \'little talk\'", "discussão \'discussion\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'strid': {'umbrella': False, 
          'description': "fight, quarrel",
          'examples': ["rixa \'row\'", "briga \'fight\'"],
          'category': "ACTIVITY-, ACTION-, PROCESS- AND EVENT-NOUNS (+V [verbality] feature)"},
    'temp': {'umbrella': False, 
          'description': "point in time [can be part of occ, -V event]",
          'examples': ["início \'beginning\'", "instante \'moment\'"],
          'category': "TIME FIELD"},
    'per': {'umbrella': False, 
          'description': "period in time [part of cI/process, time-place]",
          'examples': ["fase \'phase\'", "guerra \'war\'"],
          'category': "TIME FIELD"},                       
    'dur': {'umbrella': False, 
          'description': "measure of time [part of num+, time-unit]",
          'examples': ["hora \'hour\'", "semana \'week\'"],
          'category': "TIME FIELD"},                       
    'occ': {'umbrella': False, 
          'description': "occasion [part of cP/event, +HUM, human place-event]",
          'examples': ["concerto \'concert\'", "guerra \'war\'", "Natal \'Christmas\'"],
          'category': "TIME FIELD"}
}

In [355]:
#n_semantic

#SEMANTIC_TAGSET

new_SEMANTIC_TS = {f"<{k}>":v for k,v in SEMANTIC_TAGSET.items()}

new_SEMANTIC_TS

{'<H>': {'umbrella': True,
  'description': '+HUM-noun, human being -> cf. more specific categories below',
  'examples': ['achacador',
   "'kidnapper'",
   'acionário',
   "'shareholder'",
   'inimigo',
   "'enemy'"],
  'category': 'ANIMATE HUMAN'},
 '<HM>': {'umbrella': False,
  'description': 'mystical or religious entity, constellations in anjo ‘angel’, duende ‘goblin’, astrology',
  'examples': ["anjo 'angel'",
   "duende 'goblin'",
   "hidra 'Hydra'",
   "tauro 'Taurus'"],
  'category': 'ANIMATE HUMAN'},
 '<N>': {'umbrella': False,
  'description': '',
  'examples': ["cigano 'gypsy'", "escandinavo 'Scandinavian'"],
  'category': 'ANIMATE HUMAN'},
 '<prof>': {'umbrella': False,
  'description': 'professional',
  'examples': ["-or: escritor 'author'", "filósofo 'philosopher'"],
  'category': 'ANIMATE HUMAN'},
 '<fam>': {'umbrella': False,
  'description': 'family member',
  'examples': ["pai 'father'", "mãe 'mother'"],
  'category': 'ANIMATE HUMAN'},
 '<title>': {'umbrella': False,

In [356]:
for tag, dic_v in n_semantic.items():    
    if tag not in new_SEMANTIC_TS.keys():
        new_SEMANTIC_TS[tag] = dic_v
    else:
        new_SEMANTIC_TS[tag]['umbrella'] = dic_v['umbrella']
        new_SEMANTIC_TS[tag]['description'] += f" {dic_v['description']} {dic_v['category']}"
                

#print(len(new_SEMANTIC_TS))
#print(len(SEMANTIC_TAGSET))
new_SEMANTIC_TS

{'<H>': {'umbrella': True,
  'description': '+HUM-noun, human being -> cf. more specific categories below Human, umbrella tag Human prototypes',
  'examples': ['achacador',
   "'kidnapper'",
   'acionário',
   "'shareholder'",
   'inimigo',
   "'enemy'"],
  'category': 'ANIMATE HUMAN'},
 '<HM>': {'umbrella': False,
  'description': 'mystical or religious entity, constellations in anjo ‘angel’, duende ‘goblin’, astrology',
  'examples': ["anjo 'angel'",
   "duende 'goblin'",
   "hidra 'Hydra'",
   "tauro 'Taurus'"],
  'category': 'ANIMATE HUMAN'},
 '<N>': {'umbrella': False,
  'description': '',
  'examples': ["cigano 'gypsy'", "escandinavo 'Scandinavian'"],
  'category': 'ANIMATE HUMAN'},
 '<prof>': {'umbrella': False,
  'description': 'professional',
  'examples': ["-or: escritor 'author'", "filósofo 'philosopher'"],
  'category': 'ANIMATE HUMAN'},
 '<fam>': {'umbrella': False,
  'description': 'family member',
  'examples': ["pai 'father'", "mãe 'mother'"],
  'category': 'ANIMATE HUM

In [386]:
desc="Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)"
s1 = """@AS-<ADVL (1959), @AS-<SC (28) , @AS-A< (71) , @AS-ADVL (72) , @AS-ADVL> (737) , @AS-KOMP< (648) , @AS-N< (370) , @FS-<<ACC (17) , @FS-<ACC (5835) , @FS-<ADVS (3) , @FS-<ADVL (4103) , @FS-<OC (1) , @FS-<SC (430) , @FS-<SUBJ (1102) , @FS-A< (195) , @FS-ACC> (24) , @FS-ACC>> (868) , @FS-ADVL (224) , @FS-ADVL> (1536) , @FS-AS< (17) , @FS-KOMP< (410) , @FS-N< (14393) , @FS-P< (1303) , @FS-S< (335) , @FS-SUBJ> (440) , @ICL-<ACC (3970) , @ICL-<ADVL (3232) , @ICL-<OC (23) , @ICL-<PRED (87) , @ICL-<SC (963) , @ICL-<SUBJ (860) , @ICL-ACC> (61) , @ICL-ADVL (113) , @ICL-ADVL> (478) , @ICL-APP (3) , @ICL-AS< (88) , @ICL-AUX< (16862) , @ICL-IMV (3) , @ICL-N< (3172) , @ICL-N<PRED (1101) , @ICL-NPHR (2) , @ICL-P< (11160) , @ICL-PRED> (198) , @ICL-SUBJ> (329) """

l_s = [x.strip() for x in s1.split(',')]
l_s = {x:desc for y in l_s for x in y.split(' ')[::2]}
l_s

#l_s = {m[0]: desc for m in l.strip().split(' ') for l in s1.split(',') }


{'@AS-<ADVL': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
 '@AS-<SC': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
 '@AS-A<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
 '@AS-ADVL': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
 '@AS-ADVL>': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
 '@AS-KOMP<': 'Combinations of the main tags with clausal forms occur (FS = finite subclause, ICL = non-finite clause, AS = averbal clause, old version with @#)',
 '@AS-N<': 'Combinations of 